In [5]:
import numpy as np
import random
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from TicTacToe import TicTacToe

class SQNAgent:
    def __init__(self, state_size=9, action_size=9, gamma=0.95):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = gamma  
        self.learning_rate = 0.001
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.9995
        self.batch_size = 32
        self.replay_buffer = deque(maxlen=10000)
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential([
            Dense(64, input_dim=self.state_size, activation='relu'),
            Dense(64, activation='relu'),
            Dense(self.action_size, activation='linear')
        ])
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def process_state(self, state):
        """
        Converts the state from [0,1,2] format to [-1,0,1] format for better learning.
        """
        processed_state = np.array(state).copy()
        processed_state[processed_state == 0] = -1
        processed_state[processed_state == 1] = 0
        processed_state[processed_state == 2] = 1
        return processed_state

    def select_action(self, state, valid_actions):
        if not valid_actions:
            return None

        if random.random() < self.epsilon:
            return random.choice(valid_actions)
        
        processed_state = self.process_state(state)
        q_values = self.model.predict(processed_state.reshape(1, -1), verbose=0)[0]
        valid_q_values = [(q_values[action], action) for action in valid_actions]
        return max(valid_q_values, key=lambda x: x[0])[1]

    def store_experience(self, state, action, reward, next_state, done):
        self.replay_buffer.append((state, action, reward, next_state, done))

    def train(self):
        if len(self.replay_buffer) < self.batch_size:
            return

        mini_batch = random.sample(self.replay_buffer, self.batch_size)
        states = np.array([self.process_state(exp[0]) for exp in mini_batch])
        next_states = np.array([self.process_state(exp[3]) for exp in mini_batch])
        current_q_values = self.model.predict(states, verbose=0)
        next_q_values = self.model.predict(next_states, verbose=0)

        x = []
        y = []

        for i, (state, action, reward, next_state, done) in enumerate(mini_batch):
            if done:
                target = reward
            else:
                next_valid_actions = [j for j, val in enumerate(next_state) if val == 0]
                if next_valid_actions:
                    max_next_q = max(next_q_values[i][action] for action in next_valid_actions)
                    target = reward + self.gamma * max_next_q
                else:
                    target = reward
            
            current_q = current_q_values[i].copy()
            current_q[action] = target
            x.append(self.process_state(state))
            y.append(current_q)

        self.model.fit(np.array(x), np.array(y), batch_size=32, epochs=1, verbose=0)

def train_agent(episodes=10000):
    agent = SQNAgent()
    history = {'wins': 0, 'losses': 0, 'draws': 0}
    initial_epsilon = 1.0
    min_epsilon = 0.01
    
    
    for episode in range(episodes):
        agent.epsilon=min(min_epsilon,agent.epsilon*agent.epsilon_decay)
        
        smartness = min(1, episode / (episodes * 0.6))
        game = TicTacToe(smartMovePlayer1=smartness)
        state = np.array(game.board)
        
        game.player1_move()
        state = np.array(game.board)
        
        while True:
            valid_actions = game.empty_positions()
            if not valid_actions:
                history['draws'] += 1
                break
                
            action = agent.select_action(state, valid_actions)
            game.make_move(action, player=2)
            
            reward = 0
            if game.current_winner == 2:
                reward = 1.0
                history['wins'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            
            game.player1_move()
            if game.current_winner == 1:
                reward = -1.0
                history['losses'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            elif game.is_full():
                reward = 0.0
                history['draws'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            
            agent.store_experience(state, action, reward, game.board, False)
            state = np.array(game.board)
        
        agent.train()
        
        if episode % 100 == 0:
            win_rate = history['wins'] / (episode + 1)
            print(f"Episode: {episode}, Win Rate: {win_rate:.2f}, Epsilon: {agent.epsilon:.3f}, smartmove{smartness}")
            print(f"Wins: {history['wins']}, Losses: {history['losses']}, Draws: {history['draws']}")
        
        if episode % 1000 == 0:
            agent.model.save(f'model1_episode_{episode}.h5')
    agent.model.save('model1.h5')
agent = train_agent()



Episode: 0, Win Rate: 0.00, Epsilon: 0.010, smartmove0.0
Wins: 0, Losses: 1, Draws: 0
Episode: 100, Win Rate: 0.30, Epsilon: 0.010, smartmove0.016666666666666666
Wins: 30, Losses: 56, Draws: 15
Episode: 200, Win Rate: 0.29, Epsilon: 0.009, smartmove0.03333333333333333
Wins: 58, Losses: 106, Draws: 37
Episode: 300, Win Rate: 0.33, Epsilon: 0.009, smartmove0.05
Wins: 98, Losses: 155, Draws: 48
Episode: 400, Win Rate: 0.32, Epsilon: 0.008, smartmove0.06666666666666667
Wins: 130, Losses: 210, Draws: 61
Episode: 500, Win Rate: 0.34, Epsilon: 0.008, smartmove0.08333333333333333
Wins: 169, Losses: 257, Draws: 75
Episode: 600, Win Rate: 0.34, Epsilon: 0.007, smartmove0.1
Wins: 202, Losses: 315, Draws: 84
Episode: 700, Win Rate: 0.33, Epsilon: 0.007, smartmove0.11666666666666667
Wins: 232, Losses: 371, Draws: 98
Episode: 800, Win Rate: 0.33, Epsilon: 0.007, smartmove0.13333333333333333
Wins: 268, Losses: 431, Draws: 102
Episode: 900, Win Rate: 0.33, Epsilon: 0.006, smartmove0.15
Wins: 297, Loss

Episode: 1000, Win Rate: 0.32, Epsilon: 0.006, smartmove0.16666666666666666
Wins: 324, Losses: 542, Draws: 135
Episode: 1100, Win Rate: 0.32, Epsilon: 0.006, smartmove0.18333333333333332
Wins: 351, Losses: 600, Draws: 150
Episode: 1200, Win Rate: 0.32, Epsilon: 0.005, smartmove0.2
Wins: 384, Losses: 656, Draws: 161
Episode: 1300, Win Rate: 0.32, Epsilon: 0.005, smartmove0.21666666666666667
Wins: 410, Losses: 709, Draws: 182
Episode: 1400, Win Rate: 0.31, Epsilon: 0.005, smartmove0.23333333333333334
Wins: 428, Losses: 776, Draws: 197
Episode: 1500, Win Rate: 0.30, Epsilon: 0.005, smartmove0.25
Wins: 447, Losses: 837, Draws: 217
Episode: 1600, Win Rate: 0.30, Epsilon: 0.004, smartmove0.26666666666666666
Wins: 478, Losses: 888, Draws: 235
Episode: 1700, Win Rate: 0.30, Epsilon: 0.004, smartmove0.2833333333333333
Wins: 503, Losses: 944, Draws: 254
Episode: 1800, Win Rate: 0.30, Epsilon: 0.004, smartmove0.3
Wins: 533, Losses: 997, Draws: 271
Episode: 1900, Win Rate: 0.29, Epsilon: 0.004, sm

Episode: 2000, Win Rate: 0.28, Epsilon: 0.004, smartmove0.3333333333333333
Wins: 566, Losses: 1134, Draws: 301
Episode: 2100, Win Rate: 0.28, Epsilon: 0.003, smartmove0.35
Wins: 594, Losses: 1193, Draws: 314
Episode: 2200, Win Rate: 0.28, Epsilon: 0.003, smartmove0.36666666666666664
Wins: 617, Losses: 1262, Draws: 322
Episode: 2300, Win Rate: 0.28, Epsilon: 0.003, smartmove0.38333333333333336
Wins: 641, Losses: 1320, Draws: 340
Episode: 2400, Win Rate: 0.28, Epsilon: 0.003, smartmove0.4
Wins: 661, Losses: 1377, Draws: 363
Episode: 2500, Win Rate: 0.28, Epsilon: 0.003, smartmove0.4166666666666667
Wins: 690, Losses: 1433, Draws: 378
Episode: 2600, Win Rate: 0.27, Epsilon: 0.003, smartmove0.43333333333333335
Wins: 710, Losses: 1494, Draws: 397
Episode: 2700, Win Rate: 0.27, Epsilon: 0.003, smartmove0.45
Wins: 728, Losses: 1554, Draws: 419
Episode: 2800, Win Rate: 0.27, Epsilon: 0.002, smartmove0.4666666666666667
Wins: 747, Losses: 1611, Draws: 443
Episode: 2900, Win Rate: 0.27, Epsilon: 0

Episode: 3000, Win Rate: 0.27, Epsilon: 0.002, smartmove0.5
Wins: 799, Losses: 1732, Draws: 470
Episode: 3100, Win Rate: 0.27, Epsilon: 0.002, smartmove0.5166666666666667
Wins: 823, Losses: 1790, Draws: 488
Episode: 3200, Win Rate: 0.26, Epsilon: 0.002, smartmove0.5333333333333333
Wins: 845, Losses: 1849, Draws: 507
Episode: 3300, Win Rate: 0.26, Epsilon: 0.002, smartmove0.55
Wins: 861, Losses: 1901, Draws: 539
Episode: 3400, Win Rate: 0.26, Epsilon: 0.002, smartmove0.5666666666666667
Wins: 879, Losses: 1963, Draws: 559
Episode: 3500, Win Rate: 0.26, Epsilon: 0.002, smartmove0.5833333333333334
Wins: 896, Losses: 2018, Draws: 587
Episode: 3600, Win Rate: 0.25, Epsilon: 0.002, smartmove0.6
Wins: 910, Losses: 2084, Draws: 607
Episode: 3700, Win Rate: 0.25, Epsilon: 0.002, smartmove0.6166666666666667
Wins: 920, Losses: 2144, Draws: 637
Episode: 3800, Win Rate: 0.25, Epsilon: 0.001, smartmove0.6333333333333333
Wins: 940, Losses: 2201, Draws: 660
Episode: 3900, Win Rate: 0.24, Epsilon: 0.001

Episode: 4000, Win Rate: 0.24, Epsilon: 0.001, smartmove0.6666666666666666
Wins: 966, Losses: 2322, Draws: 713
Episode: 4100, Win Rate: 0.24, Epsilon: 0.001, smartmove0.6833333333333333
Wins: 983, Losses: 2378, Draws: 740
Episode: 4200, Win Rate: 0.24, Epsilon: 0.001, smartmove0.7
Wins: 995, Losses: 2438, Draws: 768
Episode: 4300, Win Rate: 0.23, Epsilon: 0.001, smartmove0.7166666666666667
Wins: 1009, Losses: 2495, Draws: 797
Episode: 4400, Win Rate: 0.23, Epsilon: 0.001, smartmove0.7333333333333333
Wins: 1026, Losses: 2543, Draws: 832
Episode: 4500, Win Rate: 0.23, Epsilon: 0.001, smartmove0.75
Wins: 1040, Losses: 2590, Draws: 871
Episode: 4600, Win Rate: 0.23, Epsilon: 0.001, smartmove0.7666666666666667
Wins: 1054, Losses: 2643, Draws: 904
Episode: 4700, Win Rate: 0.23, Epsilon: 0.001, smartmove0.7833333333333333
Wins: 1066, Losses: 2708, Draws: 927
Episode: 4800, Win Rate: 0.22, Epsilon: 0.001, smartmove0.8
Wins: 1079, Losses: 2758, Draws: 964
Episode: 4900, Win Rate: 0.22, Epsilon:

Episode: 5000, Win Rate: 0.22, Epsilon: 0.001, smartmove0.8333333333333334
Wins: 1106, Losses: 2876, Draws: 1019
Episode: 5100, Win Rate: 0.22, Epsilon: 0.001, smartmove0.85
Wins: 1111, Losses: 2937, Draws: 1053
Episode: 5200, Win Rate: 0.22, Epsilon: 0.001, smartmove0.8666666666666667
Wins: 1120, Losses: 3001, Draws: 1080
Episode: 5300, Win Rate: 0.21, Epsilon: 0.001, smartmove0.8833333333333333
Wins: 1133, Losses: 3058, Draws: 1110
Episode: 5400, Win Rate: 0.21, Epsilon: 0.001, smartmove0.9
Wins: 1138, Losses: 3117, Draws: 1146
Episode: 5500, Win Rate: 0.21, Epsilon: 0.001, smartmove0.9166666666666666
Wins: 1149, Losses: 3178, Draws: 1174
Episode: 5600, Win Rate: 0.21, Epsilon: 0.001, smartmove0.9333333333333333
Wins: 1156, Losses: 3243, Draws: 1202
Episode: 5700, Win Rate: 0.20, Epsilon: 0.001, smartmove0.95
Wins: 1162, Losses: 3310, Draws: 1229
Episode: 5800, Win Rate: 0.20, Epsilon: 0.001, smartmove0.9666666666666667
Wins: 1164, Losses: 3382, Draws: 1255
Episode: 5900, Win Rate: 0

Episode: 6000, Win Rate: 0.19, Epsilon: 0.000, smartmove1
Wins: 1168, Losses: 3531, Draws: 1302
Episode: 6100, Win Rate: 0.19, Epsilon: 0.000, smartmove1
Wins: 1173, Losses: 3587, Draws: 1341
Episode: 6200, Win Rate: 0.19, Epsilon: 0.000, smartmove1
Wins: 1174, Losses: 3653, Draws: 1374
Episode: 6300, Win Rate: 0.19, Epsilon: 0.000, smartmove1
Wins: 1179, Losses: 3713, Draws: 1409
Episode: 6400, Win Rate: 0.18, Epsilon: 0.000, smartmove1
Wins: 1183, Losses: 3779, Draws: 1439
Episode: 6500, Win Rate: 0.18, Epsilon: 0.000, smartmove1
Wins: 1188, Losses: 3846, Draws: 1467
Episode: 6600, Win Rate: 0.18, Epsilon: 0.000, smartmove1
Wins: 1193, Losses: 3913, Draws: 1495
Episode: 6700, Win Rate: 0.18, Epsilon: 0.000, smartmove1
Wins: 1196, Losses: 3972, Draws: 1533
Episode: 6800, Win Rate: 0.18, Epsilon: 0.000, smartmove1
Wins: 1201, Losses: 4028, Draws: 1572
Episode: 6900, Win Rate: 0.17, Epsilon: 0.000, smartmove1
Wins: 1204, Losses: 4095, Draws: 1602


Episode: 7000, Win Rate: 0.17, Epsilon: 0.000, smartmove1
Wins: 1206, Losses: 4150, Draws: 1645
Episode: 7100, Win Rate: 0.17, Epsilon: 0.000, smartmove1
Wins: 1210, Losses: 4214, Draws: 1677
Episode: 7200, Win Rate: 0.17, Epsilon: 0.000, smartmove1
Wins: 1220, Losses: 4273, Draws: 1708
Episode: 7300, Win Rate: 0.17, Epsilon: 0.000, smartmove1
Wins: 1227, Losses: 4332, Draws: 1742
Episode: 7400, Win Rate: 0.17, Epsilon: 0.000, smartmove1
Wins: 1232, Losses: 4390, Draws: 1779
Episode: 7500, Win Rate: 0.16, Epsilon: 0.000, smartmove1
Wins: 1234, Losses: 4460, Draws: 1807
Episode: 7600, Win Rate: 0.16, Epsilon: 0.000, smartmove1
Wins: 1234, Losses: 4540, Draws: 1827
Episode: 7700, Win Rate: 0.16, Epsilon: 0.000, smartmove1
Wins: 1238, Losses: 4603, Draws: 1860
Episode: 7800, Win Rate: 0.16, Epsilon: 0.000, smartmove1
Wins: 1242, Losses: 4677, Draws: 1882
Episode: 7900, Win Rate: 0.16, Epsilon: 0.000, smartmove1
Wins: 1243, Losses: 4746, Draws: 1912


Episode: 8000, Win Rate: 0.16, Epsilon: 0.000, smartmove1
Wins: 1247, Losses: 4810, Draws: 1944
Episode: 8100, Win Rate: 0.15, Epsilon: 0.000, smartmove1
Wins: 1251, Losses: 4883, Draws: 1967
Episode: 8200, Win Rate: 0.15, Epsilon: 0.000, smartmove1
Wins: 1254, Losses: 4947, Draws: 2000
Episode: 8300, Win Rate: 0.15, Epsilon: 0.000, smartmove1
Wins: 1264, Losses: 5013, Draws: 2024
Episode: 8400, Win Rate: 0.15, Epsilon: 0.000, smartmove1
Wins: 1271, Losses: 5078, Draws: 2052
Episode: 8500, Win Rate: 0.15, Epsilon: 0.000, smartmove1
Wins: 1279, Losses: 5146, Draws: 2076
Episode: 8600, Win Rate: 0.15, Epsilon: 0.000, smartmove1
Wins: 1283, Losses: 5211, Draws: 2107
Episode: 8700, Win Rate: 0.15, Epsilon: 0.000, smartmove1
Wins: 1286, Losses: 5285, Draws: 2130
Episode: 8800, Win Rate: 0.15, Epsilon: 0.000, smartmove1
Wins: 1290, Losses: 5354, Draws: 2157
Episode: 8900, Win Rate: 0.15, Epsilon: 0.000, smartmove1
Wins: 1293, Losses: 5425, Draws: 2183


Episode: 9000, Win Rate: 0.14, Epsilon: 0.000, smartmove1
Wins: 1300, Losses: 5498, Draws: 2203
Episode: 9100, Win Rate: 0.14, Epsilon: 0.000, smartmove1
Wins: 1308, Losses: 5572, Draws: 2221
Episode: 9200, Win Rate: 0.14, Epsilon: 0.000, smartmove1
Wins: 1315, Losses: 5638, Draws: 2248
Episode: 9300, Win Rate: 0.14, Epsilon: 0.000, smartmove1
Wins: 1324, Losses: 5706, Draws: 2271
Episode: 9400, Win Rate: 0.14, Epsilon: 0.000, smartmove1
Wins: 1330, Losses: 5778, Draws: 2293
Episode: 9500, Win Rate: 0.14, Epsilon: 0.000, smartmove1
Wins: 1336, Losses: 5842, Draws: 2323
Episode: 9600, Win Rate: 0.14, Epsilon: 0.000, smartmove1
Wins: 1341, Losses: 5913, Draws: 2347
Episode: 9700, Win Rate: 0.14, Epsilon: 0.000, smartmove1
Wins: 1347, Losses: 5972, Draws: 2382
Episode: 9800, Win Rate: 0.14, Epsilon: 0.000, smartmove1
Wins: 1351, Losses: 6049, Draws: 2401
Episode: 9900, Win Rate: 0.14, Epsilon: 0.000, smartmove1
Wins: 1356, Losses: 6122, Draws: 2423


In [7]:
import numpy as np
import random
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from TicTacToe import TicTacToe

class SQNAgent:
    def __init__(self, state_size=9, action_size=9, gamma=0.95):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = gamma  
        self.learning_rate = 0.001
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.9995
        self.batch_size = 32
        self.replay_buffer = deque(maxlen=10000)
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential([
            Dense(64, input_dim=self.state_size, activation='relu'),
            Dense(64, activation='relu'),
            Dense(self.action_size, activation='linear')
        ])
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def process_state(self, state):
        """
        Converts the state from [0,1,2] format to [-1,0,1] format for better learning.
        """
        processed_state = np.array(state).copy()
        processed_state[processed_state == 0] = -1
        processed_state[processed_state == 1] = 0
        processed_state[processed_state == 2] = 1
        return processed_state

    def select_action(self, state, valid_actions):
        if not valid_actions:
            return None

        if random.random() < self.epsilon:
            return random.choice(valid_actions)
        
        processed_state = self.process_state(state)
        q_values = self.model.predict(processed_state.reshape(1, -1), verbose=0)[0]
        valid_q_values = [(q_values[action], action) for action in valid_actions]
        return max(valid_q_values, key=lambda x: x[0])[1]

    def store_experience(self, state, action, reward, next_state, done):
        self.replay_buffer.append((state, action, reward, next_state, done))

    def train(self):
        if len(self.replay_buffer) < self.batch_size:
            return

        mini_batch = random.sample(self.replay_buffer, self.batch_size)
        states = np.array([self.process_state(exp[0]) for exp in mini_batch])
        next_states = np.array([self.process_state(exp[3]) for exp in mini_batch])
        current_q_values = self.model.predict(states, verbose=0)
        next_q_values = self.model.predict(next_states, verbose=0)

        x = []
        y = []

        for i, (state, action, reward, next_state, done) in enumerate(mini_batch):
            if done:
                target = reward
            else:
                next_valid_actions = [j for j, val in enumerate(next_state) if val == 0]
                if next_valid_actions:
                    max_next_q = max(next_q_values[i][action] for action in next_valid_actions)
                    target = reward + self.gamma * max_next_q
                else:
                    target = reward
            
            current_q = current_q_values[i].copy()
            current_q[action] = target
            x.append(self.process_state(state))
            y.append(current_q)

        self.model.fit(np.array(x), np.array(y), batch_size=32, epochs=1, verbose=0)

def train_agent(episodes=10000):
    agent = SQNAgent()
    history = {'wins': 0, 'losses': 0, 'draws': 0}
    initial_epsilon = 1.0
    min_epsilon = 0.01
    decay_episodes = 1000
    
    for episode in range(episodes):
        if episode % 2000 == 0 and episode > 0:
            agent.epsilon +=0.4
            print("Increasing Epsillion")
        else:
            epsilon_decay = (initial_epsilon - min_epsilon) / decay_episodes
            agent.epsilon = max(min_epsilon, initial_epsilon - (episode % 2000) * epsilon_decay)
        
        smartness = min(1, episode / (episodes * 0.6))
        game = TicTacToe(smartMovePlayer1=smartness)
        state = np.array(game.board)
        
        game.player1_move()
        state = np.array(game.board)
        
        while True:
            valid_actions = game.empty_positions()
            if not valid_actions:
                history['draws'] += 1
                break
                
            action = agent.select_action(state, valid_actions)
            game.make_move(action, player=2)
            
            reward = 0
            if game.current_winner == 2:
                reward = 1.0
                history['wins'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            
            game.player1_move()
            if game.current_winner == 1:
                reward = -1.0
                history['losses'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            elif game.is_full():
                reward = 0.0
                history['draws'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            
            agent.store_experience(state, action, reward, game.board, False)
            state = np.array(game.board)
        
        agent.train()
        
        if episode % 100 == 0:
            win_rate = history['wins'] / (episode + 1)
            print(f"Episode: {episode}, Win Rate: {win_rate:.2f}, Epsilon: {agent.epsilon:.3f}, smartmove{smartness}")
            print(f"Wins: {history['wins']}, Losses: {history['losses']}, Draws: {history['draws']}")
        
        if episode % 1000 == 0:
            agent.model.save(f'model2_episode_{episode}.h5')
    agent.model.save('model2.h5')
agent = train_agent()



Episode: 0, Win Rate: 0.00, Epsilon: 1.000, smartmove0.0
Wins: 0, Losses: 1, Draws: 0
Episode: 100, Win Rate: 0.29, Epsilon: 0.901, smartmove0.016666666666666666
Wins: 29, Losses: 57, Draws: 15
Episode: 200, Win Rate: 0.28, Epsilon: 0.802, smartmove0.03333333333333333
Wins: 57, Losses: 116, Draws: 28
Episode: 300, Win Rate: 0.28, Epsilon: 0.703, smartmove0.05
Wins: 85, Losses: 180, Draws: 36
Episode: 400, Win Rate: 0.30, Epsilon: 0.604, smartmove0.06666666666666667
Wins: 120, Losses: 238, Draws: 43
Episode: 500, Win Rate: 0.30, Epsilon: 0.505, smartmove0.08333333333333333
Wins: 152, Losses: 297, Draws: 52
Episode: 600, Win Rate: 0.32, Epsilon: 0.406, smartmove0.1
Wins: 192, Losses: 350, Draws: 59
Episode: 700, Win Rate: 0.32, Epsilon: 0.307, smartmove0.11666666666666667
Wins: 226, Losses: 408, Draws: 67
Episode: 800, Win Rate: 0.31, Epsilon: 0.208, smartmove0.13333333333333333
Wins: 252, Losses: 471, Draws: 78
Episode: 900, Win Rate: 0.31, Epsilon: 0.109, smartmove0.15
Wins: 282, Losse

Episode: 1000, Win Rate: 0.32, Epsilon: 0.010, smartmove0.16666666666666666
Wins: 320, Losses: 583, Draws: 98
Episode: 1100, Win Rate: 0.32, Epsilon: 0.010, smartmove0.18333333333333332
Wins: 352, Losses: 643, Draws: 106
Episode: 1200, Win Rate: 0.32, Epsilon: 0.010, smartmove0.2
Wins: 382, Losses: 703, Draws: 116
Episode: 1300, Win Rate: 0.31, Epsilon: 0.010, smartmove0.21666666666666667
Wins: 409, Losses: 765, Draws: 127
Episode: 1400, Win Rate: 0.32, Epsilon: 0.010, smartmove0.23333333333333334
Wins: 444, Losses: 818, Draws: 139
Episode: 1500, Win Rate: 0.32, Epsilon: 0.010, smartmove0.25
Wins: 482, Losses: 874, Draws: 145
Episode: 1600, Win Rate: 0.32, Epsilon: 0.010, smartmove0.26666666666666666
Wins: 515, Losses: 936, Draws: 150
Episode: 1700, Win Rate: 0.32, Epsilon: 0.010, smartmove0.2833333333333333
Wins: 538, Losses: 1001, Draws: 162
Episode: 1800, Win Rate: 0.31, Epsilon: 0.010, smartmove0.3
Wins: 563, Losses: 1064, Draws: 174
Episode: 1900, Win Rate: 0.31, Epsilon: 0.010, s

Increasing Epsillion
Episode: 2000, Win Rate: 0.31, Epsilon: 0.410, smartmove0.3333333333333333
Wins: 625, Losses: 1187, Draws: 189
Episode: 2100, Win Rate: 0.31, Epsilon: 0.901, smartmove0.35
Wins: 651, Losses: 1253, Draws: 197
Episode: 2200, Win Rate: 0.30, Epsilon: 0.802, smartmove0.36666666666666664
Wins: 668, Losses: 1322, Draws: 211
Episode: 2300, Win Rate: 0.30, Epsilon: 0.703, smartmove0.38333333333333336
Wins: 690, Losses: 1387, Draws: 224
Episode: 2400, Win Rate: 0.30, Epsilon: 0.604, smartmove0.4
Wins: 712, Losses: 1458, Draws: 231
Episode: 2500, Win Rate: 0.30, Epsilon: 0.505, smartmove0.4166666666666667
Wins: 739, Losses: 1520, Draws: 242
Episode: 2600, Win Rate: 0.29, Epsilon: 0.406, smartmove0.43333333333333335
Wins: 765, Losses: 1587, Draws: 249
Episode: 2700, Win Rate: 0.29, Epsilon: 0.307, smartmove0.45
Wins: 787, Losses: 1656, Draws: 258
Episode: 2800, Win Rate: 0.29, Epsilon: 0.208, smartmove0.4666666666666667
Wins: 814, Losses: 1722, Draws: 265
Episode: 2900, Win R

Episode: 3000, Win Rate: 0.28, Epsilon: 0.010, smartmove0.5
Wins: 854, Losses: 1862, Draws: 285
Episode: 3100, Win Rate: 0.28, Epsilon: 0.010, smartmove0.5166666666666667
Wins: 873, Losses: 1933, Draws: 295
Episode: 3200, Win Rate: 0.28, Epsilon: 0.010, smartmove0.5333333333333333
Wins: 904, Losses: 1999, Draws: 298
Episode: 3300, Win Rate: 0.28, Epsilon: 0.010, smartmove0.55
Wins: 936, Losses: 2055, Draws: 310
Episode: 3400, Win Rate: 0.28, Epsilon: 0.010, smartmove0.5666666666666667
Wins: 947, Losses: 2129, Draws: 325
Episode: 3500, Win Rate: 0.27, Epsilon: 0.010, smartmove0.5833333333333334
Wins: 961, Losses: 2205, Draws: 335
Episode: 3600, Win Rate: 0.27, Epsilon: 0.010, smartmove0.6
Wins: 986, Losses: 2260, Draws: 355
Episode: 3700, Win Rate: 0.27, Epsilon: 0.010, smartmove0.6166666666666667
Wins: 1013, Losses: 2322, Draws: 366
Episode: 3800, Win Rate: 0.27, Epsilon: 0.010, smartmove0.6333333333333333
Wins: 1034, Losses: 2383, Draws: 384
Episode: 3900, Win Rate: 0.27, Epsilon: 0.0

Increasing Epsillion
Episode: 4000, Win Rate: 0.27, Epsilon: 0.410, smartmove0.6666666666666666
Wins: 1073, Losses: 2514, Draws: 414
Episode: 4100, Win Rate: 0.26, Epsilon: 0.901, smartmove0.6833333333333333
Wins: 1076, Losses: 2598, Draws: 427
Episode: 4200, Win Rate: 0.26, Epsilon: 0.802, smartmove0.7
Wins: 1086, Losses: 2675, Draws: 440
Episode: 4300, Win Rate: 0.25, Epsilon: 0.703, smartmove0.7166666666666667
Wins: 1094, Losses: 2760, Draws: 447
Episode: 4400, Win Rate: 0.25, Epsilon: 0.604, smartmove0.7333333333333333
Wins: 1108, Losses: 2833, Draws: 460
Episode: 4500, Win Rate: 0.25, Epsilon: 0.505, smartmove0.75
Wins: 1120, Losses: 2913, Draws: 468
Episode: 4600, Win Rate: 0.25, Epsilon: 0.406, smartmove0.7666666666666667
Wins: 1128, Losses: 2986, Draws: 487
Episode: 4700, Win Rate: 0.24, Epsilon: 0.307, smartmove0.7833333333333333
Wins: 1140, Losses: 3056, Draws: 505
Episode: 4800, Win Rate: 0.24, Epsilon: 0.208, smartmove0.8
Wins: 1160, Losses: 3114, Draws: 527
Episode: 4900, 

Episode: 5000, Win Rate: 0.24, Epsilon: 0.010, smartmove0.8333333333333334
Wins: 1187, Losses: 3246, Draws: 568
Episode: 5100, Win Rate: 0.24, Epsilon: 0.010, smartmove0.85
Wins: 1203, Losses: 3299, Draws: 599
Episode: 5200, Win Rate: 0.23, Epsilon: 0.010, smartmove0.8666666666666667
Wins: 1218, Losses: 3365, Draws: 618
Episode: 5300, Win Rate: 0.23, Epsilon: 0.010, smartmove0.8833333333333333
Wins: 1234, Losses: 3420, Draws: 647
Episode: 5400, Win Rate: 0.23, Epsilon: 0.010, smartmove0.9
Wins: 1251, Losses: 3475, Draws: 675
Episode: 5500, Win Rate: 0.23, Epsilon: 0.010, smartmove0.9166666666666666
Wins: 1267, Losses: 3519, Draws: 715
Episode: 5600, Win Rate: 0.23, Epsilon: 0.010, smartmove0.9333333333333333
Wins: 1281, Losses: 3575, Draws: 745
Episode: 5700, Win Rate: 0.23, Epsilon: 0.010, smartmove0.95
Wins: 1292, Losses: 3632, Draws: 777
Episode: 5800, Win Rate: 0.23, Epsilon: 0.010, smartmove0.9666666666666667
Wins: 1307, Losses: 3687, Draws: 807
Episode: 5900, Win Rate: 0.22, Epsi

Increasing Epsillion
Episode: 6000, Win Rate: 0.22, Epsilon: 0.410, smartmove1
Wins: 1323, Losses: 3795, Draws: 883
Episode: 6100, Win Rate: 0.22, Epsilon: 0.901, smartmove1
Wins: 1324, Losses: 3881, Draws: 896
Episode: 6200, Win Rate: 0.21, Epsilon: 0.802, smartmove1
Wins: 1325, Losses: 3967, Draws: 909
Episode: 6300, Win Rate: 0.21, Epsilon: 0.703, smartmove1
Wins: 1328, Losses: 4053, Draws: 920
Episode: 6400, Win Rate: 0.21, Epsilon: 0.604, smartmove1
Wins: 1329, Losses: 4138, Draws: 934
Episode: 6500, Win Rate: 0.21, Epsilon: 0.505, smartmove1
Wins: 1333, Losses: 4216, Draws: 952
Episode: 6600, Win Rate: 0.20, Epsilon: 0.406, smartmove1
Wins: 1336, Losses: 4291, Draws: 974
Episode: 6700, Win Rate: 0.20, Epsilon: 0.307, smartmove1
Wins: 1337, Losses: 4369, Draws: 995
Episode: 6800, Win Rate: 0.20, Epsilon: 0.208, smartmove1
Wins: 1344, Losses: 4436, Draws: 1021
Episode: 6900, Win Rate: 0.20, Epsilon: 0.109, smartmove1
Wins: 1348, Losses: 4497, Draws: 1056


Episode: 7000, Win Rate: 0.19, Epsilon: 0.010, smartmove1
Wins: 1352, Losses: 4557, Draws: 1092
Episode: 7100, Win Rate: 0.19, Epsilon: 0.010, smartmove1
Wins: 1360, Losses: 4604, Draws: 1137
Episode: 7200, Win Rate: 0.19, Epsilon: 0.010, smartmove1
Wins: 1364, Losses: 4655, Draws: 1182
Episode: 7300, Win Rate: 0.19, Epsilon: 0.010, smartmove1
Wins: 1371, Losses: 4698, Draws: 1232
Episode: 7400, Win Rate: 0.19, Epsilon: 0.010, smartmove1
Wins: 1378, Losses: 4749, Draws: 1274
Episode: 7500, Win Rate: 0.18, Epsilon: 0.010, smartmove1
Wins: 1382, Losses: 4789, Draws: 1330
Episode: 7600, Win Rate: 0.18, Epsilon: 0.010, smartmove1
Wins: 1388, Losses: 4828, Draws: 1385
Episode: 7700, Win Rate: 0.18, Epsilon: 0.010, smartmove1
Wins: 1397, Losses: 4876, Draws: 1428
Episode: 7800, Win Rate: 0.18, Epsilon: 0.010, smartmove1
Wins: 1405, Losses: 4921, Draws: 1475
Episode: 7900, Win Rate: 0.18, Epsilon: 0.010, smartmove1
Wins: 1409, Losses: 4961, Draws: 1531


Increasing Epsillion
Episode: 8000, Win Rate: 0.18, Epsilon: 0.410, smartmove1
Wins: 1412, Losses: 5006, Draws: 1583
Episode: 8100, Win Rate: 0.17, Epsilon: 0.901, smartmove1
Wins: 1413, Losses: 5095, Draws: 1593
Episode: 8200, Win Rate: 0.17, Epsilon: 0.802, smartmove1
Wins: 1413, Losses: 5182, Draws: 1606
Episode: 8300, Win Rate: 0.17, Epsilon: 0.703, smartmove1
Wins: 1415, Losses: 5263, Draws: 1623
Episode: 8400, Win Rate: 0.17, Epsilon: 0.604, smartmove1
Wins: 1415, Losses: 5350, Draws: 1636
Episode: 8500, Win Rate: 0.17, Epsilon: 0.505, smartmove1
Wins: 1416, Losses: 5436, Draws: 1649
Episode: 8600, Win Rate: 0.16, Epsilon: 0.406, smartmove1
Wins: 1418, Losses: 5512, Draws: 1671
Episode: 8700, Win Rate: 0.16, Epsilon: 0.307, smartmove1
Wins: 1421, Losses: 5574, Draws: 1706
Episode: 8800, Win Rate: 0.16, Epsilon: 0.208, smartmove1
Wins: 1423, Losses: 5633, Draws: 1745
Episode: 8900, Win Rate: 0.16, Epsilon: 0.109, smartmove1
Wins: 1429, Losses: 5686, Draws: 1786


Episode: 9000, Win Rate: 0.16, Epsilon: 0.010, smartmove1
Wins: 1433, Losses: 5728, Draws: 1840
Episode: 9100, Win Rate: 0.16, Epsilon: 0.010, smartmove1
Wins: 1439, Losses: 5756, Draws: 1906
Episode: 9200, Win Rate: 0.16, Epsilon: 0.010, smartmove1
Wins: 1445, Losses: 5799, Draws: 1957
Episode: 9300, Win Rate: 0.16, Epsilon: 0.010, smartmove1
Wins: 1453, Losses: 5836, Draws: 2012
Episode: 9400, Win Rate: 0.16, Epsilon: 0.010, smartmove1
Wins: 1459, Losses: 5868, Draws: 2074
Episode: 9500, Win Rate: 0.15, Epsilon: 0.010, smartmove1
Wins: 1464, Losses: 5909, Draws: 2128
Episode: 9600, Win Rate: 0.15, Epsilon: 0.010, smartmove1
Wins: 1469, Losses: 5948, Draws: 2184
Episode: 9700, Win Rate: 0.15, Epsilon: 0.010, smartmove1
Wins: 1477, Losses: 5973, Draws: 2251
Episode: 9800, Win Rate: 0.15, Epsilon: 0.010, smartmove1
Wins: 1482, Losses: 6005, Draws: 2314
Episode: 9900, Win Rate: 0.15, Epsilon: 0.010, smartmove1
Wins: 1492, Losses: 6042, Draws: 2367


In [8]:
import numpy as np
import random
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from TicTacToe import TicTacToe

class SQNAgent:
    def __init__(self, state_size=9, action_size=9, gamma=0.95):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = gamma  
        self.learning_rate = 0.001
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.9995
        self.batch_size = 32
        self.replay_buffer = deque(maxlen=10000)
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential([
            Dense(64, input_dim=self.state_size, activation='relu'),
            Dense(64, activation='relu'),
            Dense(self.action_size, activation='linear')
        ])
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def process_state(self, state):
        """
        Converts the state from [0,1,2] format to [-1,0,1] format for better learning.
        """
        processed_state = np.array(state).copy()
        processed_state[processed_state == 0] = -1
        processed_state[processed_state == 1] = 0
        processed_state[processed_state == 2] = 1
        return processed_state

    def select_action(self, state, valid_actions):
        if not valid_actions:
            return None

        if random.random() < self.epsilon:
            return random.choice(valid_actions)
        
        processed_state = self.process_state(state)
        q_values = self.model.predict(processed_state.reshape(1, -1), verbose=0)[0]
        valid_q_values = [(q_values[action], action) for action in valid_actions]
        return max(valid_q_values, key=lambda x: x[0])[1]

    def store_experience(self, state, action, reward, next_state, done):
        self.replay_buffer.append((state, action, reward, next_state, done))

    def train(self):
        if len(self.replay_buffer) < self.batch_size:
            return

        mini_batch = random.sample(self.replay_buffer, self.batch_size)
        states = np.array([self.process_state(exp[0]) for exp in mini_batch])
        next_states = np.array([self.process_state(exp[3]) for exp in mini_batch])
        current_q_values = self.model.predict(states, verbose=0)
        next_q_values = self.model.predict(next_states, verbose=0)

        x = []
        y = []

        for i, (state, action, reward, next_state, done) in enumerate(mini_batch):
            if done:
                target = reward
            else:
                next_valid_actions = [j for j, val in enumerate(next_state) if val == 0]
                if next_valid_actions:
                    max_next_q = max(next_q_values[i][action] for action in next_valid_actions)
                    target = reward + self.gamma * max_next_q
                else:
                    target = reward
            
            current_q = current_q_values[i].copy()
            current_q[action] = target
            x.append(self.process_state(state))
            y.append(current_q)

        self.model.fit(np.array(x), np.array(y), batch_size=32, epochs=1, verbose=0)

def train_agent(episodes=10000):
    agent = SQNAgent()
    history = {'wins': 0, 'losses': 0, 'draws': 0}
    initial_epsilon = 1.0
    min_epsilon = 0.01
    decay_episodes = 1000
    
    for episode in range(episodes):
        if episode % 2000 == 0 and episode > 0:
            agent.epsilon = 1.0
            print("Resetting epsilon to 1.0")
        else:
            epsilon_decay = (initial_epsilon - min_epsilon) / decay_episodes
            agent.epsilon = max(min_epsilon, initial_epsilon - (episode % 2000) * epsilon_decay)
        
        smartness = min(1, episode / (episodes * 0.6))
        game = TicTacToe(smartMovePlayer1=smartness)
        state = np.array(game.board)
        
        game.player1_move()
        state = np.array(game.board)
        
        while True:
            valid_actions = game.empty_positions()
            if not valid_actions:
                history['draws'] += 1
                break
                
            action = agent.select_action(state, valid_actions)
            game.make_move(action, player=2)
            
            reward = 0
            if game.current_winner == 2:
                reward = 1.0
                history['wins'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            
            game.player1_move()
            if game.current_winner == 1:
                reward = -1.0
                history['losses'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            elif game.is_full():
                reward = 0.0
                history['draws'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            
            agent.store_experience(state, action, reward, game.board, False)
            state = np.array(game.board)
        
        agent.train()
        
        if episode % 100 == 0:
            win_rate = history['wins'] / (episode + 1)
            print(f"Episode: {episode}, Win Rate: {win_rate:.2f}, Epsilon: {agent.epsilon:.3f}, smartmove{smartness}")
            print(f"Wins: {history['wins']}, Losses: {history['losses']}, Draws: {history['draws']}")
        
        if episode % 1000 == 0:
            agent.model.save(f'model3_episode_{episode}.h5')
    agent.model.save('model3.h5')
agent = train_agent()



Episode: 0, Win Rate: 1.00, Epsilon: 1.000, smartmove0.0
Wins: 1, Losses: 0, Draws: 0
Episode: 100, Win Rate: 0.26, Epsilon: 0.901, smartmove0.016666666666666666
Wins: 26, Losses: 59, Draws: 16
Episode: 200, Win Rate: 0.27, Epsilon: 0.802, smartmove0.03333333333333333
Wins: 54, Losses: 120, Draws: 27
Episode: 300, Win Rate: 0.27, Epsilon: 0.703, smartmove0.05
Wins: 82, Losses: 179, Draws: 40
Episode: 400, Win Rate: 0.29, Epsilon: 0.604, smartmove0.06666666666666667
Wins: 115, Losses: 239, Draws: 47
Episode: 500, Win Rate: 0.29, Epsilon: 0.505, smartmove0.08333333333333333
Wins: 146, Losses: 296, Draws: 59
Episode: 600, Win Rate: 0.29, Epsilon: 0.406, smartmove0.1
Wins: 177, Losses: 358, Draws: 66
Episode: 700, Win Rate: 0.30, Epsilon: 0.307, smartmove0.11666666666666667
Wins: 207, Losses: 422, Draws: 72
Episode: 800, Win Rate: 0.30, Epsilon: 0.208, smartmove0.13333333333333333
Wins: 244, Losses: 477, Draws: 80
Episode: 900, Win Rate: 0.31, Epsilon: 0.109, smartmove0.15
Wins: 283, Losse

Episode: 1000, Win Rate: 0.32, Epsilon: 0.010, smartmove0.16666666666666666
Wins: 325, Losses: 581, Draws: 95
Episode: 1100, Win Rate: 0.33, Epsilon: 0.010, smartmove0.18333333333333332
Wins: 364, Losses: 632, Draws: 105
Episode: 1200, Win Rate: 0.33, Epsilon: 0.010, smartmove0.2
Wins: 398, Losses: 691, Draws: 112
Episode: 1300, Win Rate: 0.33, Epsilon: 0.010, smartmove0.21666666666666667
Wins: 430, Losses: 754, Draws: 117
Episode: 1400, Win Rate: 0.33, Epsilon: 0.010, smartmove0.23333333333333334
Wins: 459, Losses: 815, Draws: 127
Episode: 1500, Win Rate: 0.33, Epsilon: 0.010, smartmove0.25
Wins: 495, Losses: 865, Draws: 141
Episode: 1600, Win Rate: 0.33, Epsilon: 0.010, smartmove0.26666666666666666
Wins: 524, Losses: 924, Draws: 153
Episode: 1700, Win Rate: 0.33, Epsilon: 0.010, smartmove0.2833333333333333
Wins: 564, Losses: 973, Draws: 164
Episode: 1800, Win Rate: 0.33, Epsilon: 0.010, smartmove0.3
Wins: 603, Losses: 1023, Draws: 175
Episode: 1900, Win Rate: 0.34, Epsilon: 0.010, sm

Resetting epsilon to 1.0
Episode: 2000, Win Rate: 0.34, Epsilon: 1.000, smartmove0.3333333333333333
Wins: 673, Losses: 1139, Draws: 189
Episode: 2100, Win Rate: 0.33, Epsilon: 0.901, smartmove0.35
Wins: 698, Losses: 1210, Draws: 193
Episode: 2200, Win Rate: 0.33, Epsilon: 0.802, smartmove0.36666666666666664
Wins: 716, Losses: 1276, Draws: 209
Episode: 2300, Win Rate: 0.32, Epsilon: 0.703, smartmove0.38333333333333336
Wins: 730, Losses: 1351, Draws: 220
Episode: 2400, Win Rate: 0.31, Epsilon: 0.604, smartmove0.4
Wins: 746, Losses: 1426, Draws: 229
Episode: 2500, Win Rate: 0.31, Epsilon: 0.505, smartmove0.4166666666666667
Wins: 772, Losses: 1488, Draws: 241
Episode: 2600, Win Rate: 0.31, Epsilon: 0.406, smartmove0.43333333333333335
Wins: 797, Losses: 1557, Draws: 247
Episode: 2700, Win Rate: 0.30, Epsilon: 0.307, smartmove0.45
Wins: 819, Losses: 1630, Draws: 252
Episode: 2800, Win Rate: 0.30, Epsilon: 0.208, smartmove0.4666666666666667
Wins: 846, Losses: 1698, Draws: 257
Episode: 2900, W

Episode: 3000, Win Rate: 0.30, Epsilon: 0.010, smartmove0.5
Wins: 901, Losses: 1822, Draws: 278
Episode: 3100, Win Rate: 0.30, Epsilon: 0.010, smartmove0.5166666666666667
Wins: 916, Losses: 1899, Draws: 286
Episode: 3200, Win Rate: 0.29, Epsilon: 0.010, smartmove0.5333333333333333
Wins: 942, Losses: 1962, Draws: 297
Episode: 3300, Win Rate: 0.29, Epsilon: 0.010, smartmove0.55
Wins: 967, Losses: 2028, Draws: 306
Episode: 3400, Win Rate: 0.29, Epsilon: 0.010, smartmove0.5666666666666667
Wins: 998, Losses: 2087, Draws: 316
Episode: 3500, Win Rate: 0.29, Epsilon: 0.010, smartmove0.5833333333333334
Wins: 1013, Losses: 2157, Draws: 331
Episode: 3600, Win Rate: 0.29, Epsilon: 0.010, smartmove0.6
Wins: 1027, Losses: 2231, Draws: 343
Episode: 3700, Win Rate: 0.28, Epsilon: 0.010, smartmove0.6166666666666667
Wins: 1048, Losses: 2303, Draws: 350
Episode: 3800, Win Rate: 0.28, Epsilon: 0.010, smartmove0.6333333333333333
Wins: 1072, Losses: 2368, Draws: 361
Episode: 3900, Win Rate: 0.28, Epsilon: 0

Resetting epsilon to 1.0
Episode: 4000, Win Rate: 0.28, Epsilon: 1.000, smartmove0.6666666666666666
Wins: 1106, Losses: 2501, Draws: 394
Episode: 4100, Win Rate: 0.27, Epsilon: 0.901, smartmove0.6833333333333333
Wins: 1117, Losses: 2582, Draws: 402
Episode: 4200, Win Rate: 0.27, Epsilon: 0.802, smartmove0.7
Wins: 1129, Losses: 2663, Draws: 409
Episode: 4300, Win Rate: 0.27, Epsilon: 0.703, smartmove0.7166666666666667
Wins: 1142, Losses: 2739, Draws: 420
Episode: 4400, Win Rate: 0.26, Epsilon: 0.604, smartmove0.7333333333333333
Wins: 1155, Losses: 2808, Draws: 438
Episode: 4500, Win Rate: 0.26, Epsilon: 0.505, smartmove0.75
Wins: 1164, Losses: 2892, Draws: 445
Episode: 4600, Win Rate: 0.26, Epsilon: 0.406, smartmove0.7666666666666667
Wins: 1174, Losses: 2973, Draws: 454
Episode: 4700, Win Rate: 0.25, Epsilon: 0.307, smartmove0.7833333333333333
Wins: 1180, Losses: 3051, Draws: 470
Episode: 4800, Win Rate: 0.25, Epsilon: 0.208, smartmove0.8
Wins: 1187, Losses: 3127, Draws: 487
Episode: 49

Episode: 5000, Win Rate: 0.24, Epsilon: 0.010, smartmove0.8333333333333334
Wins: 1210, Losses: 3270, Draws: 521
Episode: 5100, Win Rate: 0.24, Epsilon: 0.010, smartmove0.85
Wins: 1224, Losses: 3332, Draws: 545
Episode: 5200, Win Rate: 0.24, Epsilon: 0.010, smartmove0.8666666666666667
Wins: 1241, Losses: 3390, Draws: 570
Episode: 5300, Win Rate: 0.24, Epsilon: 0.010, smartmove0.8833333333333333
Wins: 1253, Losses: 3462, Draws: 586
Episode: 5400, Win Rate: 0.24, Epsilon: 0.010, smartmove0.9
Wins: 1270, Losses: 3518, Draws: 613
Episode: 5500, Win Rate: 0.23, Epsilon: 0.010, smartmove0.9166666666666666
Wins: 1287, Losses: 3568, Draws: 646
Episode: 5600, Win Rate: 0.23, Epsilon: 0.010, smartmove0.9333333333333333
Wins: 1291, Losses: 3644, Draws: 666
Episode: 5700, Win Rate: 0.23, Epsilon: 0.010, smartmove0.95
Wins: 1296, Losses: 3714, Draws: 691
Episode: 5800, Win Rate: 0.22, Epsilon: 0.010, smartmove0.9666666666666667
Wins: 1299, Losses: 3780, Draws: 722
Episode: 5900, Win Rate: 0.22, Epsi

Resetting epsilon to 1.0
Episode: 6000, Win Rate: 0.22, Epsilon: 1.000, smartmove1
Wins: 1305, Losses: 3907, Draws: 789
Episode: 6100, Win Rate: 0.21, Epsilon: 0.901, smartmove1
Wins: 1305, Losses: 3998, Draws: 798
Episode: 6200, Win Rate: 0.21, Epsilon: 0.802, smartmove1
Wins: 1308, Losses: 4080, Draws: 813
Episode: 6300, Win Rate: 0.21, Epsilon: 0.703, smartmove1
Wins: 1310, Losses: 4168, Draws: 823
Episode: 6400, Win Rate: 0.21, Epsilon: 0.604, smartmove1
Wins: 1314, Losses: 4247, Draws: 840
Episode: 6500, Win Rate: 0.20, Epsilon: 0.505, smartmove1
Wins: 1316, Losses: 4329, Draws: 856
Episode: 6600, Win Rate: 0.20, Epsilon: 0.406, smartmove1
Wins: 1317, Losses: 4402, Draws: 882
Episode: 6700, Win Rate: 0.20, Epsilon: 0.307, smartmove1
Wins: 1322, Losses: 4466, Draws: 913
Episode: 6800, Win Rate: 0.19, Epsilon: 0.208, smartmove1
Wins: 1325, Losses: 4534, Draws: 942
Episode: 6900, Win Rate: 0.19, Epsilon: 0.109, smartmove1
Wins: 1326, Losses: 4608, Draws: 967


Episode: 7000, Win Rate: 0.19, Epsilon: 0.010, smartmove1
Wins: 1329, Losses: 4664, Draws: 1008
Episode: 7100, Win Rate: 0.19, Epsilon: 0.010, smartmove1
Wins: 1331, Losses: 4712, Draws: 1058
Episode: 7200, Win Rate: 0.19, Epsilon: 0.010, smartmove1
Wins: 1334, Losses: 4761, Draws: 1106
Episode: 7300, Win Rate: 0.18, Epsilon: 0.010, smartmove1
Wins: 1342, Losses: 4816, Draws: 1143
Episode: 7400, Win Rate: 0.18, Epsilon: 0.010, smartmove1
Wins: 1349, Losses: 4853, Draws: 1199
Episode: 7500, Win Rate: 0.18, Epsilon: 0.010, smartmove1
Wins: 1354, Losses: 4900, Draws: 1247
Episode: 7600, Win Rate: 0.18, Epsilon: 0.010, smartmove1
Wins: 1359, Losses: 4939, Draws: 1303
Episode: 7700, Win Rate: 0.18, Epsilon: 0.010, smartmove1
Wins: 1360, Losses: 4977, Draws: 1364
Episode: 7800, Win Rate: 0.17, Epsilon: 0.010, smartmove1
Wins: 1364, Losses: 5016, Draws: 1421
Episode: 7900, Win Rate: 0.17, Epsilon: 0.010, smartmove1
Wins: 1367, Losses: 5061, Draws: 1473


Resetting epsilon to 1.0
Episode: 8000, Win Rate: 0.17, Epsilon: 1.000, smartmove1
Wins: 1373, Losses: 5101, Draws: 1527
Episode: 8100, Win Rate: 0.17, Epsilon: 0.901, smartmove1
Wins: 1375, Losses: 5192, Draws: 1534
Episode: 8200, Win Rate: 0.17, Epsilon: 0.802, smartmove1
Wins: 1377, Losses: 5275, Draws: 1549
Episode: 8300, Win Rate: 0.17, Epsilon: 0.703, smartmove1
Wins: 1380, Losses: 5357, Draws: 1564
Episode: 8400, Win Rate: 0.16, Epsilon: 0.604, smartmove1
Wins: 1382, Losses: 5439, Draws: 1580
Episode: 8500, Win Rate: 0.16, Epsilon: 0.505, smartmove1
Wins: 1382, Losses: 5520, Draws: 1599
Episode: 8600, Win Rate: 0.16, Epsilon: 0.406, smartmove1
Wins: 1385, Losses: 5590, Draws: 1626
Episode: 8700, Win Rate: 0.16, Epsilon: 0.307, smartmove1
Wins: 1388, Losses: 5653, Draws: 1660
Episode: 8800, Win Rate: 0.16, Epsilon: 0.208, smartmove1
Wins: 1392, Losses: 5712, Draws: 1697
Episode: 8900, Win Rate: 0.16, Epsilon: 0.109, smartmove1
Wins: 1398, Losses: 5752, Draws: 1751


Episode: 9000, Win Rate: 0.16, Epsilon: 0.010, smartmove1
Wins: 1405, Losses: 5788, Draws: 1808
Episode: 9100, Win Rate: 0.16, Epsilon: 0.010, smartmove1
Wins: 1416, Losses: 5818, Draws: 1867
Episode: 9200, Win Rate: 0.15, Epsilon: 0.010, smartmove1
Wins: 1420, Losses: 5849, Draws: 1932
Episode: 9300, Win Rate: 0.15, Epsilon: 0.010, smartmove1
Wins: 1430, Losses: 5874, Draws: 1997
Episode: 9400, Win Rate: 0.15, Epsilon: 0.010, smartmove1
Wins: 1437, Losses: 5899, Draws: 2065
Episode: 9500, Win Rate: 0.15, Epsilon: 0.010, smartmove1
Wins: 1441, Losses: 5922, Draws: 2138
Episode: 9600, Win Rate: 0.15, Epsilon: 0.010, smartmove1
Wins: 1443, Losses: 5949, Draws: 2209
Episode: 9700, Win Rate: 0.15, Epsilon: 0.010, smartmove1
Wins: 1447, Losses: 5978, Draws: 2276
Episode: 9800, Win Rate: 0.15, Epsilon: 0.010, smartmove1
Wins: 1455, Losses: 6004, Draws: 2342
Episode: 9900, Win Rate: 0.15, Epsilon: 0.010, smartmove1
Wins: 1461, Losses: 6030, Draws: 2410


In [9]:
import numpy as np
import random
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from TicTacToe import TicTacToe

class SQNAgent:
    def __init__(self, state_size=9, action_size=9, gamma=0.95):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = gamma  
        self.learning_rate = 0.001
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.9995
        self.batch_size = 32
        self.replay_buffer = deque(maxlen=10000)
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential([
            Dense(64, input_dim=self.state_size, activation='relu'),
            Dense(64, activation='relu'),
            Dense(self.action_size, activation='linear')
        ])
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def process_state(self, state):
        """
        Converts the state from [0,1,2] format to [-1,0,1] format for better learning.
        """
        processed_state = np.array(state).copy()
        processed_state[processed_state == 0] = -1
        processed_state[processed_state == 1] = 0
        processed_state[processed_state == 2] = 1
        return processed_state

    def select_action(self, state, valid_actions):
        if not valid_actions:
            return None

        if random.random() < self.epsilon:
            return random.choice(valid_actions)
        
        processed_state = self.process_state(state)
        q_values = self.model.predict(processed_state.reshape(1, -1), verbose=0)[0]
        valid_q_values = [(q_values[action], action) for action in valid_actions]
        return max(valid_q_values, key=lambda x: x[0])[1]

    def store_experience(self, state, action, reward, next_state, done):
        self.replay_buffer.append((state, action, reward, next_state, done))

    def train(self):
        if len(self.replay_buffer) < self.batch_size:
            return

        mini_batch = random.sample(self.replay_buffer, self.batch_size)
        states = np.array([self.process_state(exp[0]) for exp in mini_batch])
        next_states = np.array([self.process_state(exp[3]) for exp in mini_batch])
        current_q_values = self.model.predict(states, verbose=0)
        next_q_values = self.model.predict(next_states, verbose=0)

        x = []
        y = []

        for i, (state, action, reward, next_state, done) in enumerate(mini_batch):
            if done:
                target = reward
            else:
                next_valid_actions = [j for j, val in enumerate(next_state) if val == 0]
                if next_valid_actions:
                    max_next_q = max(next_q_values[i][action] for action in next_valid_actions)
                    target = reward + self.gamma * max_next_q
                else:
                    target = reward
            
            current_q = current_q_values[i].copy()
            current_q[action] = target
            x.append(self.process_state(state))
            y.append(current_q)

        self.model.fit(np.array(x), np.array(y), batch_size=32, epochs=1, verbose=0)

def train_agent(episodes=10000):
    agent = SQNAgent()
    history = {'wins': 0, 'losses': 0, 'draws': 0}
    initial_epsilon = 1.0
    min_epsilon = 0.01
    decay_episodes = 1000
    
    for episode in range(episodes):
        if episode % 2000 == 0 and episode > 0:
            agent.epsilon = 1.0
            print("Resetting epsilon to 1.0")
        else:
            epsilon_decay = (initial_epsilon - min_epsilon) / decay_episodes
            agent.epsilon = max(min_epsilon, initial_epsilon - (episode % 2000) * epsilon_decay)
        
        smartness = min(1, episode / (episodes * 0.6))
        game = TicTacToe(smartMovePlayer1=smartness)
        state = np.array(game.board)
        
        game.player1_move()
        state = np.array(game.board)
        
        while True:
            valid_actions = game.empty_positions()
            if not valid_actions:
                history['draws'] += 1
                break
                
            action = agent.select_action(state, valid_actions)
            game.make_move(action, player=2)
            
            reward = 0
            if game.current_winner == 2:
                reward = 10.0
                history['wins'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            
            game.player1_move()
            if game.current_winner == 1:
                reward = -10.0
                history['losses'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            elif game.is_full():
                reward = 0.0
                history['draws'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            
            agent.store_experience(state, action, reward, game.board, False)
            state = np.array(game.board)
        
        agent.train()
        
        if episode % 100 == 0:
            win_rate = history['wins'] / (episode + 1)
            print(f"Episode: {episode}, Win Rate: {win_rate:.2f}, Epsilon: {agent.epsilon:.3f}, smartmove{smartness}")
            print(f"Wins: {history['wins']}, Losses: {history['losses']}, Draws: {history['draws']}")
        
        if episode % 1000 == 0:
            agent.model.save(f'model4_episode_{episode}.h5')
    agent.model.save('model4.h5')
agent = train_agent()



Episode: 0, Win Rate: 0.00, Epsilon: 1.000, smartmove0.0
Wins: 0, Losses: 0, Draws: 1
Episode: 100, Win Rate: 0.35, Epsilon: 0.901, smartmove0.016666666666666666
Wins: 35, Losses: 54, Draws: 12
Episode: 200, Win Rate: 0.32, Epsilon: 0.802, smartmove0.03333333333333333
Wins: 64, Losses: 114, Draws: 23
Episode: 300, Win Rate: 0.31, Epsilon: 0.703, smartmove0.05
Wins: 92, Losses: 173, Draws: 36
Episode: 400, Win Rate: 0.34, Epsilon: 0.604, smartmove0.06666666666666667
Wins: 136, Losses: 222, Draws: 43
Episode: 500, Win Rate: 0.34, Epsilon: 0.505, smartmove0.08333333333333333
Wins: 169, Losses: 277, Draws: 55
Episode: 600, Win Rate: 0.33, Epsilon: 0.406, smartmove0.1
Wins: 200, Losses: 339, Draws: 62
Episode: 700, Win Rate: 0.34, Epsilon: 0.307, smartmove0.11666666666666667
Wins: 236, Losses: 393, Draws: 72
Episode: 800, Win Rate: 0.35, Epsilon: 0.208, smartmove0.13333333333333333
Wins: 278, Losses: 442, Draws: 81
Episode: 900, Win Rate: 0.35, Epsilon: 0.109, smartmove0.15
Wins: 318, Losse

Episode: 1000, Win Rate: 0.37, Epsilon: 0.010, smartmove0.16666666666666666
Wins: 369, Losses: 543, Draws: 89
Episode: 1100, Win Rate: 0.37, Epsilon: 0.010, smartmove0.18333333333333332
Wins: 408, Losses: 601, Draws: 92
Episode: 1200, Win Rate: 0.38, Epsilon: 0.010, smartmove0.2
Wins: 457, Losses: 644, Draws: 100
Episode: 1300, Win Rate: 0.39, Epsilon: 0.010, smartmove0.21666666666666667
Wins: 505, Losses: 690, Draws: 106
Episode: 1400, Win Rate: 0.38, Epsilon: 0.010, smartmove0.23333333333333334
Wins: 534, Losses: 755, Draws: 112
Episode: 1500, Win Rate: 0.37, Epsilon: 0.010, smartmove0.25
Wins: 562, Losses: 817, Draws: 122
Episode: 1600, Win Rate: 0.37, Epsilon: 0.010, smartmove0.26666666666666666
Wins: 598, Losses: 872, Draws: 131
Episode: 1700, Win Rate: 0.37, Epsilon: 0.010, smartmove0.2833333333333333
Wins: 629, Losses: 935, Draws: 137
Episode: 1800, Win Rate: 0.36, Epsilon: 0.010, smartmove0.3
Wins: 653, Losses: 998, Draws: 150
Episode: 1900, Win Rate: 0.36, Epsilon: 0.010, smar

Resetting epsilon to 1.0
Episode: 2000, Win Rate: 0.36, Epsilon: 1.000, smartmove0.3333333333333333
Wins: 718, Losses: 1121, Draws: 162
Episode: 2100, Win Rate: 0.35, Epsilon: 0.901, smartmove0.35
Wins: 735, Losses: 1192, Draws: 174
Episode: 2200, Win Rate: 0.34, Epsilon: 0.802, smartmove0.36666666666666664
Wins: 753, Losses: 1264, Draws: 184
Episode: 2300, Win Rate: 0.34, Epsilon: 0.703, smartmove0.38333333333333336
Wins: 774, Losses: 1335, Draws: 192
Episode: 2400, Win Rate: 0.33, Epsilon: 0.604, smartmove0.4
Wins: 786, Losses: 1412, Draws: 203
Episode: 2500, Win Rate: 0.32, Epsilon: 0.505, smartmove0.4166666666666667
Wins: 801, Losses: 1487, Draws: 213
Episode: 2600, Win Rate: 0.32, Epsilon: 0.406, smartmove0.43333333333333335
Wins: 826, Losses: 1557, Draws: 218
Episode: 2700, Win Rate: 0.31, Epsilon: 0.307, smartmove0.45
Wins: 842, Losses: 1635, Draws: 224
Episode: 2800, Win Rate: 0.30, Epsilon: 0.208, smartmove0.4666666666666667
Wins: 854, Losses: 1714, Draws: 233
Episode: 2900, W

Episode: 3000, Win Rate: 0.30, Epsilon: 0.010, smartmove0.5
Wins: 896, Losses: 1855, Draws: 250
Episode: 3100, Win Rate: 0.30, Epsilon: 0.010, smartmove0.5166666666666667
Wins: 921, Losses: 1923, Draws: 257
Episode: 3200, Win Rate: 0.30, Epsilon: 0.010, smartmove0.5333333333333333
Wins: 945, Losses: 1989, Draws: 267
Episode: 3300, Win Rate: 0.29, Epsilon: 0.010, smartmove0.55
Wins: 967, Losses: 2055, Draws: 279
Episode: 3400, Win Rate: 0.29, Epsilon: 0.010, smartmove0.5666666666666667
Wins: 987, Losses: 2129, Draws: 285
Episode: 3500, Win Rate: 0.29, Epsilon: 0.010, smartmove0.5833333333333334
Wins: 1003, Losses: 2200, Draws: 298
Episode: 3600, Win Rate: 0.29, Epsilon: 0.010, smartmove0.6
Wins: 1031, Losses: 2262, Draws: 308
Episode: 3700, Win Rate: 0.29, Epsilon: 0.010, smartmove0.6166666666666667
Wins: 1061, Losses: 2324, Draws: 316
Episode: 3800, Win Rate: 0.28, Epsilon: 0.010, smartmove0.6333333333333333
Wins: 1081, Losses: 2390, Draws: 330
Episode: 3900, Win Rate: 0.28, Epsilon: 0

Resetting epsilon to 1.0
Episode: 4000, Win Rate: 0.28, Epsilon: 1.000, smartmove0.6666666666666666
Wins: 1128, Losses: 2526, Draws: 347
Episode: 4100, Win Rate: 0.28, Epsilon: 0.901, smartmove0.6833333333333333
Wins: 1137, Losses: 2614, Draws: 350
Episode: 4200, Win Rate: 0.27, Epsilon: 0.802, smartmove0.7
Wins: 1148, Losses: 2698, Draws: 355
Episode: 4300, Win Rate: 0.27, Epsilon: 0.703, smartmove0.7166666666666667
Wins: 1156, Losses: 2782, Draws: 363
Episode: 4400, Win Rate: 0.27, Epsilon: 0.604, smartmove0.7333333333333333
Wins: 1169, Losses: 2856, Draws: 376
Episode: 4500, Win Rate: 0.26, Epsilon: 0.505, smartmove0.75
Wins: 1181, Losses: 2933, Draws: 387
Episode: 4600, Win Rate: 0.26, Epsilon: 0.406, smartmove0.7666666666666667
Wins: 1191, Losses: 3011, Draws: 399
Episode: 4700, Win Rate: 0.26, Epsilon: 0.307, smartmove0.7833333333333333
Wins: 1208, Losses: 3082, Draws: 411
Episode: 4800, Win Rate: 0.25, Epsilon: 0.208, smartmove0.8
Wins: 1221, Losses: 3163, Draws: 417
Episode: 49

Episode: 5000, Win Rate: 0.25, Epsilon: 0.010, smartmove0.8333333333333334
Wins: 1250, Losses: 3311, Draws: 440
Episode: 5100, Win Rate: 0.25, Epsilon: 0.010, smartmove0.85
Wins: 1259, Losses: 3392, Draws: 450
Episode: 5200, Win Rate: 0.24, Epsilon: 0.010, smartmove0.8666666666666667
Wins: 1273, Losses: 3469, Draws: 459
Episode: 5300, Win Rate: 0.24, Epsilon: 0.010, smartmove0.8833333333333333
Wins: 1294, Losses: 3535, Draws: 472
Episode: 5400, Win Rate: 0.24, Epsilon: 0.010, smartmove0.9
Wins: 1307, Losses: 3610, Draws: 484
Episode: 5500, Win Rate: 0.24, Epsilon: 0.010, smartmove0.9166666666666666
Wins: 1320, Losses: 3674, Draws: 507
Episode: 5600, Win Rate: 0.24, Epsilon: 0.010, smartmove0.9333333333333333
Wins: 1327, Losses: 3744, Draws: 530
Episode: 5700, Win Rate: 0.23, Epsilon: 0.010, smartmove0.95
Wins: 1334, Losses: 3822, Draws: 545
Episode: 5800, Win Rate: 0.23, Epsilon: 0.010, smartmove0.9666666666666667
Wins: 1345, Losses: 3889, Draws: 567
Episode: 5900, Win Rate: 0.23, Epsi

Resetting epsilon to 1.0
Episode: 6000, Win Rate: 0.23, Epsilon: 1.000, smartmove1
Wins: 1360, Losses: 4027, Draws: 614
Episode: 6100, Win Rate: 0.22, Epsilon: 0.901, smartmove1
Wins: 1361, Losses: 4116, Draws: 624
Episode: 6200, Win Rate: 0.22, Epsilon: 0.802, smartmove1
Wins: 1362, Losses: 4209, Draws: 630
Episode: 6300, Win Rate: 0.22, Epsilon: 0.703, smartmove1
Wins: 1364, Losses: 4296, Draws: 641
Episode: 6400, Win Rate: 0.21, Epsilon: 0.604, smartmove1
Wins: 1365, Losses: 4379, Draws: 657
Episode: 6500, Win Rate: 0.21, Epsilon: 0.505, smartmove1
Wins: 1370, Losses: 4457, Draws: 674
Episode: 6600, Win Rate: 0.21, Epsilon: 0.406, smartmove1
Wins: 1373, Losses: 4539, Draws: 689
Episode: 6700, Win Rate: 0.21, Epsilon: 0.307, smartmove1
Wins: 1376, Losses: 4618, Draws: 707
Episode: 6800, Win Rate: 0.20, Epsilon: 0.208, smartmove1
Wins: 1379, Losses: 4697, Draws: 725
Episode: 6900, Win Rate: 0.20, Epsilon: 0.109, smartmove1
Wins: 1384, Losses: 4770, Draws: 747


Episode: 7000, Win Rate: 0.20, Epsilon: 0.010, smartmove1
Wins: 1390, Losses: 4843, Draws: 768
Episode: 7100, Win Rate: 0.20, Epsilon: 0.010, smartmove1
Wins: 1399, Losses: 4904, Draws: 798
Episode: 7200, Win Rate: 0.20, Epsilon: 0.010, smartmove1
Wins: 1409, Losses: 4963, Draws: 829
Episode: 7300, Win Rate: 0.19, Epsilon: 0.010, smartmove1
Wins: 1412, Losses: 5027, Draws: 862
Episode: 7400, Win Rate: 0.19, Epsilon: 0.010, smartmove1
Wins: 1417, Losses: 5084, Draws: 900
Episode: 7500, Win Rate: 0.19, Epsilon: 0.010, smartmove1
Wins: 1428, Losses: 5142, Draws: 931
Episode: 7600, Win Rate: 0.19, Epsilon: 0.010, smartmove1
Wins: 1437, Losses: 5190, Draws: 974
Episode: 7700, Win Rate: 0.19, Epsilon: 0.010, smartmove1
Wins: 1443, Losses: 5250, Draws: 1008
Episode: 7800, Win Rate: 0.19, Epsilon: 0.010, smartmove1
Wins: 1449, Losses: 5299, Draws: 1053
Episode: 7900, Win Rate: 0.18, Epsilon: 0.010, smartmove1
Wins: 1459, Losses: 5350, Draws: 1092


Resetting epsilon to 1.0
Episode: 8000, Win Rate: 0.18, Epsilon: 1.000, smartmove1
Wins: 1466, Losses: 5401, Draws: 1134
Episode: 8100, Win Rate: 0.18, Epsilon: 0.901, smartmove1
Wins: 1467, Losses: 5493, Draws: 1141
Episode: 8200, Win Rate: 0.18, Epsilon: 0.802, smartmove1
Wins: 1467, Losses: 5589, Draws: 1145
Episode: 8300, Win Rate: 0.18, Epsilon: 0.703, smartmove1
Wins: 1468, Losses: 5682, Draws: 1151
Episode: 8400, Win Rate: 0.17, Epsilon: 0.604, smartmove1
Wins: 1469, Losses: 5767, Draws: 1165
Episode: 8500, Win Rate: 0.17, Epsilon: 0.505, smartmove1
Wins: 1472, Losses: 5850, Draws: 1179
Episode: 8600, Win Rate: 0.17, Epsilon: 0.406, smartmove1
Wins: 1475, Losses: 5924, Draws: 1202
Episode: 8700, Win Rate: 0.17, Epsilon: 0.307, smartmove1
Wins: 1478, Losses: 6004, Draws: 1219
Episode: 8800, Win Rate: 0.17, Epsilon: 0.208, smartmove1
Wins: 1487, Losses: 6068, Draws: 1246
Episode: 8900, Win Rate: 0.17, Epsilon: 0.109, smartmove1
Wins: 1494, Losses: 6123, Draws: 1284


Episode: 9000, Win Rate: 0.17, Epsilon: 0.010, smartmove1
Wins: 1503, Losses: 6172, Draws: 1326
Episode: 9100, Win Rate: 0.17, Epsilon: 0.010, smartmove1
Wins: 1516, Losses: 6219, Draws: 1366
Episode: 9200, Win Rate: 0.17, Epsilon: 0.010, smartmove1
Wins: 1527, Losses: 6259, Draws: 1415
Episode: 9300, Win Rate: 0.17, Epsilon: 0.010, smartmove1
Wins: 1537, Losses: 6299, Draws: 1465
Episode: 9400, Win Rate: 0.16, Epsilon: 0.010, smartmove1
Wins: 1549, Losses: 6343, Draws: 1509
Episode: 9500, Win Rate: 0.16, Epsilon: 0.010, smartmove1
Wins: 1557, Losses: 6395, Draws: 1549
Episode: 9600, Win Rate: 0.16, Epsilon: 0.010, smartmove1
Wins: 1565, Losses: 6436, Draws: 1600
Episode: 9700, Win Rate: 0.16, Epsilon: 0.010, smartmove1
Wins: 1575, Losses: 6473, Draws: 1653
Episode: 9800, Win Rate: 0.16, Epsilon: 0.010, smartmove1
Wins: 1589, Losses: 6520, Draws: 1692
Episode: 9900, Win Rate: 0.16, Epsilon: 0.010, smartmove1
Wins: 1599, Losses: 6570, Draws: 1732


In [10]:
import numpy as np
import random
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from TicTacToe import TicTacToe

class SQNAgent:
    def __init__(self, state_size=9, action_size=9, gamma=0.95):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = gamma  
        self.learning_rate = 0.001
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.9995
        self.batch_size = 32
        self.replay_buffer = deque(maxlen=10000)
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential([
            Dense(64, input_dim=self.state_size, activation='relu'),
            Dense(64, activation='relu'),
            Dense(self.action_size, activation='linear')
        ])
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def process_state(self, state):
        """
        Converts the state from [0,1,2] format to [-1,0,1] format for better learning.
        """
        processed_state = np.array(state).copy()
        processed_state[processed_state == 0] = -1
        processed_state[processed_state == 1] = 0
        processed_state[processed_state == 2] = 1
        return processed_state

    def select_action(self, state, valid_actions):
        if not valid_actions:
            return None

        if random.random() < self.epsilon:
            return random.choice(valid_actions)
        
        processed_state = self.process_state(state)
        q_values = self.model.predict(processed_state.reshape(1, -1), verbose=0)[0]
        valid_q_values = [(q_values[action], action) for action in valid_actions]
        return max(valid_q_values, key=lambda x: x[0])[1]

    def store_experience(self, state, action, reward, next_state, done):
        self.replay_buffer.append((state, action, reward, next_state, done))

    def train(self):
        if len(self.replay_buffer) < self.batch_size:
            return

        mini_batch = random.sample(self.replay_buffer, self.batch_size)
        states = np.array([self.process_state(exp[0]) for exp in mini_batch])
        next_states = np.array([self.process_state(exp[3]) for exp in mini_batch])
        current_q_values = self.model.predict(states, verbose=0)
        next_q_values = self.model.predict(next_states, verbose=0)

        x = []
        y = []

        for i, (state, action, reward, next_state, done) in enumerate(mini_batch):
            if done:
                target = reward
            else:
                next_valid_actions = [j for j, val in enumerate(next_state) if val == 0]
                if next_valid_actions:
                    max_next_q = max(next_q_values[i][action] for action in next_valid_actions)
                    target = reward + self.gamma * max_next_q
                else:
                    target = reward
            
            current_q = current_q_values[i].copy()
            current_q[action] = target
            x.append(self.process_state(state))
            y.append(current_q)

        self.model.fit(np.array(x), np.array(y), batch_size=32, epochs=1, verbose=0)

def train_agent(episodes=10000):
    agent = SQNAgent()
    history = {'wins': 0, 'losses': 0, 'draws': 0}
    initial_epsilon = 1.0
    min_epsilon = 0.01
    decay_episodes = 1000
    
    for episode in range(episodes):
        if episode % 2000 == 0 and episode > 0:
            agent.epsilon = 1.0
            print("Resetting epsilon to 1.0")
        else:
            epsilon_decay = (initial_epsilon - min_epsilon) / decay_episodes
            agent.epsilon = max(min_epsilon, initial_epsilon - (episode % 2000) * epsilon_decay)
        
        smartness = min(1, episode / (episodes * 0.6))
        game = TicTacToe(smartMovePlayer1=smartness)
        state = np.array(game.board)
        
        game.player1_move()
        state = np.array(game.board)
        
        while True:
            valid_actions = game.empty_positions()
            if not valid_actions:
                history['draws'] += 1
                break
                
            action = agent.select_action(state, valid_actions)
            game.make_move(action, player=2)
            
            reward = 0
            if game.current_winner == 2:
                reward = 1.0
                history['wins'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            
            game.player1_move()
            if game.current_winner == 1:
                reward = -1.0
                history['losses'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            elif game.is_full():
                reward = -0.5
                history['draws'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            
            agent.store_experience(state, action, reward, game.board, False)
            state = np.array(game.board)
        
        agent.train()
        
        if episode % 100 == 0:
            win_rate = history['wins'] / (episode + 1)
            print(f"Episode: {episode}, Win Rate: {win_rate:.2f}, Epsilon: {agent.epsilon:.3f}, smartmove{smartness}")
            print(f"Wins: {history['wins']}, Losses: {history['losses']}, Draws: {history['draws']}")
        
        if episode % 1000 == 0:
            agent.model.save(f'model5_episode_{episode}.h5')
    agent.model.save('model5.h5')
agent = train_agent()



/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Episode: 0, Win Rate: 1.00, Epsilon: 1.000, smartmove0.0
Wins: 1, Losses: 0, Draws: 0
Episode: 100, Win Rate: 0.32, Epsilon: 0.901, smartmove0.016666666666666666
Wins: 32, Losses: 59, Draws: 10
Episode: 200, Win Rate: 0.32, Epsilon: 0.802, smartmove0.03333333333333333
Wins: 65, Losses: 118, Draws: 18
Episode: 300, Win Rate: 0.31, Epsilon: 0.703, smartmove0.05
Wins: 93, Losses: 180, Draws: 28
Episode: 400, Win Rate: 0.30, Epsilon: 0.604, smartmove0.06666666666666667
Wins: 120, Losses: 243, Draws: 38
Episode: 500, Win Rate: 0.29, Epsilon: 0.505, smartmove0.08333333333333333
Wins: 145, Losses: 310, Draws: 46
Episode: 600, Win Rate: 0.29, Epsilon: 0.406, smartmove0.1
Wins: 175, Losses: 370, Draws: 56
Episode: 700, Win Rate: 0.29, Epsilon: 0.307, smartmove0.11666666666666667
Wins: 202, Losses: 435, Draws: 64
Episode: 800, Win Rate: 0.29, Epsilon: 0.208, smartmove0.13333333333333333
Wins: 231, Losses: 500, Draws: 70
Episode: 900, Win Rate: 0.30, Epsilon: 0.109, smartmove0.15
Wins: 266, Losse

Episode: 1000, Win Rate: 0.30, Epsilon: 0.010, smartmove0.16666666666666666
Wins: 299, Losses: 622, Draws: 80
Episode: 1100, Win Rate: 0.30, Epsilon: 0.010, smartmove0.18333333333333332
Wins: 328, Losses: 688, Draws: 85
Episode: 1200, Win Rate: 0.30, Epsilon: 0.010, smartmove0.2
Wins: 362, Losses: 750, Draws: 89
Episode: 1300, Win Rate: 0.30, Epsilon: 0.010, smartmove0.21666666666666667
Wins: 391, Losses: 817, Draws: 93
Episode: 1400, Win Rate: 0.30, Epsilon: 0.010, smartmove0.23333333333333334
Wins: 422, Losses: 884, Draws: 95
Episode: 1500, Win Rate: 0.29, Epsilon: 0.010, smartmove0.25
Wins: 439, Losses: 953, Draws: 109
Episode: 1600, Win Rate: 0.29, Epsilon: 0.010, smartmove0.26666666666666666
Wins: 463, Losses: 1021, Draws: 117
Episode: 1700, Win Rate: 0.29, Epsilon: 0.010, smartmove0.2833333333333333
Wins: 489, Losses: 1083, Draws: 129
Episode: 1800, Win Rate: 0.29, Epsilon: 0.010, smartmove0.3
Wins: 515, Losses: 1149, Draws: 137
Episode: 1900, Win Rate: 0.28, Epsilon: 0.010, smar

Resetting epsilon to 1.0
Episode: 2000, Win Rate: 0.28, Epsilon: 1.000, smartmove0.3333333333333333
Wins: 567, Losses: 1281, Draws: 153
Episode: 2100, Win Rate: 0.28, Epsilon: 0.901, smartmove0.35
Wins: 588, Losses: 1347, Draws: 166
Episode: 2200, Win Rate: 0.28, Epsilon: 0.802, smartmove0.36666666666666664
Wins: 610, Losses: 1414, Draws: 177
Episode: 2300, Win Rate: 0.27, Epsilon: 0.703, smartmove0.38333333333333336
Wins: 630, Losses: 1489, Draws: 182
Episode: 2400, Win Rate: 0.27, Epsilon: 0.604, smartmove0.4
Wins: 648, Losses: 1558, Draws: 195
Episode: 2500, Win Rate: 0.27, Epsilon: 0.505, smartmove0.4166666666666667
Wins: 667, Losses: 1629, Draws: 205
Episode: 2600, Win Rate: 0.26, Epsilon: 0.406, smartmove0.43333333333333335
Wins: 689, Losses: 1700, Draws: 212
Episode: 2700, Win Rate: 0.26, Epsilon: 0.307, smartmove0.45
Wins: 710, Losses: 1768, Draws: 223
Episode: 2800, Win Rate: 0.26, Epsilon: 0.208, smartmove0.4666666666666667
Wins: 728, Losses: 1839, Draws: 234
Episode: 2900, W

Episode: 3000, Win Rate: 0.26, Epsilon: 0.010, smartmove0.5
Wins: 768, Losses: 1985, Draws: 248
Episode: 3100, Win Rate: 0.25, Epsilon: 0.010, smartmove0.5166666666666667
Wins: 785, Losses: 2062, Draws: 254
Episode: 3200, Win Rate: 0.25, Epsilon: 0.010, smartmove0.5333333333333333
Wins: 811, Losses: 2130, Draws: 260
Episode: 3300, Win Rate: 0.25, Epsilon: 0.010, smartmove0.55
Wins: 834, Losses: 2201, Draws: 266
Episode: 3400, Win Rate: 0.25, Epsilon: 0.010, smartmove0.5666666666666667
Wins: 852, Losses: 2270, Draws: 279
Episode: 3500, Win Rate: 0.25, Epsilon: 0.010, smartmove0.5833333333333334
Wins: 878, Losses: 2335, Draws: 288
Episode: 3600, Win Rate: 0.25, Epsilon: 0.010, smartmove0.6
Wins: 897, Losses: 2408, Draws: 296
Episode: 3700, Win Rate: 0.25, Epsilon: 0.010, smartmove0.6166666666666667
Wins: 919, Losses: 2471, Draws: 311
Episode: 3800, Win Rate: 0.24, Epsilon: 0.010, smartmove0.6333333333333333
Wins: 926, Losses: 2549, Draws: 326
Episode: 3900, Win Rate: 0.24, Epsilon: 0.010

...

Episode: 5000, Win Rate: 0.21, Epsilon: 0.010, smartmove0.8333333333333334
Wins: 1060, Losses: 3485, Draws: 456
Episode: 5100, Win Rate: 0.21, Epsilon: 0.010, smartmove0.85
Wins: 1070, Losses: 3553, Draws: 478
Episode: 5200, Win Rate: 0.21, Epsilon: 0.010, smartmove0.8666666666666667
Wins: 1092, Losses: 3616, Draws: 493
Episode: 5300, Win Rate: 0.21, Epsilon: 0.010, smartmove0.8833333333333333
Wins: 1105, Losses: 3684, Draws: 512
Episode: 5400, Win Rate: 0.21, Epsilon: 0.010, smartmove0.9
Wins: 1121, Losses: 3750, Draws: 530
Episode: 5500, Win Rate: 0.21, Epsilon: 0.010, smartmove0.9166666666666666
Wins: 1134, Losses: 3818, Draws: 549
Episode: 5600, Win Rate: 0.20, Epsilon: 0.010, smartmove0.9333333333333333
Wins: 1146, Losses: 3882, Draws: 573
Episode: 5700, Win Rate: 0.20, Epsilon: 0.010, smartmove0.95
Wins: 1151, Losses: 3946, Draws: 604
Episode: 5800, Win Rate: 0.20, Epsilon: 0.010, smartmove0.9666666666666667
Wins: 1156, Losses: 4005, Draws: 640
Episode: 5900, Win Rate: 0.20, Epsi

Resetting epsilon to 1.0
Episode: 6000, Win Rate: 0.20, Epsilon: 1.000, smartmove1
Wins: 1179, Losses: 4123, Draws: 699
Episode: 6100, Win Rate: 0.19, Epsilon: 0.901, smartmove1
Wins: 1183, Losses: 4205, Draws: 713
Episode: 6200, Win Rate: 0.19, Epsilon: 0.802, smartmove1
Wins: 1184, Losses: 4294, Draws: 723
Episode: 6300, Win Rate: 0.19, Epsilon: 0.703, smartmove1
Wins: 1185, Losses: 4383, Draws: 733
Episode: 6400, Win Rate: 0.19, Epsilon: 0.604, smartmove1
Wins: 1186, Losses: 4472, Draws: 743
Episode: 6500, Win Rate: 0.18, Epsilon: 0.505, smartmove1
Wins: 1188, Losses: 4557, Draws: 756
Episode: 6600, Win Rate: 0.18, Epsilon: 0.406, smartmove1
Wins: 1193, Losses: 4627, Draws: 781
Episode: 6700, Win Rate: 0.18, Epsilon: 0.307, smartmove1
Wins: 1197, Losses: 4700, Draws: 804
Episode: 6800, Win Rate: 0.18, Epsilon: 0.208, smartmove1
Wins: 1199, Losses: 4781, Draws: 821
Episode: 6900, Win Rate: 0.17, Epsilon: 0.109, smartmove1
Wins: 1200, Losses: 4846, Draws: 855


Episode: 7000, Win Rate: 0.17, Epsilon: 0.010, smartmove1
Wins: 1201, Losses: 4915, Draws: 885
Episode: 7100, Win Rate: 0.17, Epsilon: 0.010, smartmove1
Wins: 1212, Losses: 4955, Draws: 934
Episode: 7200, Win Rate: 0.17, Epsilon: 0.010, smartmove1
Wins: 1221, Losses: 5007, Draws: 973
Episode: 7300, Win Rate: 0.17, Epsilon: 0.010, smartmove1
Wins: 1228, Losses: 5056, Draws: 1017
Episode: 7400, Win Rate: 0.17, Epsilon: 0.010, smartmove1
Wins: 1230, Losses: 5114, Draws: 1057
Episode: 7500, Win Rate: 0.16, Epsilon: 0.010, smartmove1
Wins: 1236, Losses: 5155, Draws: 1110
Episode: 7600, Win Rate: 0.16, Epsilon: 0.010, smartmove1
Wins: 1240, Losses: 5201, Draws: 1160
Episode: 7700, Win Rate: 0.16, Epsilon: 0.010, smartmove1
Wins: 1241, Losses: 5246, Draws: 1214
Episode: 7800, Win Rate: 0.16, Epsilon: 0.010, smartmove1
Wins: 1245, Losses: 5288, Draws: 1268
Episode: 7900, Win Rate: 0.16, Epsilon: 0.010, smartmove1
Wins: 1251, Losses: 5342, Draws: 1308


Resetting epsilon to 1.0
Episode: 8000, Win Rate: 0.16, Epsilon: 1.000, smartmove1
Wins: 1254, Losses: 5391, Draws: 1356
Episode: 8100, Win Rate: 0.16, Epsilon: 0.901, smartmove1
Wins: 1257, Losses: 5479, Draws: 1365
Episode: 8200, Win Rate: 0.15, Epsilon: 0.802, smartmove1
Wins: 1258, Losses: 5569, Draws: 1374
Episode: 8300, Win Rate: 0.15, Epsilon: 0.703, smartmove1
Wins: 1262, Losses: 5653, Draws: 1386
Episode: 8400, Win Rate: 0.15, Epsilon: 0.604, smartmove1
Wins: 1264, Losses: 5734, Draws: 1403
Episode: 8500, Win Rate: 0.15, Epsilon: 0.505, smartmove1
Wins: 1265, Losses: 5809, Draws: 1427
Episode: 8600, Win Rate: 0.15, Epsilon: 0.406, smartmove1
Wins: 1268, Losses: 5885, Draws: 1448
Episode: 8700, Win Rate: 0.15, Epsilon: 0.307, smartmove1
Wins: 1268, Losses: 5964, Draws: 1469
Episode: 8800, Win Rate: 0.14, Epsilon: 0.208, smartmove1
Wins: 1272, Losses: 6025, Draws: 1504
Episode: 8900, Win Rate: 0.14, Epsilon: 0.109, smartmove1
Wins: 1280, Losses: 6066, Draws: 1555


Episode: 9000, Win Rate: 0.14, Epsilon: 0.010, smartmove1
Wins: 1281, Losses: 6109, Draws: 1611
Episode: 9100, Win Rate: 0.14, Epsilon: 0.010, smartmove1
Wins: 1283, Losses: 6142, Draws: 1676
Episode: 9200, Win Rate: 0.14, Epsilon: 0.010, smartmove1
Wins: 1288, Losses: 6173, Draws: 1740
Episode: 9300, Win Rate: 0.14, Epsilon: 0.010, smartmove1
Wins: 1292, Losses: 6208, Draws: 1801
Episode: 9400, Win Rate: 0.14, Epsilon: 0.010, smartmove1
Wins: 1292, Losses: 6248, Draws: 1861
Episode: 9500, Win Rate: 0.14, Epsilon: 0.010, smartmove1
Wins: 1295, Losses: 6288, Draws: 1918
Episode: 9600, Win Rate: 0.14, Epsilon: 0.010, smartmove1
Wins: 1298, Losses: 6317, Draws: 1986
Episode: 9700, Win Rate: 0.13, Epsilon: 0.010, smartmove1
Wins: 1299, Losses: 6349, Draws: 2053
Episode: 9800, Win Rate: 0.13, Epsilon: 0.010, smartmove1
Wins: 1304, Losses: 6383, Draws: 2114
Episode: 9900, Win Rate: 0.13, Epsilon: 0.010, smartmove1
Wins: 1305, Losses: 6427, Draws: 2169


In [ ]:
import numpy as np
import random
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from TicTacToe import TicTacToe

class SQNAgent:
    def __init__(self, state_size=9, action_size=9, gamma=0.95):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = gamma  
        self.learning_rate = 0.001
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.9995
        self.batch_size = 32
        self.replay_buffer = deque(maxlen=10000)
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential([
            Dense(64, input_dim=self.state_size, activation='relu'),
            Dense(64, activation='relu'),
            Dense(self.action_size, activation='linear')
        ])
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def process_state(self, state):
        """
        Converts the state from [0,1,2] format to [-1,0,1] format for better learning.
        """
        processed_state = np.array(state).copy()
        processed_state[processed_state == 0] = -1
        processed_state[processed_state == 1] = 0
        processed_state[processed_state == 2] = 1
        return processed_state

    def select_action(self, state, valid_actions):
        if not valid_actions:
            return None

        if random.random() < self.epsilon:
            return random.choice(valid_actions)
        
        processed_state = self.process_state(state)
        q_values = self.model.predict(processed_state.reshape(1, -1), verbose=0)[0]
        valid_q_values = [(q_values[action], action) for action in valid_actions]
        return max(valid_q_values, key=lambda x: x[0])[1]

    def store_experience(self, state, action, reward, next_state, done):
        self.replay_buffer.append((state, action, reward, next_state, done))

    def train(self):
        if len(self.replay_buffer) < self.batch_size:
            return

        mini_batch = random.sample(self.replay_buffer, self.batch_size)
        states = np.array([self.process_state(exp[0]) for exp in mini_batch])
        next_states = np.array([self.process_state(exp[3]) for exp in mini_batch])
        current_q_values = self.model.predict(states, verbose=0)
        next_q_values = self.model.predict(next_states, verbose=0)

        x = []
        y = []

        for i, (state, action, reward, next_state, done) in enumerate(mini_batch):
            if done:
                target = reward
            else:
                next_valid_actions = [j for j, val in enumerate(next_state) if val == 0]
                if next_valid_actions:
                    max_next_q = max(next_q_values[i][action] for action in next_valid_actions)
                    target = reward + self.gamma * max_next_q
                else:
                    target = reward
            
            current_q = current_q_values[i].copy()
            current_q[action] = target
            x.append(self.process_state(state))
            y.append(current_q)

        self.model.fit(np.array(x), np.array(y), batch_size=32, epochs=1, verbose=0)

def train_agent(episodes=10000):
    agent = SQNAgent()
    history = {'wins': 0, 'losses': 0, 'draws': 0}
    initial_epsilon = 1.0
    min_epsilon = 0.01
    decay_episodes = 1000
    
    for episode in range(episodes):
        if episode % 1000 == 0 and episode > 0:
            agent.epsilon = 1.0
            print("Resetting epsilon to 1.0")
        else:
            epsilon_decay = (initial_epsilon - min_epsilon) / decay_episodes
            agent.epsilon = max(min_epsilon, initial_epsilon - (episode % 1000) * epsilon_decay)
        
        smartness = min(1, episode / (episodes * 0.6))
        game = TicTacToe(smartMovePlayer1=smartness)
        state = np.array(game.board)
        
        game.player1_move()
        state = np.array(game.board)
        
        while True:
            valid_actions = game.empty_positions()
            if not valid_actions:
                history['draws'] += 1
                break
                
            action = agent.select_action(state, valid_actions)
            game.make_move(action, player=2)
            
            reward = 0
            if game.current_winner == 2:
                reward = 1.0
                history['wins'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            
            game.player1_move()
            if game.current_winner == 1:
                reward = -1.0
                history['losses'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            elif game.is_full():
                reward = min(-0.1,-0.5*smartness)
                history['draws'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            
            agent.store_experience(state, action, reward, game.board, False)
            state = np.array(game.board)
        
        agent.train()
        
        if episode % 100 == 0:
            win_rate = history['wins'] / (episode + 1)
            print(f"Episode: {episode}, Win Rate: {win_rate:.2f}, Epsilon: {agent.epsilon:.3f}, smartmove{smartness}")
            print(f"Wins: {history['wins']}, Losses: {history['losses']}, Draws: {history['draws']}")
        
        if episode % 1000 == 0:
            agent.model.save(f'model6_episode_{episode}.h5')
    agent.model.save('model6.h5')
agent = train_agent()



Episode: 0, Win Rate: 0.00, Epsilon: 1.000, smartmove0.0
Wins: 0, Losses: 1, Draws: 0
Episode: 100, Win Rate: 0.30, Epsilon: 0.901, smartmove0.016666666666666666
Wins: 30, Losses: 63, Draws: 8
Episode: 200, Win Rate: 0.28, Epsilon: 0.802, smartmove0.03333333333333333
Wins: 57, Losses: 127, Draws: 17
Episode: 300, Win Rate: 0.31, Epsilon: 0.703, smartmove0.05
Wins: 94, Losses: 173, Draws: 34
Episode: 400, Win Rate: 0.31, Epsilon: 0.604, smartmove0.06666666666666667
Wins: 123, Losses: 239, Draws: 39
Episode: 500, Win Rate: 0.32, Epsilon: 0.505, smartmove0.08333333333333333
Wins: 158, Losses: 291, Draws: 52
Episode: 600, Win Rate: 0.32, Epsilon: 0.406, smartmove0.1
Wins: 190, Losses: 345, Draws: 66
Episode: 700, Win Rate: 0.32, Epsilon: 0.307, smartmove0.11666666666666667
Wins: 221, Losses: 405, Draws: 75
Episode: 800, Win Rate: 0.32, Epsilon: 0.208, smartmove0.13333333333333333
Wins: 257, Losses: 461, Draws: 83
Episode: 900, Win Rate: 0.32, Epsilon: 0.109, smartmove0.15
Wins: 285, Losses

Resetting epsilon to 1.0
Episode: 1000, Win Rate: 0.32, Epsilon: 1.000, smartmove0.16666666666666666
Wins: 318, Losses: 575, Draws: 108
Episode: 1100, Win Rate: 0.31, Epsilon: 0.901, smartmove0.18333333333333332
Wins: 341, Losses: 639, Draws: 121
Episode: 1200, Win Rate: 0.30, Epsilon: 0.802, smartmove0.2
Wins: 361, Losses: 704, Draws: 136
Episode: 1300, Win Rate: 0.30, Epsilon: 0.703, smartmove0.21666666666666667
Wins: 385, Losses: 769, Draws: 147
Episode: 1400, Win Rate: 0.29, Epsilon: 0.604, smartmove0.23333333333333334
Wins: 412, Losses: 830, Draws: 159
Episode: 1500, Win Rate: 0.30, Epsilon: 0.505, smartmove0.25
Wins: 447, Losses: 884, Draws: 170
Episode: 1600, Win Rate: 0.30, Epsilon: 0.406, smartmove0.26666666666666666
Wins: 473, Losses: 947, Draws: 181
Episode: 1700, Win Rate: 0.30, Epsilon: 0.307, smartmove0.2833333333333333
Wins: 502, Losses: 1011, Draws: 188
Episode: 1800, Win Rate: 0.30, Epsilon: 0.208, smartmove0.3
Wins: 534, Losses: 1074, Draws: 193
Episode: 1900, Win Rat

Resetting epsilon to 1.0
Episode: 2000, Win Rate: 0.29, Epsilon: 1.000, smartmove0.3333333333333333
Wins: 582, Losses: 1209, Draws: 210
Episode: 2100, Win Rate: 0.29, Epsilon: 0.901, smartmove0.35
Wins: 601, Losses: 1277, Draws: 223
Episode: 2200, Win Rate: 0.28, Epsilon: 0.802, smartmove0.36666666666666664
Wins: 624, Losses: 1343, Draws: 234
Episode: 2300, Win Rate: 0.28, Epsilon: 0.703, smartmove0.38333333333333336
Wins: 644, Losses: 1411, Draws: 246
Episode: 2400, Win Rate: 0.28, Epsilon: 0.604, smartmove0.4
Wins: 671, Losses: 1472, Draws: 258
Episode: 2500, Win Rate: 0.28, Epsilon: 0.505, smartmove0.4166666666666667
Wins: 693, Losses: 1547, Draws: 261
Episode: 2600, Win Rate: 0.28, Epsilon: 0.406, smartmove0.43333333333333335
Wins: 718, Losses: 1613, Draws: 270
Episode: 2700, Win Rate: 0.28, Epsilon: 0.307, smartmove0.45
Wins: 748, Losses: 1666, Draws: 287
Episode: 2800, Win Rate: 0.27, Epsilon: 0.208, smartmove0.4666666666666667
Wins: 766, Losses: 1736, Draws: 299
Episode: 2900, W

Resetting epsilon to 1.0
Episode: 3000, Win Rate: 0.27, Epsilon: 1.000, smartmove0.5
Wins: 803, Losses: 1884, Draws: 314
Episode: 3100, Win Rate: 0.26, Epsilon: 0.901, smartmove0.5166666666666667
Wins: 819, Losses: 1959, Draws: 323
Episode: 3200, Win Rate: 0.26, Epsilon: 0.802, smartmove0.5333333333333333
Wins: 837, Losses: 2030, Draws: 334
Episode: 3300, Win Rate: 0.26, Epsilon: 0.703, smartmove0.55
Wins: 859, Losses: 2096, Draws: 346
Episode: 3400, Win Rate: 0.26, Epsilon: 0.604, smartmove0.5666666666666667
Wins: 875, Losses: 2175, Draws: 351
Episode: 3500, Win Rate: 0.25, Epsilon: 0.505, smartmove0.5833333333333334
Wins: 891, Losses: 2249, Draws: 361
Episode: 3600, Win Rate: 0.25, Epsilon: 0.406, smartmove0.6
Wins: 908, Losses: 2328, Draws: 365
Episode: 3700, Win Rate: 0.25, Epsilon: 0.307, smartmove0.6166666666666667
Wins: 920, Losses: 2399, Draws: 382
Episode: 3800, Win Rate: 0.25, Epsilon: 0.208, smartmove0.6333333333333333
Wins: 935, Losses: 2466, Draws: 400
Episode: 3900, Win R

Resetting epsilon to 1.0
Episode: 4000, Win Rate: 0.24, Epsilon: 1.000, smartmove0.6666666666666666
Wins: 974, Losses: 2607, Draws: 420
Episode: 4100, Win Rate: 0.24, Epsilon: 0.901, smartmove0.6833333333333333
Wins: 986, Losses: 2689, Draws: 426
Episode: 4200, Win Rate: 0.24, Epsilon: 0.802, smartmove0.7
Wins: 994, Losses: 2770, Draws: 437
Episode: 4300, Win Rate: 0.23, Epsilon: 0.703, smartmove0.7166666666666667
Wins: 1004, Losses: 2846, Draws: 451
Episode: 4400, Win Rate: 0.23, Epsilon: 0.604, smartmove0.7333333333333333
Wins: 1011, Losses: 2931, Draws: 459
Episode: 4500, Win Rate: 0.23, Epsilon: 0.505, smartmove0.75
Wins: 1025, Losses: 3011, Draws: 465
Episode: 4600, Win Rate: 0.23, Epsilon: 0.406, smartmove0.7666666666666667
Wins: 1036, Losses: 3086, Draws: 479
Episode: 4700, Win Rate: 0.22, Epsilon: 0.307, smartmove0.7833333333333333
Wins: 1049, Losses: 3165, Draws: 487
Episode: 4800, Win Rate: 0.22, Epsilon: 0.208, smartmove0.8
Wins: 1058, Losses: 3242, Draws: 501
Episode: 4900,

Resetting epsilon to 1.0
Episode: 5000, Win Rate: 0.21, Epsilon: 1.000, smartmove0.8333333333333334
Wins: 1073, Losses: 3383, Draws: 545
Episode: 5100, Win Rate: 0.21, Epsilon: 0.901, smartmove0.85
Wins: 1078, Losses: 3468, Draws: 555
Episode: 5200, Win Rate: 0.21, Epsilon: 0.802, smartmove0.8666666666666667
Wins: 1084, Losses: 3553, Draws: 564
Episode: 5300, Win Rate: 0.20, Epsilon: 0.703, smartmove0.8833333333333333
Wins: 1084, Losses: 3640, Draws: 577
Episode: 5400, Win Rate: 0.20, Epsilon: 0.604, smartmove0.9
Wins: 1093, Losses: 3716, Draws: 592
Episode: 5500, Win Rate: 0.20, Epsilon: 0.505, smartmove0.9166666666666666
Wins: 1099, Losses: 3800, Draws: 602
Episode: 5600, Win Rate: 0.20, Epsilon: 0.406, smartmove0.9333333333333333
Wins: 1103, Losses: 3882, Draws: 616
Episode: 5700, Win Rate: 0.19, Epsilon: 0.307, smartmove0.95
Wins: 1108, Losses: 3960, Draws: 633
Episode: 5800, Win Rate: 0.19, Epsilon: 0.208, smartmove0.9666666666666667
Wins: 1112, Losses: 4042, Draws: 647
Episode: 5

Resetting epsilon to 1.0
Episode: 6000, Win Rate: 0.19, Epsilon: 1.000, smartmove1
Wins: 1116, Losses: 4194, Draws: 691
Episode: 6100, Win Rate: 0.18, Epsilon: 0.901, smartmove1
Wins: 1118, Losses: 4284, Draws: 699
Episode: 6200, Win Rate: 0.18, Epsilon: 0.802, smartmove1
Wins: 1120, Losses: 4373, Draws: 708
Episode: 6300, Win Rate: 0.18, Epsilon: 0.703, smartmove1
Wins: 1120, Losses: 4459, Draws: 722
Episode: 6400, Win Rate: 0.18, Epsilon: 0.604, smartmove1
Wins: 1121, Losses: 4539, Draws: 741
Episode: 6500, Win Rate: 0.17, Epsilon: 0.505, smartmove1
Wins: 1122, Losses: 4623, Draws: 756
Episode: 6600, Win Rate: 0.17, Epsilon: 0.406, smartmove1
Wins: 1125, Losses: 4710, Draws: 766
Episode: 6700, Win Rate: 0.17, Epsilon: 0.307, smartmove1
Wins: 1128, Losses: 4783, Draws: 790
Episode: 6800, Win Rate: 0.17, Epsilon: 0.208, smartmove1
Wins: 1130, Losses: 4858, Draws: 813
Episode: 6900, Win Rate: 0.16, Epsilon: 0.109, smartmove1
Wins: 1133, Losses: 4923, Draws: 845


Resetting epsilon to 1.0
Episode: 7000, Win Rate: 0.16, Epsilon: 1.000, smartmove1
Wins: 1138, Losses: 4988, Draws: 875
Episode: 7100, Win Rate: 0.16, Epsilon: 0.901, smartmove1
Wins: 1139, Losses: 5079, Draws: 883
Episode: 7200, Win Rate: 0.16, Epsilon: 0.802, smartmove1
Wins: 1143, Losses: 5165, Draws: 893
Episode: 7300, Win Rate: 0.16, Epsilon: 0.703, smartmove1
Wins: 1145, Losses: 5254, Draws: 902
Episode: 7400, Win Rate: 0.15, Epsilon: 0.604, smartmove1
Wins: 1146, Losses: 5337, Draws: 918
Episode: 7500, Win Rate: 0.15, Epsilon: 0.505, smartmove1
Wins: 1149, Losses: 5416, Draws: 936
Episode: 7600, Win Rate: 0.15, Epsilon: 0.406, smartmove1
Wins: 1149, Losses: 5500, Draws: 952
Episode: 7700, Win Rate: 0.15, Epsilon: 0.307, smartmove1
Wins: 1155, Losses: 5557, Draws: 989
Episode: 7800, Win Rate: 0.15, Epsilon: 0.208, smartmove1
Wins: 1158, Losses: 5618, Draws: 1025
Episode: 7900, Win Rate: 0.15, Epsilon: 0.109, smartmove1
Wins: 1165, Losses: 5672, Draws: 1064


Resetting epsilon to 1.0
Episode: 8000, Win Rate: 0.15, Epsilon: 1.000, smartmove1
Wins: 1172, Losses: 5729, Draws: 1100
Episode: 8100, Win Rate: 0.15, Epsilon: 0.901, smartmove1
Wins: 1175, Losses: 5812, Draws: 1114
Episode: 8200, Win Rate: 0.14, Epsilon: 0.802, smartmove1
Wins: 1177, Losses: 5896, Draws: 1128
Episode: 8300, Win Rate: 0.14, Epsilon: 0.703, smartmove1
Wins: 1180, Losses: 5981, Draws: 1140
Episode: 8400, Win Rate: 0.14, Epsilon: 0.604, smartmove1
Wins: 1184, Losses: 6057, Draws: 1160
Episode: 8500, Win Rate: 0.14, Epsilon: 0.505, smartmove1
Wins: 1188, Losses: 6130, Draws: 1183
Episode: 8600, Win Rate: 0.14, Epsilon: 0.406, smartmove1
Wins: 1190, Losses: 6202, Draws: 1209
Episode: 8700, Win Rate: 0.14, Epsilon: 0.307, smartmove1
Wins: 1194, Losses: 6253, Draws: 1254
Episode: 8800, Win Rate: 0.14, Epsilon: 0.208, smartmove1
Wins: 1200, Losses: 6304, Draws: 1297
Episode: 8900, Win Rate: 0.14, Epsilon: 0.109, smartmove1
Wins: 1207, Losses: 6354, Draws: 1340


Resetting epsilon to 1.0
Episode: 9000, Win Rate: 0.13, Epsilon: 1.000, smartmove1
Wins: 1211, Losses: 6403, Draws: 1387
Episode: 9100, Win Rate: 0.13, Epsilon: 0.901, smartmove1
Wins: 1211, Losses: 6491, Draws: 1399
Episode: 9200, Win Rate: 0.13, Epsilon: 0.802, smartmove1
Wins: 1213, Losses: 6578, Draws: 1410
Episode: 9300, Win Rate: 0.13, Epsilon: 0.703, smartmove1
Wins: 1216, Losses: 6663, Draws: 1422
Episode: 9400, Win Rate: 0.13, Epsilon: 0.604, smartmove1
Wins: 1217, Losses: 6744, Draws: 1440
Episode: 9500, Win Rate: 0.13, Epsilon: 0.505, smartmove1
Wins: 1219, Losses: 6821, Draws: 1461
Episode: 9600, Win Rate: 0.13, Epsilon: 0.406, smartmove1
Wins: 1224, Losses: 6885, Draws: 1492
Episode: 9700, Win Rate: 0.13, Epsilon: 0.307, smartmove1
Wins: 1229, Losses: 6948, Draws: 1524
Episode: 9800, Win Rate: 0.13, Epsilon: 0.208, smartmove1
Wins: 1233, Losses: 7004, Draws: 1564
Episode: 9900, Win Rate: 0.13, Epsilon: 0.109, smartmove1
Wins: 1239, Losses: 7056, Draws: 1606


In [2]:
import numpy as np
import random
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from TicTacToe import TicTacToe

class SQNAgent:
    def __init__(self, state_size=9, action_size=9, gamma=0.95):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = gamma  
        self.learning_rate = 0.001
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.9995
        self.batch_size = 32
        self.replay_buffer = deque(maxlen=10000)
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential([
            Dense(64, input_dim=self.state_size, activation='relu'),
            Dense(64, activation='relu'),
            Dense(self.action_size, activation='linear')
        ])
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def process_state(self, state):
        """
        Converts the state from [0,1,2] format to [-1,0,1] format for better learning.
        """
        processed_state = np.array(state).copy()
        processed_state[processed_state == 0] = -1
        processed_state[processed_state == 1] = 0
        processed_state[processed_state == 2] = 1
        return processed_state

    def select_action(self, state, valid_actions):
        if not valid_actions:
            return None

        if random.random() < self.epsilon:
            return random.choice(valid_actions)
        
        processed_state = self.process_state(state)
        q_values = self.model.predict(processed_state.reshape(1, -1), verbose=0)[0]
        valid_q_values = [(q_values[action], action) for action in valid_actions]
        return max(valid_q_values, key=lambda x: x[0])[1]

    def store_experience(self, state, action, reward, next_state, done):
        self.replay_buffer.append((state, action, reward, next_state, done))

    def train(self):
        if len(self.replay_buffer) < self.batch_size:
            return

        mini_batch = random.sample(self.replay_buffer, self.batch_size)
        states = np.array([self.process_state(exp[0]) for exp in mini_batch])
        next_states = np.array([self.process_state(exp[3]) for exp in mini_batch])
        current_q_values = self.model.predict(states, verbose=0)
        next_q_values = self.model.predict(next_states, verbose=0)

        x = []
        y = []

        for i, (state, action, reward, next_state, done) in enumerate(mini_batch):
            if done:
                target = reward
            else:
                next_valid_actions = [j for j, val in enumerate(next_state) if val == 0]
                if next_valid_actions:
                    max_next_q = max(next_q_values[i][action] for action in next_valid_actions)
                    target = reward + self.gamma * max_next_q
                else:
                    target = reward
            
            current_q = current_q_values[i].copy()
            current_q[action] = target
            x.append(self.process_state(state))
            y.append(current_q)

        self.model.fit(np.array(x), np.array(y), batch_size=32, epochs=1, verbose=0)

def train_agent(episodes=10000):
    agent = SQNAgent()
    history = {'wins': 0, 'losses': 0, 'draws': 0}
    initial_epsilon = 1.0
    min_epsilon = 0.01
    decay_episodes = 1000
    
    for episode in range(episodes):
        if episode % 1000 == 0 and episode > 0:
            agent.epsilon = 1.0
            print("Resetting epsilon to 1.0")
        else:
            epsilon_decay = (initial_epsilon - min_epsilon) / decay_episodes
            agent.epsilon = max(min_epsilon, initial_epsilon - (episode % 2000) * epsilon_decay)
        
        smartness = min(1, episode / (episodes))
        game = TicTacToe(smartMovePlayer1=smartness)
        state = np.array(game.board)
        
        game.player1_move()
        state = np.array(game.board)
        
        while True:
            valid_actions = game.empty_positions()
            if not valid_actions:
                history['draws'] += 1
                break
                
            action = agent.select_action(state, valid_actions)
            game.make_move(action, player=2)
            
            reward = 0
            if game.current_winner == 2:
                reward = 1
                history['wins'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            
            game.player1_move()
            if game.current_winner == 1:
                reward = -1
                history['losses'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            elif game.is_full():
                reward = -max(1,0.5*smartness)*8/10
                history['draws'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            
            agent.store_experience(state, action, reward, game.board, False)
            state = np.array(game.board)
        
        agent.train()
        
        if episode % 100 == 0:
            win_rate = history['wins'] / (episode + 1)
            print(f"Episode: {episode}, Win Rate: {win_rate:.2f}, Epsilon: {agent.epsilon:.3f}, smartmove{smartness}")
            print(f"Wins: {history['wins']}, Losses: {history['losses']}, Draws: {history['draws']}")
        
        if episode % 1000 == 0:
            agent.model.save(f'model7_episode_{episode}.h5')
    agent.model.save('model7.h5')
agent = train_agent()



/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Episode: 0, Win Rate: 1.00, Epsilon: 1.000, smartmove0.0
Wins: 1, Losses: 0, Draws: 0
Episode: 100, Win Rate: 0.27, Epsilon: 0.901, smartmove0.01
Wins: 27, Losses: 58, Draws: 16
Episode: 200, Win Rate: 0.27, Epsilon: 0.802, smartmove0.02
Wins: 54, Losses: 116, Draws: 31
Episode: 300, Win Rate: 0.28, Epsilon: 0.703, smartmove0.03
Wins: 85, Losses: 173, Draws: 43
Episode: 400, Win Rate: 0.27, Epsilon: 0.604, smartmove0.04
Wins: 108, Losses: 239, Draws: 54
Episode: 500, Win Rate: 0.28, Epsilon: 0.505, smartmove0.05
Wins: 141, Losses: 299, Draws: 61
Episode: 600, Win Rate: 0.29, Epsilon: 0.406, smartmove0.06
Wins: 174, Losses: 356, Draws: 71
Episode: 700, Win Rate: 0.29, Epsilon: 0.307, smartmove0.07
Wins: 203, Losses: 420, Draws: 78
Episode: 800, Win Rate: 0.29, Epsilon: 0.208, smartmove0.08
Wins: 230, Losses: 486, Draws: 85
Episode: 900, Win Rate: 0.29, Epsilon: 0.109, smartmove0.09
Wins: 264, Losses: 548, Draws: 89


Resetting epsilon to 1.0
Episode: 1000, Win Rate: 0.29, Epsilon: 1.000, smartmove0.1
Wins: 289, Losses: 617, Draws: 95
Episode: 1100, Win Rate: 0.29, Epsilon: 0.010, smartmove0.11
Wins: 318, Losses: 684, Draws: 99
Episode: 1200, Win Rate: 0.29, Epsilon: 0.010, smartmove0.12
Wins: 351, Losses: 743, Draws: 107
Episode: 1300, Win Rate: 0.30, Epsilon: 0.010, smartmove0.13
Wins: 390, Losses: 803, Draws: 108
Episode: 1400, Win Rate: 0.30, Epsilon: 0.010, smartmove0.14
Wins: 426, Losses: 863, Draws: 112
Episode: 1500, Win Rate: 0.30, Epsilon: 0.010, smartmove0.15
Wins: 455, Losses: 931, Draws: 115
Episode: 1600, Win Rate: 0.32, Epsilon: 0.010, smartmove0.16
Wins: 507, Losses: 977, Draws: 117
Episode: 1700, Win Rate: 0.32, Epsilon: 0.010, smartmove0.17
Wins: 547, Losses: 1035, Draws: 119
Episode: 1800, Win Rate: 0.32, Epsilon: 0.010, smartmove0.18
Wins: 570, Losses: 1106, Draws: 125
Episode: 1900, Win Rate: 0.32, Epsilon: 0.010, smartmove0.19
Wins: 604, Losses: 1171, Draws: 126


Resetting epsilon to 1.0
Episode: 2000, Win Rate: 0.32, Epsilon: 1.000, smartmove0.2
Wins: 644, Losses: 1227, Draws: 130
Episode: 2100, Win Rate: 0.32, Epsilon: 0.901, smartmove0.21
Wins: 669, Losses: 1294, Draws: 138
Episode: 2200, Win Rate: 0.31, Epsilon: 0.802, smartmove0.22
Wins: 693, Losses: 1360, Draws: 148
Episode: 2300, Win Rate: 0.31, Epsilon: 0.703, smartmove0.23
Wins: 721, Losses: 1424, Draws: 156
Episode: 2400, Win Rate: 0.31, Epsilon: 0.604, smartmove0.24
Wins: 750, Losses: 1488, Draws: 163
Episode: 2500, Win Rate: 0.31, Epsilon: 0.505, smartmove0.25
Wins: 786, Losses: 1537, Draws: 178
Episode: 2600, Win Rate: 0.31, Epsilon: 0.406, smartmove0.26
Wins: 809, Losses: 1605, Draws: 187
Episode: 2700, Win Rate: 0.31, Epsilon: 0.307, smartmove0.27
Wins: 841, Losses: 1664, Draws: 196
Episode: 2800, Win Rate: 0.31, Epsilon: 0.208, smartmove0.28
Wins: 867, Losses: 1731, Draws: 203
Episode: 2900, Win Rate: 0.31, Epsilon: 0.109, smartmove0.29
Wins: 896, Losses: 1796, Draws: 209


Resetting epsilon to 1.0
Episode: 3000, Win Rate: 0.31, Epsilon: 1.000, smartmove0.3
Wins: 926, Losses: 1863, Draws: 212
Episode: 3100, Win Rate: 0.30, Epsilon: 0.010, smartmove0.31
Wins: 945, Losses: 1936, Draws: 220
Episode: 3200, Win Rate: 0.30, Epsilon: 0.010, smartmove0.32
Wins: 973, Losses: 1998, Draws: 230
Episode: 3300, Win Rate: 0.30, Epsilon: 0.010, smartmove0.33
Wins: 1004, Losses: 2060, Draws: 237
Episode: 3400, Win Rate: 0.30, Epsilon: 0.010, smartmove0.34
Wins: 1032, Losses: 2120, Draws: 249
Episode: 3500, Win Rate: 0.30, Epsilon: 0.010, smartmove0.35
Wins: 1057, Losses: 2188, Draws: 256
Episode: 3600, Win Rate: 0.30, Epsilon: 0.010, smartmove0.36
Wins: 1086, Losses: 2247, Draws: 268
Episode: 3700, Win Rate: 0.30, Epsilon: 0.010, smartmove0.37
Wins: 1107, Losses: 2320, Draws: 274
Episode: 3800, Win Rate: 0.30, Epsilon: 0.010, smartmove0.38
Wins: 1132, Losses: 2387, Draws: 282
Episode: 3900, Win Rate: 0.30, Epsilon: 0.010, smartmove0.39
Wins: 1166, Losses: 2445, Draws: 290

...

Resetting epsilon to 1.0
Episode: 5000, Win Rate: 0.28, Epsilon: 1.000, smartmove0.5
Wins: 1383, Losses: 3215, Draws: 403
Episode: 5100, Win Rate: 0.28, Epsilon: 0.010, smartmove0.51
Wins: 1410, Losses: 3278, Draws: 413
Episode: 5200, Win Rate: 0.28, Epsilon: 0.010, smartmove0.52
Wins: 1439, Losses: 3338, Draws: 424
Episode: 5300, Win Rate: 0.28, Epsilon: 0.010, smartmove0.53
Wins: 1463, Losses: 3400, Draws: 438
Episode: 5400, Win Rate: 0.27, Epsilon: 0.010, smartmove0.54
Wins: 1479, Losses: 3465, Draws: 457
Episode: 5500, Win Rate: 0.28, Epsilon: 0.010, smartmove0.55
Wins: 1514, Losses: 3522, Draws: 465
Episode: 5600, Win Rate: 0.28, Epsilon: 0.010, smartmove0.56
Wins: 1544, Losses: 3581, Draws: 476
Episode: 5700, Win Rate: 0.28, Epsilon: 0.010, smartmove0.57
Wins: 1571, Losses: 3642, Draws: 488
Episode: 5800, Win Rate: 0.28, Epsilon: 0.010, smartmove0.58
Wins: 1599, Losses: 3699, Draws: 503
Episode: 5900, Win Rate: 0.27, Epsilon: 0.010, smartmove0.59
Wins: 1620, Losses: 3761, Draws: 

Resetting epsilon to 1.0
Episode: 6000, Win Rate: 0.27, Epsilon: 1.000, smartmove0.6
Wins: 1643, Losses: 3825, Draws: 533
Episode: 6100, Win Rate: 0.27, Epsilon: 0.901, smartmove0.61
Wins: 1651, Losses: 3904, Draws: 546
Episode: 6200, Win Rate: 0.27, Epsilon: 0.802, smartmove0.62
Wins: 1670, Losses: 3973, Draws: 558
Episode: 6300, Win Rate: 0.27, Epsilon: 0.703, smartmove0.63
Wins: 1675, Losses: 4058, Draws: 568
Episode: 6400, Win Rate: 0.26, Epsilon: 0.604, smartmove0.64
Wins: 1686, Losses: 4137, Draws: 578
Episode: 6500, Win Rate: 0.26, Epsilon: 0.505, smartmove0.65
Wins: 1701, Losses: 4208, Draws: 592
Episode: 6600, Win Rate: 0.26, Epsilon: 0.406, smartmove0.66
Wins: 1720, Losses: 4276, Draws: 605
Episode: 6700, Win Rate: 0.26, Epsilon: 0.307, smartmove0.67
Wins: 1736, Losses: 4349, Draws: 616
Episode: 6800, Win Rate: 0.26, Epsilon: 0.208, smartmove0.68
Wins: 1752, Losses: 4416, Draws: 633
Episode: 6900, Win Rate: 0.26, Epsilon: 0.109, smartmove0.69
Wins: 1777, Losses: 4475, Draws: 

Resetting epsilon to 1.0
Episode: 7000, Win Rate: 0.26, Epsilon: 1.000, smartmove0.7
Wins: 1797, Losses: 4538, Draws: 666
Episode: 7100, Win Rate: 0.25, Epsilon: 0.010, smartmove0.71
Wins: 1806, Losses: 4609, Draws: 686
Episode: 7200, Win Rate: 0.25, Epsilon: 0.010, smartmove0.72
Wins: 1827, Losses: 4672, Draws: 702
Episode: 7300, Win Rate: 0.25, Epsilon: 0.010, smartmove0.73
Wins: 1844, Losses: 4736, Draws: 721
Episode: 7400, Win Rate: 0.25, Epsilon: 0.010, smartmove0.74
Wins: 1866, Losses: 4794, Draws: 741
Episode: 7500, Win Rate: 0.25, Epsilon: 0.010, smartmove0.75
Wins: 1892, Losses: 4848, Draws: 761
Episode: 7600, Win Rate: 0.25, Epsilon: 0.010, smartmove0.76
Wins: 1910, Losses: 4901, Draws: 790
Episode: 7700, Win Rate: 0.25, Epsilon: 0.010, smartmove0.77
Wins: 1931, Losses: 4956, Draws: 814
Episode: 7800, Win Rate: 0.25, Epsilon: 0.010, smartmove0.78
Wins: 1946, Losses: 5018, Draws: 837
Episode: 7900, Win Rate: 0.25, Epsilon: 0.010, smartmove0.79
Wins: 1961, Losses: 5081, Draws: 

Resetting epsilon to 1.0
Episode: 8000, Win Rate: 0.25, Epsilon: 1.000, smartmove0.8
Wins: 1977, Losses: 5140, Draws: 884
Episode: 8100, Win Rate: 0.24, Epsilon: 0.901, smartmove0.81
Wins: 1981, Losses: 5223, Draws: 897
Episode: 8200, Win Rate: 0.24, Epsilon: 0.802, smartmove0.82
Wins: 1989, Losses: 5300, Draws: 912
Episode: 8300, Win Rate: 0.24, Epsilon: 0.703, smartmove0.83
Wins: 1997, Losses: 5381, Draws: 923
Episode: 8400, Win Rate: 0.24, Epsilon: 0.604, smartmove0.84
Wins: 2005, Losses: 5464, Draws: 932
Episode: 8500, Win Rate: 0.24, Epsilon: 0.505, smartmove0.85
Wins: 2016, Losses: 5535, Draws: 950
Episode: 8600, Win Rate: 0.24, Epsilon: 0.406, smartmove0.86
Wins: 2026, Losses: 5609, Draws: 966
Episode: 8700, Win Rate: 0.23, Epsilon: 0.307, smartmove0.87
Wins: 2035, Losses: 5677, Draws: 989
Episode: 8800, Win Rate: 0.23, Epsilon: 0.208, smartmove0.88
Wins: 2042, Losses: 5756, Draws: 1003
Episode: 8900, Win Rate: 0.23, Epsilon: 0.109, smartmove0.89
Wins: 2054, Losses: 5817, Draws:

Resetting epsilon to 1.0
Episode: 9000, Win Rate: 0.23, Epsilon: 1.000, smartmove0.9
Wins: 2061, Losses: 5872, Draws: 1068
Episode: 9100, Win Rate: 0.23, Epsilon: 0.010, smartmove0.91
Wins: 2073, Losses: 5926, Draws: 1102
Episode: 9200, Win Rate: 0.23, Epsilon: 0.010, smartmove0.92
Wins: 2085, Losses: 5981, Draws: 1135
Episode: 9300, Win Rate: 0.22, Epsilon: 0.010, smartmove0.93
Wins: 2089, Losses: 6040, Draws: 1172
Episode: 9400, Win Rate: 0.22, Epsilon: 0.010, smartmove0.94
Wins: 2099, Losses: 6104, Draws: 1198
Episode: 9500, Win Rate: 0.22, Epsilon: 0.010, smartmove0.95
Wins: 2103, Losses: 6170, Draws: 1228
Episode: 9600, Win Rate: 0.22, Epsilon: 0.010, smartmove0.96
Wins: 2106, Losses: 6234, Draws: 1261
Episode: 9700, Win Rate: 0.22, Epsilon: 0.010, smartmove0.97
Wins: 2115, Losses: 6288, Draws: 1298
Episode: 9800, Win Rate: 0.22, Epsilon: 0.010, smartmove0.98
Wins: 2127, Losses: 6338, Draws: 1336
Episode: 9900, Win Rate: 0.22, Epsilon: 0.010, smartmove0.99
Wins: 2131, Losses: 6385

In [3]:
import numpy as np
import random
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization, Dropout
from TicTacToe import TicTacToe

class SQNAgent:
    def __init__(self, state_size=9, action_size=9, gamma=0.99):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = gamma  
        self.learning_rate = 0.0005  # Reduced learning rate for more stable learning
        self.epsilon = 1.0
        self.epsilon_min = 0.05
        self.epsilon_decay = 0.85
        self.batch_size = 64  # Increased batch size
        self.replay_buffer = deque(maxlen=50000)  # Increased buffer size
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_counter = 0
        self.update_target_frequency = 1000

    def _build_model(self):
        model = Sequential([
            Dense(64, input_dim=self.state_size, activation='relu'),
            Dense(64, activation='relu'),
            Dense(self.action_size, activation='linear')
        ])
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def process_state(self, state):
        processed_state = np.array(state).copy()
        processed_state[processed_state == 0] = -1
        processed_state[processed_state == 1] = 0
        processed_state[processed_state == 2] = 1
        return processed_state

    def select_action(self, state, valid_actions):
        if not valid_actions:
            return None

        if random.random() < self.epsilon:
            return random.choice(valid_actions)
        
        processed_state = self.process_state(state)
        q_values = self.model.predict(processed_state.reshape(1, -1), verbose=0)[0]
        
        # Prioritize winning moves and block opponent wins
        temp_board = np.array(state).copy()
        for action in valid_actions:
            # Check for winning move
            temp_board[action] = 2
            if self._check_win(temp_board, 2):
                return action
            temp_board[action] = 0
            
            # Check for blocking opponent win
            temp_board[action] = 1
            if self._check_win(temp_board, 1):
                return action
            temp_board[action] = 0

        # If no immediate winning/blocking moves, use Q-values
        valid_q_values = [(q_values[action], action) for action in valid_actions]
        return max(valid_q_values, key=lambda x: x[0])[1]

    def _check_win(self, board, player):
        win_combinations = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8],  # Rows
            [0, 3, 6], [1, 4, 7], [2, 5, 8],  # Columns
            [0, 4, 8], [2, 4, 6]  # Diagonals
        ]
        return any(all(board[i] == player for i in combo) for combo in win_combinations)

    def store_experience(self, state, action, reward, next_state, done):
        self.replay_buffer.append((state, action, reward, next_state, done))

    def train(self):
        if len(self.replay_buffer) < self.batch_size:
            return

        mini_batch = random.sample(self.replay_buffer, self.batch_size)
        states = np.array([self.process_state(exp[0]) for exp in mini_batch])
        next_states = np.array([self.process_state(exp[3]) for exp in mini_batch])
        
        # Use target network for more stable training
        current_q_values = self.model.predict(states, verbose=0)
        next_q_values = self.target_model.predict(next_states, verbose=0)

        x = []
        y = []

        for i, (state, action, reward, next_state, done) in enumerate(mini_batch):
            if done:
                target = reward
            else:
                next_valid_actions = [j for j, val in enumerate(next_state) if val == 0]
                if next_valid_actions:
                    # Double DQN: Select action using online network, evaluate using target network
                    next_q = self.model.predict(next_states[i:i+1], verbose=0)[0]
                    best_action = max([(next_q[a], a) for a in next_valid_actions], key=lambda x: x[0])[1]
                    target = reward + self.gamma * next_q_values[i][best_action]
                else:
                    target = reward
            
            current_q = current_q_values[i].copy()
            current_q[action] = target
            x.append(self.process_state(state))
            y.append(current_q)

        self.model.fit(np.array(x), np.array(y), batch_size=self.batch_size, epochs=1, verbose=0)
        
        # Update target network periodically
        self.update_target_counter += 1
        if self.update_target_counter >= self.update_target_frequency:
            self.target_model.set_weights(self.model.get_weights())
            self.update_target_counter = 0

def train_agent(episodes=10000):
    agent = SQNAgent()
    history = {'wins': 0, 'losses': 0, 'draws': 0}
    best_win_rate = 0
    no_improvement_counter = 0
    smartness = 0
    for episode in range(episodes):
        # Decrease epsilon every 100 iterations
        if episode % 100 == 0 and episode%1000!=0:
            agent.epsilon = max(agent.epsilon_min, agent.epsilon * agent.epsilon_decay)

        # Increase smartness and partially reset epsilon every 2000 iterations
        if episode % 1000 == 0 and episode > 0:
            smartness = min(0.8, smartness + 0.1)
            agent.epsilon = 1  # Partial epsilon reset
            
        game = TicTacToe(smartMovePlayer1=smartness)
        state = np.array(game.board)
        
        
        
        game.player1_move()
        state = np.array(game.board)
        
        while True:
            valid_actions = game.empty_positions()
            if not valid_actions:
                history['draws'] += 1
                break
                
            action = agent.select_action(state, valid_actions)
            game.make_move(action, player=2)
            
            reward = 0
            if game.current_winner == 2:
                reward = 1 + (0.1 * smartness)  # Higher reward for winning against smarter opponent
                history['wins'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            
            if not game.is_full():
                game.player1_move()
                
            if game.current_winner == 1:
                reward = -1
                history['losses'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            elif game.is_full():
                reward = -0.1 * smartness  # Small negative reward for draws against smart opponent
                history['draws'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            
            agent.store_experience(state, action, reward, game.board, False)
            state = np.array(game.board)
        
        agent.train()
        
        # Evaluation and model saving logic
        if episode % 100 == 0:
            win_rate = history['wins'] / (episode + 1)
            print(f"Episode: {episode}, Win Rate: {win_rate:.2f}, Epsilon: {agent.epsilon:.3f}, Smartness: {smartness}")
            print(f"Wins: {history['wins']}, Losses: {history['losses']}, Draws: {history['draws']}")
            
            if win_rate > best_win_rate:
                best_win_rate = win_rate
                no_improvement_counter = 0
            else:
                no_improvement_counter += 1
            
            if episode % 1000 == 0:
                agent.model.save(f'model9_episode_{episode}.h5')
        
    agent.model.save('model9.h5')
    return agent

agent = train_agent()


Episode: 0, Win Rate: 0.00, Epsilon: 1.000, Smartness: 0
Wins: 0, Losses: 1, Draws: 0
Episode: 100, Win Rate: 0.24, Epsilon: 0.850, Smartness: 0
Wins: 24, Losses: 60, Draws: 17
Episode: 200, Win Rate: 0.26, Epsilon: 0.722, Smartness: 0
Wins: 52, Losses: 120, Draws: 29
Episode: 300, Win Rate: 0.28, Epsilon: 0.614, Smartness: 0
Wins: 85, Losses: 169, Draws: 47
Episode: 400, Win Rate: 0.30, Epsilon: 0.522, Smartness: 0
Wins: 122, Losses: 210, Draws: 69
Episode: 500, Win Rate: 0.33, Epsilon: 0.444, Smartness: 0
Wins: 164, Losses: 240, Draws: 97
Episode: 600, Win Rate: 0.33, Epsilon: 0.377, Smartness: 0
Wins: 198, Losses: 277, Draws: 126
Episode: 700, Win Rate: 0.35, Epsilon: 0.321, Smartness: 0
Wins: 247, Losses: 301, Draws: 153
Episode: 800, Win Rate: 0.36, Epsilon: 0.272, Smartness: 0
Wins: 291, Losses: 332, Draws: 178
Episode: 900, Win Rate: 0.37, Epsilon: 0.232, Smartness: 0
Wins: 335, Losses: 356, Draws: 210


Episode: 1000, Win Rate: 0.39, Epsilon: 1.000, Smartness: 0.1
Wins: 391, Losses: 382, Draws: 228
Episode: 1100, Win Rate: 0.38, Epsilon: 0.850, Smartness: 0.1
Wins: 420, Losses: 441, Draws: 240
Episode: 1200, Win Rate: 0.37, Epsilon: 0.722, Smartness: 0.1
Wins: 449, Losses: 498, Draws: 254
Episode: 1300, Win Rate: 0.38, Epsilon: 0.614, Smartness: 0.1
Wins: 488, Losses: 547, Draws: 266
Episode: 1400, Win Rate: 0.37, Epsilon: 0.522, Smartness: 0.1
Wins: 525, Losses: 591, Draws: 285
Episode: 1500, Win Rate: 0.37, Epsilon: 0.444, Smartness: 0.1
Wins: 560, Losses: 631, Draws: 310
Episode: 1600, Win Rate: 0.38, Epsilon: 0.377, Smartness: 0.1
Wins: 612, Losses: 657, Draws: 332
Episode: 1700, Win Rate: 0.39, Epsilon: 0.321, Smartness: 0.1
Wins: 657, Losses: 691, Draws: 353
Episode: 1800, Win Rate: 0.39, Epsilon: 0.272, Smartness: 0.1
Wins: 697, Losses: 720, Draws: 384
Episode: 1900, Win Rate: 0.39, Epsilon: 0.232, Smartness: 0.1
Wins: 748, Losses: 747, Draws: 406


Episode: 2000, Win Rate: 0.39, Epsilon: 1.000, Smartness: 0.2
Wins: 789, Losses: 775, Draws: 437
Episode: 2100, Win Rate: 0.38, Epsilon: 0.850, Smartness: 0.2
Wins: 808, Losses: 848, Draws: 445
Episode: 2200, Win Rate: 0.38, Epsilon: 0.722, Smartness: 0.2
Wins: 838, Losses: 903, Draws: 460
Episode: 2300, Win Rate: 0.38, Epsilon: 0.614, Smartness: 0.2
Wins: 868, Losses: 954, Draws: 479
Episode: 2400, Win Rate: 0.37, Epsilon: 0.522, Smartness: 0.2
Wins: 897, Losses: 1010, Draws: 494
Episode: 2500, Win Rate: 0.37, Epsilon: 0.444, Smartness: 0.2
Wins: 928, Losses: 1056, Draws: 517
Episode: 2600, Win Rate: 0.37, Epsilon: 0.377, Smartness: 0.2
Wins: 961, Losses: 1103, Draws: 537
Episode: 2700, Win Rate: 0.37, Epsilon: 0.321, Smartness: 0.2
Wins: 1010, Losses: 1135, Draws: 556
Episode: 2800, Win Rate: 0.38, Epsilon: 0.272, Smartness: 0.2
Wins: 1059, Losses: 1167, Draws: 575
Episode: 2900, Win Rate: 0.38, Epsilon: 0.232, Smartness: 0.2
Wins: 1106, Losses: 1193, Draws: 602


Episode: 3000, Win Rate: 0.38, Epsilon: 1.000, Smartness: 0.30000000000000004
Wins: 1150, Losses: 1226, Draws: 625
Episode: 3100, Win Rate: 0.38, Epsilon: 0.850, Smartness: 0.30000000000000004
Wins: 1174, Losses: 1294, Draws: 633
Episode: 3200, Win Rate: 0.37, Epsilon: 0.722, Smartness: 0.30000000000000004
Wins: 1189, Losses: 1362, Draws: 650
Episode: 3300, Win Rate: 0.37, Epsilon: 0.614, Smartness: 0.30000000000000004
Wins: 1220, Losses: 1416, Draws: 665
Episode: 3400, Win Rate: 0.37, Epsilon: 0.522, Smartness: 0.30000000000000004
Wins: 1250, Losses: 1471, Draws: 680
Episode: 3500, Win Rate: 0.37, Epsilon: 0.444, Smartness: 0.30000000000000004
Wins: 1282, Losses: 1509, Draws: 710
Episode: 3600, Win Rate: 0.36, Epsilon: 0.377, Smartness: 0.30000000000000004
Wins: 1313, Losses: 1552, Draws: 736
Episode: 3700, Win Rate: 0.36, Epsilon: 0.321, Smartness: 0.30000000000000004
Wins: 1347, Losses: 1593, Draws: 761
Episode: 3800, Win Rate: 0.36, Epsilon: 0.272, Smartness: 0.30000000000000004
Wi

Episode: 4000, Win Rate: 0.37, Epsilon: 1.000, Smartness: 0.4
Wins: 1466, Losses: 1704, Draws: 831
Episode: 4100, Win Rate: 0.36, Epsilon: 0.850, Smartness: 0.4
Wins: 1479, Losses: 1776, Draws: 846
Episode: 4200, Win Rate: 0.36, Epsilon: 0.722, Smartness: 0.4
Wins: 1497, Losses: 1846, Draws: 858
Episode: 4300, Win Rate: 0.35, Epsilon: 0.614, Smartness: 0.4
Wins: 1520, Losses: 1909, Draws: 872
Episode: 4400, Win Rate: 0.35, Epsilon: 0.522, Smartness: 0.4
Wins: 1546, Losses: 1966, Draws: 889
Episode: 4500, Win Rate: 0.35, Epsilon: 0.444, Smartness: 0.4
Wins: 1571, Losses: 2011, Draws: 919
Episode: 4600, Win Rate: 0.35, Epsilon: 0.377, Smartness: 0.4
Wins: 1603, Losses: 2052, Draws: 946
Episode: 4700, Win Rate: 0.35, Epsilon: 0.321, Smartness: 0.4
Wins: 1637, Losses: 2092, Draws: 972
Episode: 4800, Win Rate: 0.35, Epsilon: 0.272, Smartness: 0.4
Wins: 1675, Losses: 2130, Draws: 996
Episode: 4900, Win Rate: 0.35, Epsilon: 0.232, Smartness: 0.4
Wins: 1708, Losses: 2162, Draws: 1031


Episode: 5000, Win Rate: 0.35, Epsilon: 1.000, Smartness: 0.5
Wins: 1741, Losses: 2203, Draws: 1057
Episode: 5100, Win Rate: 0.34, Epsilon: 0.850, Smartness: 0.5
Wins: 1746, Losses: 2289, Draws: 1066
Episode: 5200, Win Rate: 0.34, Epsilon: 0.722, Smartness: 0.5
Wins: 1765, Losses: 2357, Draws: 1079
Episode: 5300, Win Rate: 0.34, Epsilon: 0.614, Smartness: 0.5
Wins: 1784, Losses: 2421, Draws: 1096
Episode: 5400, Win Rate: 0.33, Epsilon: 0.522, Smartness: 0.5
Wins: 1807, Losses: 2476, Draws: 1118
Episode: 5500, Win Rate: 0.33, Epsilon: 0.444, Smartness: 0.5
Wins: 1831, Losses: 2527, Draws: 1143
Episode: 5600, Win Rate: 0.33, Epsilon: 0.377, Smartness: 0.5
Wins: 1865, Losses: 2574, Draws: 1162
Episode: 5700, Win Rate: 0.33, Epsilon: 0.321, Smartness: 0.5
Wins: 1895, Losses: 2611, Draws: 1195
Episode: 5800, Win Rate: 0.33, Epsilon: 0.272, Smartness: 0.5
Wins: 1924, Losses: 2652, Draws: 1225
Episode: 5900, Win Rate: 0.33, Epsilon: 0.232, Smartness: 0.5
Wins: 1950, Losses: 2695, Draws: 1256


Episode: 6000, Win Rate: 0.33, Epsilon: 1.000, Smartness: 0.6
Wins: 1985, Losses: 2729, Draws: 1287
Episode: 6100, Win Rate: 0.33, Epsilon: 0.850, Smartness: 0.6
Wins: 1999, Losses: 2799, Draws: 1303
Episode: 6200, Win Rate: 0.32, Epsilon: 0.722, Smartness: 0.6
Wins: 2012, Losses: 2872, Draws: 1317
Episode: 6300, Win Rate: 0.32, Epsilon: 0.614, Smartness: 0.6
Wins: 2029, Losses: 2939, Draws: 1333
Episode: 6400, Win Rate: 0.32, Epsilon: 0.522, Smartness: 0.6
Wins: 2044, Losses: 3007, Draws: 1350
Episode: 6500, Win Rate: 0.32, Epsilon: 0.444, Smartness: 0.6
Wins: 2069, Losses: 3063, Draws: 1369
Episode: 6600, Win Rate: 0.32, Epsilon: 0.377, Smartness: 0.6
Wins: 2092, Losses: 3113, Draws: 1396
Episode: 6700, Win Rate: 0.32, Epsilon: 0.321, Smartness: 0.6
Wins: 2112, Losses: 3165, Draws: 1424
Episode: 6800, Win Rate: 0.31, Epsilon: 0.272, Smartness: 0.6
Wins: 2135, Losses: 3216, Draws: 1450
Episode: 6900, Win Rate: 0.31, Epsilon: 0.232, Smartness: 0.6
Wins: 2159, Losses: 3259, Draws: 1483


Episode: 7000, Win Rate: 0.31, Epsilon: 1.000, Smartness: 0.7
Wins: 2190, Losses: 3298, Draws: 1513
Episode: 7100, Win Rate: 0.31, Epsilon: 0.850, Smartness: 0.7
Wins: 2198, Losses: 3381, Draws: 1522
Episode: 7200, Win Rate: 0.31, Epsilon: 0.722, Smartness: 0.7
Wins: 2209, Losses: 3455, Draws: 1537
Episode: 7300, Win Rate: 0.30, Epsilon: 0.614, Smartness: 0.7
Wins: 2218, Losses: 3531, Draws: 1552
Episode: 7400, Win Rate: 0.30, Epsilon: 0.522, Smartness: 0.7
Wins: 2229, Losses: 3596, Draws: 1576
Episode: 7500, Win Rate: 0.30, Epsilon: 0.444, Smartness: 0.7
Wins: 2249, Losses: 3648, Draws: 1604
Episode: 7600, Win Rate: 0.30, Epsilon: 0.377, Smartness: 0.7
Wins: 2268, Losses: 3704, Draws: 1629
Episode: 7700, Win Rate: 0.30, Epsilon: 0.321, Smartness: 0.7
Wins: 2281, Losses: 3763, Draws: 1657
Episode: 7800, Win Rate: 0.29, Epsilon: 0.272, Smartness: 0.7
Wins: 2301, Losses: 3804, Draws: 1696
Episode: 7900, Win Rate: 0.29, Epsilon: 0.232, Smartness: 0.7
Wins: 2322, Losses: 3835, Draws: 1744


Episode: 8000, Win Rate: 0.29, Epsilon: 1.000, Smartness: 0.7999999999999999
Wins: 2337, Losses: 3881, Draws: 1783
Episode: 8100, Win Rate: 0.29, Epsilon: 0.850, Smartness: 0.7999999999999999
Wins: 2344, Losses: 3961, Draws: 1796
Episode: 8200, Win Rate: 0.29, Epsilon: 0.722, Smartness: 0.7999999999999999
Wins: 2354, Losses: 4037, Draws: 1810
Episode: 8300, Win Rate: 0.28, Epsilon: 0.614, Smartness: 0.7999999999999999
Wins: 2363, Losses: 4102, Draws: 1836
Episode: 8400, Win Rate: 0.28, Epsilon: 0.522, Smartness: 0.7999999999999999
Wins: 2378, Losses: 4169, Draws: 1854
Episode: 8500, Win Rate: 0.28, Epsilon: 0.444, Smartness: 0.7999999999999999
Wins: 2388, Losses: 4233, Draws: 1880
Episode: 8600, Win Rate: 0.28, Epsilon: 0.377, Smartness: 0.7999999999999999
Wins: 2396, Losses: 4295, Draws: 1910
Episode: 8700, Win Rate: 0.28, Epsilon: 0.321, Smartness: 0.7999999999999999
Wins: 2408, Losses: 4352, Draws: 1941
Episode: 8800, Win Rate: 0.28, Epsilon: 0.272, Smartness: 0.7999999999999999
Win

Episode: 9000, Win Rate: 0.27, Epsilon: 1.000, Smartness: 0.8
Wins: 2438, Losses: 4497, Draws: 2066
Episode: 9100, Win Rate: 0.27, Epsilon: 0.850, Smartness: 0.8
Wins: 2442, Losses: 4585, Draws: 2074
Episode: 9200, Win Rate: 0.27, Epsilon: 0.722, Smartness: 0.8
Wins: 2448, Losses: 4660, Draws: 2093
Episode: 9300, Win Rate: 0.26, Epsilon: 0.614, Smartness: 0.8
Wins: 2457, Losses: 4735, Draws: 2109
Episode: 9400, Win Rate: 0.26, Epsilon: 0.522, Smartness: 0.8
Wins: 2469, Losses: 4799, Draws: 2133
Episode: 9500, Win Rate: 0.26, Epsilon: 0.444, Smartness: 0.8
Wins: 2482, Losses: 4850, Draws: 2169
Episode: 9600, Win Rate: 0.26, Epsilon: 0.377, Smartness: 0.8
Wins: 2490, Losses: 4908, Draws: 2203
Episode: 9700, Win Rate: 0.26, Epsilon: 0.321, Smartness: 0.8
Wins: 2503, Losses: 4963, Draws: 2235
Episode: 9800, Win Rate: 0.26, Epsilon: 0.272, Smartness: 0.8
Wins: 2513, Losses: 5012, Draws: 2276
Episode: 9900, Win Rate: 0.26, Epsilon: 0.232, Smartness: 0.8
Wins: 2529, Losses: 5056, Draws: 2316


In [4]:
import numpy as np
import random
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from TicTacToe import TicTacToe

class SQNAgent:
    def __init__(self, state_size=9, action_size=9, gamma=0.99):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = gamma  
        self.learning_rate = 0.0005  # Reduced learning rate for more stable learning
        self.epsilon = 1.0
        self.epsilon_min = 0.05
        self.epsilon_decay = 0.85
        self.batch_size = 64  # Increased batch size
        self.replay_buffer = deque(maxlen=50000)  # Increased buffer size
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_counter = 0
        self.update_target_frequency = 1000

    def _build_model(self):
        model = Sequential([
            Dense(64, input_dim=self.state_size, activation='relu'),
            Dense(64, activation='relu'),
            Dense(self.action_size, activation='linear')
        ])
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def process_state(self, state):
        processed_state = np.array(state).copy()
        processed_state[processed_state == 0] = -1
        processed_state[processed_state == 1] = 0
        processed_state[processed_state == 2] = 1
        return processed_state

    def select_action(self, state, valid_actions):
        if not valid_actions:
            return None

        if random.random() < self.epsilon:
            return random.choice(valid_actions)
        
        processed_state = self.process_state(state)
        q_values = self.model.predict(processed_state.reshape(1, -1), verbose=0)[0]
        
        # Prioritize winning moves and block opponent wins
        temp_board = np.array(state).copy()
        for action in valid_actions:
            # Check for winning move
            temp_board[action] = 2
            if self._check_win(temp_board, 2):
                return action
            temp_board[action] = 0
            
            # Check for blocking opponent win
            temp_board[action] = 1
            if self._check_win(temp_board, 1):
                return action
            temp_board[action] = 0

        # If no immediate winning/blocking moves, use Q-values
        valid_q_values = [(q_values[action], action) for action in valid_actions]
        return max(valid_q_values, key=lambda x: x[0])[1]

    def _check_win(self, board, player):
        win_combinations = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8],  # Rows
            [0, 3, 6], [1, 4, 7], [2, 5, 8],  # Columns
            [0, 4, 8], [2, 4, 6]  # Diagonals
        ]
        return any(all(board[i] == player for i in combo) for combo in win_combinations)

    def store_experience(self, state, action, reward, next_state, done):
        self.replay_buffer.append((state, action, reward, next_state, done))

    def train(self):
        if len(self.replay_buffer) < self.batch_size:
            return

        mini_batch = random.sample(self.replay_buffer, self.batch_size)
        states = np.array([self.process_state(exp[0]) for exp in mini_batch])
        next_states = np.array([self.process_state(exp[3]) for exp in mini_batch])
        
        # Directly use the target network's Q-values
        current_q_values = self.model.predict(states, verbose=0)
        next_q_values = self.target_model.predict(next_states, verbose=0)

        x = []
        y = []

        for i, (state, action, reward, next_state, done) in enumerate(mini_batch):
            if done:
                target = reward
            else:
                target = reward + self.gamma * max(next_q_values[i])
            
            current_q = current_q_values[i].copy()
            current_q[action] = target
            x.append(self.process_state(state))
            y.append(current_q)

        self.model.fit(np.array(x), np.array(y), batch_size=self.batch_size, epochs=1, verbose=0)
        
        # Update target network periodically
        self.update_target_counter += 1
        if self.update_target_counter >= self.update_target_frequency:
            self.target_model.set_weights(self.model.get_weights())
            self.update_target_counter = 0

def train_agent(episodes=10000):
    agent = SQNAgent()
    history = {'wins': 0, 'losses': 0, 'draws': 0}
    best_win_rate = 0
    no_improvement_counter = 0
    smartness = 0
    for episode in range(episodes):
        # Decrease epsilon every 100 iterations
        if episode % 100 == 0 and episode % 1000 != 0:
            agent.epsilon = max(agent.epsilon_min, agent.epsilon * agent.epsilon_decay)

        # Increase smartness and partially reset epsilon every 2000 iterations
        if episode % 1000 == 0 and episode > 0:
            smartness = min(0.8, smartness + 0.1)
            agent.epsilon = 1  # Partial epsilon reset
            
        game = TicTacToe(smartMovePlayer1=smartness)
        state = np.array(game.board)
        
        game.player1_move()
        state = np.array(game.board)
        
        while True:
            valid_actions = game.empty_positions()
            if not valid_actions:
                history['draws'] += 1
                break
                
            action = agent.select_action(state, valid_actions)
            game.make_move(action, player=2)
            
            reward = 0
            if game.current_winner == 2:
                reward = 1 + (0.1 * smartness)  # Higher reward for winning against smarter opponent
                history['wins'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            
            if not game.is_full():
                game.player1_move()
                
            if game.current_winner == 1:
                reward = -1
                history['losses'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            elif game.is_full():
                reward = -0.1 * smartness  # Small negative reward for draws against smart opponent
                history['draws'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            
            agent.store_experience(state, action, reward, game.board, False)
            state = np.array(game.board)
        
        agent.train()
        
        # Evaluation and model saving logic
        if episode % 100 == 0:
            win_rate = history['wins'] / (episode + 1)
            print(f"Episode: {episode}, Win Rate: {win_rate:.2f}, Epsilon: {agent.epsilon:.3f}, Smartness: {smartness}")
            print(f"Wins: {history['wins']}, Losses: {history['losses']}, Draws: {history['draws']}")
            
            if win_rate > best_win_rate:
                best_win_rate = win_rate
                no_improvement_counter = 0
            else:
                no_improvement_counter += 1
            
            if episode % 1000 == 0:
                agent.model.save(f'model10_episode_{episode}.h5')
        
    agent.model.save('model10.h5')
    return agent

agent = train_agent()


Episode: 0, Win Rate: 0.00, Epsilon: 1.000, Smartness: 0
Wins: 0, Losses: 0, Draws: 1
Episode: 100, Win Rate: 0.28, Epsilon: 0.850, Smartness: 0
Wins: 28, Losses: 57, Draws: 16
Episode: 200, Win Rate: 0.28, Epsilon: 0.722, Smartness: 0
Wins: 57, Losses: 108, Draws: 36
Episode: 300, Win Rate: 0.32, Epsilon: 0.614, Smartness: 0
Wins: 95, Losses: 157, Draws: 49
Episode: 400, Win Rate: 0.34, Epsilon: 0.522, Smartness: 0
Wins: 136, Losses: 198, Draws: 67
Episode: 500, Win Rate: 0.36, Epsilon: 0.444, Smartness: 0
Wins: 180, Losses: 238, Draws: 83
Episode: 600, Win Rate: 0.38, Epsilon: 0.377, Smartness: 0
Wins: 231, Losses: 270, Draws: 100
Episode: 700, Win Rate: 0.39, Epsilon: 0.321, Smartness: 0
Wins: 274, Losses: 301, Draws: 126
Episode: 800, Win Rate: 0.39, Epsilon: 0.272, Smartness: 0
Wins: 310, Losses: 336, Draws: 155
Episode: 900, Win Rate: 0.39, Epsilon: 0.232, Smartness: 0
Wins: 350, Losses: 367, Draws: 184


Episode: 1000, Win Rate: 0.39, Epsilon: 1.000, Smartness: 0.1
Wins: 393, Losses: 397, Draws: 211
Episode: 1100, Win Rate: 0.38, Epsilon: 0.850, Smartness: 0.1
Wins: 417, Losses: 466, Draws: 218
Episode: 1200, Win Rate: 0.38, Epsilon: 0.722, Smartness: 0.1
Wins: 452, Losses: 517, Draws: 232
Episode: 1300, Win Rate: 0.38, Epsilon: 0.614, Smartness: 0.1
Wins: 491, Losses: 563, Draws: 247
Episode: 1400, Win Rate: 0.38, Epsilon: 0.522, Smartness: 0.1
Wins: 534, Losses: 604, Draws: 263
Episode: 1500, Win Rate: 0.38, Epsilon: 0.444, Smartness: 0.1
Wins: 568, Losses: 651, Draws: 282
Episode: 1600, Win Rate: 0.39, Epsilon: 0.377, Smartness: 0.1
Wins: 617, Losses: 679, Draws: 305
Episode: 1700, Win Rate: 0.39, Epsilon: 0.321, Smartness: 0.1
Wins: 656, Losses: 712, Draws: 333
Episode: 1800, Win Rate: 0.39, Epsilon: 0.272, Smartness: 0.1
Wins: 710, Losses: 736, Draws: 355
Episode: 1900, Win Rate: 0.41, Epsilon: 0.232, Smartness: 0.1
Wins: 773, Losses: 758, Draws: 370


Episode: 2000, Win Rate: 0.42, Epsilon: 1.000, Smartness: 0.2
Wins: 835, Losses: 775, Draws: 391
Episode: 2100, Win Rate: 0.41, Epsilon: 0.850, Smartness: 0.2
Wins: 857, Losses: 846, Draws: 398
Episode: 2200, Win Rate: 0.40, Epsilon: 0.722, Smartness: 0.2
Wins: 882, Losses: 907, Draws: 412
Episode: 2300, Win Rate: 0.40, Epsilon: 0.614, Smartness: 0.2
Wins: 922, Losses: 952, Draws: 427
Episode: 2400, Win Rate: 0.40, Epsilon: 0.522, Smartness: 0.2
Wins: 963, Losses: 990, Draws: 448
Episode: 2500, Win Rate: 0.40, Epsilon: 0.444, Smartness: 0.2
Wins: 997, Losses: 1032, Draws: 472
Episode: 2600, Win Rate: 0.40, Epsilon: 0.377, Smartness: 0.2
Wins: 1040, Losses: 1068, Draws: 493
Episode: 2700, Win Rate: 0.40, Epsilon: 0.321, Smartness: 0.2
Wins: 1083, Losses: 1096, Draws: 522
Episode: 2800, Win Rate: 0.40, Epsilon: 0.272, Smartness: 0.2
Wins: 1128, Losses: 1125, Draws: 548
Episode: 2900, Win Rate: 0.41, Epsilon: 0.232, Smartness: 0.2
Wins: 1177, Losses: 1146, Draws: 578


Episode: 3000, Win Rate: 0.41, Epsilon: 1.000, Smartness: 0.30000000000000004
Wins: 1228, Losses: 1171, Draws: 602
Episode: 3100, Win Rate: 0.40, Epsilon: 0.850, Smartness: 0.30000000000000004
Wins: 1254, Losses: 1234, Draws: 613
Episode: 3200, Win Rate: 0.40, Epsilon: 0.722, Smartness: 0.30000000000000004
Wins: 1269, Losses: 1300, Draws: 632
Episode: 3300, Win Rate: 0.40, Epsilon: 0.614, Smartness: 0.30000000000000004
Wins: 1304, Losses: 1345, Draws: 652
Episode: 3400, Win Rate: 0.39, Epsilon: 0.522, Smartness: 0.30000000000000004
Wins: 1334, Losses: 1396, Draws: 671
Episode: 3500, Win Rate: 0.39, Epsilon: 0.444, Smartness: 0.30000000000000004
Wins: 1363, Losses: 1451, Draws: 687
Episode: 3600, Win Rate: 0.39, Epsilon: 0.377, Smartness: 0.30000000000000004
Wins: 1402, Losses: 1489, Draws: 710
Episode: 3700, Win Rate: 0.39, Epsilon: 0.321, Smartness: 0.30000000000000004
Wins: 1440, Losses: 1532, Draws: 729
Episode: 3800, Win Rate: 0.39, Epsilon: 0.272, Smartness: 0.30000000000000004
Wi

Episode: 4000, Win Rate: 0.39, Epsilon: 1.000, Smartness: 0.4
Wins: 1575, Losses: 1607, Draws: 819
Episode: 4100, Win Rate: 0.39, Epsilon: 0.850, Smartness: 0.4
Wins: 1589, Losses: 1682, Draws: 830
Episode: 4200, Win Rate: 0.38, Epsilon: 0.722, Smartness: 0.4
Wins: 1610, Losses: 1741, Draws: 850
Episode: 4300, Win Rate: 0.38, Epsilon: 0.614, Smartness: 0.4
Wins: 1636, Losses: 1797, Draws: 868
Episode: 4400, Win Rate: 0.38, Epsilon: 0.522, Smartness: 0.4
Wins: 1663, Losses: 1847, Draws: 891
Episode: 4500, Win Rate: 0.38, Epsilon: 0.444, Smartness: 0.4
Wins: 1699, Losses: 1891, Draws: 911
Episode: 4600, Win Rate: 0.38, Epsilon: 0.377, Smartness: 0.4
Wins: 1732, Losses: 1934, Draws: 935
Episode: 4700, Win Rate: 0.38, Epsilon: 0.321, Smartness: 0.4
Wins: 1763, Losses: 1970, Draws: 968
Episode: 4800, Win Rate: 0.38, Epsilon: 0.272, Smartness: 0.4
Wins: 1803, Losses: 2008, Draws: 990
Episode: 4900, Win Rate: 0.38, Epsilon: 0.232, Smartness: 0.4
Wins: 1840, Losses: 2042, Draws: 1019


Episode: 5000, Win Rate: 0.38, Epsilon: 1.000, Smartness: 0.5
Wins: 1883, Losses: 2069, Draws: 1049
Episode: 5100, Win Rate: 0.37, Epsilon: 0.850, Smartness: 0.5
Wins: 1893, Losses: 2148, Draws: 1060
Episode: 5200, Win Rate: 0.37, Epsilon: 0.722, Smartness: 0.5
Wins: 1907, Losses: 2221, Draws: 1073
Episode: 5300, Win Rate: 0.36, Epsilon: 0.614, Smartness: 0.5
Wins: 1920, Losses: 2296, Draws: 1085
Episode: 5400, Win Rate: 0.36, Epsilon: 0.522, Smartness: 0.5
Wins: 1946, Losses: 2344, Draws: 1111
Episode: 5500, Win Rate: 0.36, Epsilon: 0.444, Smartness: 0.5
Wins: 1971, Losses: 2398, Draws: 1132
Episode: 5600, Win Rate: 0.36, Epsilon: 0.377, Smartness: 0.5
Wins: 1998, Losses: 2448, Draws: 1155
Episode: 5700, Win Rate: 0.36, Epsilon: 0.321, Smartness: 0.5
Wins: 2025, Losses: 2498, Draws: 1178
Episode: 5800, Win Rate: 0.35, Epsilon: 0.272, Smartness: 0.5
Wins: 2058, Losses: 2539, Draws: 1204
Episode: 5900, Win Rate: 0.35, Epsilon: 0.232, Smartness: 0.5
Wins: 2088, Losses: 2573, Draws: 1240


Episode: 6000, Win Rate: 0.35, Epsilon: 1.000, Smartness: 0.6
Wins: 2122, Losses: 2608, Draws: 1271
Episode: 6100, Win Rate: 0.35, Epsilon: 0.850, Smartness: 0.6
Wins: 2134, Losses: 2685, Draws: 1282
Episode: 6200, Win Rate: 0.35, Epsilon: 0.722, Smartness: 0.6
Wins: 2145, Losses: 2760, Draws: 1296
Episode: 6300, Win Rate: 0.34, Epsilon: 0.614, Smartness: 0.6
Wins: 2162, Losses: 2824, Draws: 1315
Episode: 6400, Win Rate: 0.34, Epsilon: 0.522, Smartness: 0.6
Wins: 2177, Losses: 2890, Draws: 1334
Episode: 6500, Win Rate: 0.34, Epsilon: 0.444, Smartness: 0.6
Wins: 2202, Losses: 2941, Draws: 1358
Episode: 6600, Win Rate: 0.34, Epsilon: 0.377, Smartness: 0.6
Wins: 2232, Losses: 2988, Draws: 1381
Episode: 6700, Win Rate: 0.34, Epsilon: 0.321, Smartness: 0.6
Wins: 2259, Losses: 3037, Draws: 1405
Episode: 6800, Win Rate: 0.34, Epsilon: 0.272, Smartness: 0.6
Wins: 2281, Losses: 3091, Draws: 1429
Episode: 6900, Win Rate: 0.33, Epsilon: 0.232, Smartness: 0.6
Wins: 2307, Losses: 3138, Draws: 1456


Episode: 7000, Win Rate: 0.33, Epsilon: 1.000, Smartness: 0.7
Wins: 2337, Losses: 3175, Draws: 1489
Episode: 7100, Win Rate: 0.33, Epsilon: 0.850, Smartness: 0.7
Wins: 2346, Losses: 3252, Draws: 1503
Episode: 7200, Win Rate: 0.33, Epsilon: 0.722, Smartness: 0.7
Wins: 2360, Losses: 3324, Draws: 1517
Episode: 7300, Win Rate: 0.32, Epsilon: 0.614, Smartness: 0.7
Wins: 2370, Losses: 3394, Draws: 1537
Episode: 7400, Win Rate: 0.32, Epsilon: 0.522, Smartness: 0.7
Wins: 2379, Losses: 3467, Draws: 1555
Episode: 7500, Win Rate: 0.32, Epsilon: 0.444, Smartness: 0.7
Wins: 2397, Losses: 3531, Draws: 1573
Episode: 7600, Win Rate: 0.32, Epsilon: 0.377, Smartness: 0.7
Wins: 2412, Losses: 3586, Draws: 1603
Episode: 7700, Win Rate: 0.32, Epsilon: 0.321, Smartness: 0.7
Wins: 2428, Losses: 3634, Draws: 1639
Episode: 7800, Win Rate: 0.31, Epsilon: 0.272, Smartness: 0.7
Wins: 2447, Losses: 3678, Draws: 1676
Episode: 7900, Win Rate: 0.31, Epsilon: 0.232, Smartness: 0.7
Wins: 2473, Losses: 3721, Draws: 1707


Episode: 8000, Win Rate: 0.31, Epsilon: 1.000, Smartness: 0.7999999999999999
Wins: 2493, Losses: 3767, Draws: 1741
Episode: 8100, Win Rate: 0.31, Epsilon: 0.850, Smartness: 0.7999999999999999
Wins: 2499, Losses: 3855, Draws: 1747
Episode: 8200, Win Rate: 0.31, Epsilon: 0.722, Smartness: 0.7999999999999999
Wins: 2507, Losses: 3934, Draws: 1760
Episode: 8300, Win Rate: 0.30, Epsilon: 0.614, Smartness: 0.7999999999999999
Wins: 2516, Losses: 4002, Draws: 1783
Episode: 8400, Win Rate: 0.30, Epsilon: 0.522, Smartness: 0.7999999999999999
Wins: 2527, Losses: 4070, Draws: 1804
Episode: 8500, Win Rate: 0.30, Epsilon: 0.444, Smartness: 0.7999999999999999
Wins: 2538, Losses: 4136, Draws: 1827
Episode: 8600, Win Rate: 0.30, Epsilon: 0.377, Smartness: 0.7999999999999999
Wins: 2545, Losses: 4201, Draws: 1855
Episode: 8700, Win Rate: 0.29, Epsilon: 0.321, Smartness: 0.7999999999999999
Wins: 2560, Losses: 4252, Draws: 1889
Episode: 8800, Win Rate: 0.29, Epsilon: 0.272, Smartness: 0.7999999999999999
Win

Episode: 9000, Win Rate: 0.29, Epsilon: 1.000, Smartness: 0.8
Wins: 2609, Losses: 4394, Draws: 1998
Episode: 9100, Win Rate: 0.29, Epsilon: 0.850, Smartness: 0.8
Wins: 2612, Losses: 4481, Draws: 2008
Episode: 9200, Win Rate: 0.29, Epsilon: 0.722, Smartness: 0.8
Wins: 2625, Losses: 4554, Draws: 2022
Episode: 9300, Win Rate: 0.28, Epsilon: 0.614, Smartness: 0.8
Wins: 2633, Losses: 4632, Draws: 2036
Episode: 9400, Win Rate: 0.28, Epsilon: 0.522, Smartness: 0.8
Wins: 2645, Losses: 4698, Draws: 2058
Episode: 9500, Win Rate: 0.28, Epsilon: 0.444, Smartness: 0.8
Wins: 2657, Losses: 4759, Draws: 2085
Episode: 9600, Win Rate: 0.28, Epsilon: 0.377, Smartness: 0.8
Wins: 2666, Losses: 4822, Draws: 2113
Episode: 9700, Win Rate: 0.28, Epsilon: 0.321, Smartness: 0.8
Wins: 2674, Losses: 4873, Draws: 2154
Episode: 9800, Win Rate: 0.27, Epsilon: 0.272, Smartness: 0.8
Wins: 2688, Losses: 4923, Draws: 2190
Episode: 9900, Win Rate: 0.27, Epsilon: 0.232, Smartness: 0.8
Wins: 2706, Losses: 4971, Draws: 2224


In [8]:
import numpy as np
import random
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from TicTacToe import TicTacToe

class SQNAgent:
    def __init__(self, state_size=9, action_size=9, gamma=0.95):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = gamma  
        self.learning_rate = 0.001
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.9995
        self.batch_size = 32
        self.replay_buffer = deque(maxlen=10000)
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential([
            Dense(64, input_dim=self.state_size, activation='relu'),
            Dense(64, activation='relu'),
            Dense(self.action_size, activation='linear')
        ])
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def process_state(self, state):
        """
        Converts the state from [0,1,2] format to [-1,0,1] format for better learning.
        """
        processed_state = np.array(state).copy()
        processed_state[processed_state == 0] = -1
        processed_state[processed_state == 1] = 0
        processed_state[processed_state == 2] = 1
        return processed_state

    def select_action(self, state, valid_actions):
        if not valid_actions:
            return None

        if random.random() < self.epsilon:
            return random.choice(valid_actions)
        
        processed_state = self.process_state(state)
        q_values = self.model.predict(processed_state.reshape(1, -1), verbose=0)[0]
        valid_q_values = [(q_values[action], action) for action in valid_actions]
        return max(valid_q_values, key=lambda x: x[0])[1]

    def store_experience(self, state, action, reward, next_state, done):
        self.replay_buffer.append((state, action, reward, next_state, done))

    def train(self):
        if len(self.replay_buffer) < self.batch_size:
            return

        mini_batch = random.sample(self.replay_buffer, self.batch_size)
        states = np.array([self.process_state(exp[0]) for exp in mini_batch])
        next_states = np.array([self.process_state(exp[3]) for exp in mini_batch])
        current_q_values = self.model.predict(states, verbose=0)
        next_q_values = self.model.predict(next_states, verbose=0)

        x = []
        y = []

        for i, (state, action, reward, next_state, done) in enumerate(mini_batch):
            if done:
                target = reward
            else:
                next_valid_actions = [j for j, val in enumerate(next_state) if val == 0]
                if next_valid_actions:
                    max_next_q = max(next_q_values[i][action] for action in next_valid_actions)
                    target = reward + self.gamma * max_next_q
                else:
                    target = reward
            
            current_q = current_q_values[i].copy()
            current_q[action] = target
            x.append(self.process_state(state))
            y.append(current_q)

        self.model.fit(np.array(x), np.array(y), batch_size=32, epochs=1, verbose=0)

def train_agent(episodes=10000):
    agent = SQNAgent()
    history = {'wins': 0, 'losses': 0, 'draws': 0}
    initial_epsilon = 1.0
    min_epsilon = 0.01
    decay_episodes = 1000
    smartness=0
    for episode in range(episodes):
        if episode % 1000 == 0 and episode > 0:
            agent.epsilon = 1.0
            print("Resetting epsilon to 1.0")
        else:
            epsilon_decay = (initial_epsilon - min_epsilon) / decay_episodes
            agent.epsilon = max(min_epsilon, initial_epsilon - (episode % 1000) * epsilon_decay)
        if episode%1000==0 and episode>0:
            smartness = min(0.8, episode / (episodes * 0.9))
            
        game = TicTacToe(smartMovePlayer1=smartness)
        state = np.array(game.board)
        
        game.player1_move()
        state = np.array(game.board)
        
        while True:
            valid_actions = game.empty_positions()
            if not valid_actions:
                history['draws'] += 1
                break
                
            action = agent.select_action(state, valid_actions)
            game.make_move(action, player=2)
            
            reward = 0
            if game.current_winner == 2:
                reward = 1.0+smartness
                history['wins'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            
            game.player1_move()
            if game.current_winner == 1:
                reward = -1.0-smartness
                history['losses'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            elif game.is_full():
                reward = min(-0.1,-0.5*smartness)
                history['draws'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            
            agent.store_experience(state, action, reward, game.board, False)
            state = np.array(game.board)
        
        agent.train()
        
        if episode % 100 == 0:
            win_rate = history['wins'] / (episode + 1)
            print(f"Episode: {episode}, Win Rate: {win_rate:.2f}, Epsilon: {agent.epsilon:.3f}, smartmove{smartness}")
            print(f"Wins: {history['wins']}, Losses: {history['losses']}, Draws: {history['draws']}")
        
        if episode % 1000 == 0:
            agent.model.save(f'model11_episode_{episode}.h5')
    agent.model.save('model11.h5')
agent = train_agent()


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Episode: 0, Win Rate: 0.00, Epsilon: 1.000, smartmove0
Wins: 0, Losses: 1, Draws: 0
Episode: 100, Win Rate: 0.27, Epsilon: 0.901, smartmove0
Wins: 27, Losses: 56, Draws: 18
Episode: 200, Win Rate: 0.27, Epsilon: 0.802, smartmove0
Wins: 55, Losses: 113, Draws: 33
Episode: 300, Win Rate: 0.27, Epsilon: 0.703, smartmove0
Wins: 81, Losses: 169, Draws: 51
Episode: 400, Win Rate: 0.29, Epsilon: 0.604, smartmove0
Wins: 116, Losses: 222, Draws: 63
Episode: 500, Win Rate: 0.30, Epsilon: 0.505, smartmove0
Wins: 149, Losses: 281, Draws: 71
Episode: 600, Win Rate: 0.30, Epsilon: 0.406, smartmove0
Wins: 183, Losses: 341, Draws: 77
Episode: 700, Win Rate: 0.31, Epsilon: 0.307, smartmove0
Wins: 216, Losses: 398, Draws: 87
Episode: 800, Win Rate: 0.32, Epsilon: 0.208, smartmove0
Wins: 255, Losses: 451, Draws: 95
Episode: 900, Win Rate: 0.32, Epsilon: 0.109, smartmove0
Wins: 291, Losses: 505, Draws: 105


Resetting epsilon to 1.0
Episode: 1000, Win Rate: 0.32, Epsilon: 1.000, smartmove0.1111111111111111
Wins: 323, Losses: 565, Draws: 113
Episode: 1100, Win Rate: 0.32, Epsilon: 0.901, smartmove0.1111111111111111
Wins: 348, Losses: 627, Draws: 126
Episode: 1200, Win Rate: 0.31, Epsilon: 0.802, smartmove0.1111111111111111
Wins: 377, Losses: 691, Draws: 133
Episode: 1300, Win Rate: 0.31, Epsilon: 0.703, smartmove0.1111111111111111
Wins: 407, Losses: 749, Draws: 145
Episode: 1400, Win Rate: 0.31, Epsilon: 0.604, smartmove0.1111111111111111
Wins: 431, Losses: 816, Draws: 154
Episode: 1500, Win Rate: 0.31, Epsilon: 0.505, smartmove0.1111111111111111
Wins: 463, Losses: 873, Draws: 165
Episode: 1600, Win Rate: 0.32, Epsilon: 0.406, smartmove0.1111111111111111
Wins: 505, Losses: 920, Draws: 176
Episode: 1700, Win Rate: 0.32, Epsilon: 0.307, smartmove0.1111111111111111
Wins: 543, Losses: 973, Draws: 185
Episode: 1800, Win Rate: 0.32, Epsilon: 0.208, smartmove0.1111111111111111
Wins: 579, Losses: 1

Resetting epsilon to 1.0
Episode: 2000, Win Rate: 0.33, Epsilon: 1.000, smartmove0.2222222222222222
Wins: 659, Losses: 1132, Draws: 210
Episode: 2100, Win Rate: 0.32, Epsilon: 0.901, smartmove0.2222222222222222
Wins: 678, Losses: 1202, Draws: 221
Episode: 2200, Win Rate: 0.32, Epsilon: 0.802, smartmove0.2222222222222222
Wins: 698, Losses: 1269, Draws: 234
Episode: 2300, Win Rate: 0.32, Epsilon: 0.703, smartmove0.2222222222222222
Wins: 730, Losses: 1328, Draws: 243
Episode: 2400, Win Rate: 0.31, Epsilon: 0.604, smartmove0.2222222222222222
Wins: 752, Losses: 1394, Draws: 255
Episode: 2500, Win Rate: 0.31, Epsilon: 0.505, smartmove0.2222222222222222
Wins: 784, Losses: 1455, Draws: 262
Episode: 2600, Win Rate: 0.31, Epsilon: 0.406, smartmove0.2222222222222222
Wins: 818, Losses: 1512, Draws: 271
Episode: 2700, Win Rate: 0.32, Epsilon: 0.307, smartmove0.2222222222222222
Wins: 858, Losses: 1568, Draws: 275
Episode: 2800, Win Rate: 0.32, Epsilon: 0.208, smartmove0.2222222222222222
Wins: 884, L

Resetting epsilon to 1.0
Episode: 3000, Win Rate: 0.32, Epsilon: 1.000, smartmove0.3333333333333333
Wins: 947, Losses: 1751, Draws: 303
Episode: 3100, Win Rate: 0.31, Epsilon: 0.901, smartmove0.3333333333333333
Wins: 972, Losses: 1813, Draws: 316
Episode: 3200, Win Rate: 0.31, Epsilon: 0.802, smartmove0.3333333333333333
Wins: 987, Losses: 1889, Draws: 325
Episode: 3300, Win Rate: 0.30, Epsilon: 0.703, smartmove0.3333333333333333
Wins: 1005, Losses: 1957, Draws: 339
Episode: 3400, Win Rate: 0.30, Epsilon: 0.604, smartmove0.3333333333333333
Wins: 1032, Losses: 2018, Draws: 351
Episode: 3500, Win Rate: 0.30, Epsilon: 0.505, smartmove0.3333333333333333
Wins: 1052, Losses: 2086, Draws: 363
Episode: 3600, Win Rate: 0.30, Epsilon: 0.406, smartmove0.3333333333333333
Wins: 1083, Losses: 2146, Draws: 372
Episode: 3700, Win Rate: 0.30, Epsilon: 0.307, smartmove0.3333333333333333
Wins: 1116, Losses: 2203, Draws: 382
Episode: 3800, Win Rate: 0.30, Epsilon: 0.208, smartmove0.3333333333333333
Wins: 1

...

Resetting epsilon to 1.0
Episode: 5000, Win Rate: 0.28, Epsilon: 1.000, smartmove0.5555555555555556
Wins: 1410, Losses: 3066, Draws: 525
Episode: 5100, Win Rate: 0.28, Epsilon: 0.901, smartmove0.5555555555555556
Wins: 1421, Losses: 3144, Draws: 536
Episode: 5200, Win Rate: 0.28, Epsilon: 0.802, smartmove0.5555555555555556
Wins: 1435, Losses: 3224, Draws: 542
Episode: 5300, Win Rate: 0.27, Epsilon: 0.703, smartmove0.5555555555555556
Wins: 1445, Losses: 3296, Draws: 560
Episode: 5400, Win Rate: 0.27, Epsilon: 0.604, smartmove0.5555555555555556
Wins: 1460, Losses: 3369, Draws: 572
Episode: 5500, Win Rate: 0.27, Epsilon: 0.505, smartmove0.5555555555555556
Wins: 1481, Losses: 3433, Draws: 587
Episode: 5600, Win Rate: 0.27, Epsilon: 0.406, smartmove0.5555555555555556
Wins: 1499, Losses: 3506, Draws: 596
Episode: 5700, Win Rate: 0.27, Epsilon: 0.307, smartmove0.5555555555555556
Wins: 1520, Losses: 3574, Draws: 607
Episode: 5800, Win Rate: 0.27, Epsilon: 0.208, smartmove0.5555555555555556
Wins

Resetting epsilon to 1.0
Episode: 6000, Win Rate: 0.27, Epsilon: 1.000, smartmove0.6666666666666666
Wins: 1592, Losses: 3763, Draws: 646
Episode: 6100, Win Rate: 0.26, Epsilon: 0.901, smartmove0.6666666666666666
Wins: 1599, Losses: 3845, Draws: 657
Episode: 6200, Win Rate: 0.26, Epsilon: 0.802, smartmove0.6666666666666666
Wins: 1609, Losses: 3926, Draws: 666
Episode: 6300, Win Rate: 0.26, Epsilon: 0.703, smartmove0.6666666666666666
Wins: 1617, Losses: 4007, Draws: 677
Episode: 6400, Win Rate: 0.25, Epsilon: 0.604, smartmove0.6666666666666666
Wins: 1628, Losses: 4080, Draws: 693
Episode: 6500, Win Rate: 0.25, Epsilon: 0.505, smartmove0.6666666666666666
Wins: 1650, Losses: 4148, Draws: 703
Episode: 6600, Win Rate: 0.25, Epsilon: 0.406, smartmove0.6666666666666666
Wins: 1664, Losses: 4222, Draws: 715
Episode: 6700, Win Rate: 0.25, Epsilon: 0.307, smartmove0.6666666666666666
Wins: 1683, Losses: 4295, Draws: 723
Episode: 6800, Win Rate: 0.25, Epsilon: 0.208, smartmove0.6666666666666666
Wins

Resetting epsilon to 1.0
Episode: 7000, Win Rate: 0.25, Epsilon: 1.000, smartmove0.7777777777777778
Wins: 1745, Losses: 4472, Draws: 784
Episode: 7100, Win Rate: 0.25, Epsilon: 0.901, smartmove0.7777777777777778
Wins: 1748, Losses: 4561, Draws: 792
Episode: 7200, Win Rate: 0.24, Epsilon: 0.802, smartmove0.7777777777777778
Wins: 1762, Losses: 4634, Draws: 805
Episode: 7300, Win Rate: 0.24, Epsilon: 0.703, smartmove0.7777777777777778
Wins: 1768, Losses: 4713, Draws: 820
Episode: 7400, Win Rate: 0.24, Epsilon: 0.604, smartmove0.7777777777777778
Wins: 1771, Losses: 4797, Draws: 833
Episode: 7500, Win Rate: 0.24, Epsilon: 0.505, smartmove0.7777777777777778
Wins: 1784, Losses: 4869, Draws: 848
Episode: 7600, Win Rate: 0.24, Epsilon: 0.406, smartmove0.7777777777777778
Wins: 1794, Losses: 4943, Draws: 864
Episode: 7700, Win Rate: 0.23, Epsilon: 0.307, smartmove0.7777777777777778
Wins: 1808, Losses: 5015, Draws: 878
Episode: 7800, Win Rate: 0.23, Epsilon: 0.208, smartmove0.7777777777777778
Wins

Resetting epsilon to 1.0
Episode: 8000, Win Rate: 0.23, Epsilon: 1.000, smartmove0.8
Wins: 1852, Losses: 5206, Draws: 943
Episode: 8100, Win Rate: 0.23, Epsilon: 0.901, smartmove0.8
Wins: 1859, Losses: 5291, Draws: 951
Episode: 8200, Win Rate: 0.23, Epsilon: 0.802, smartmove0.8
Wins: 1870, Losses: 5371, Draws: 960
Episode: 8300, Win Rate: 0.23, Epsilon: 0.703, smartmove0.8
Wins: 1884, Losses: 5447, Draws: 970
Episode: 8400, Win Rate: 0.22, Epsilon: 0.604, smartmove0.8
Wins: 1890, Losses: 5525, Draws: 986
Episode: 8500, Win Rate: 0.22, Epsilon: 0.505, smartmove0.8
Wins: 1901, Losses: 5596, Draws: 1004
Episode: 8600, Win Rate: 0.22, Epsilon: 0.406, smartmove0.8
Wins: 1913, Losses: 5670, Draws: 1018
Episode: 8700, Win Rate: 0.22, Epsilon: 0.307, smartmove0.8
Wins: 1925, Losses: 5739, Draws: 1037
Episode: 8800, Win Rate: 0.22, Epsilon: 0.208, smartmove0.8
Wins: 1931, Losses: 5815, Draws: 1055
Episode: 8900, Win Rate: 0.22, Epsilon: 0.109, smartmove0.8
Wins: 1949, Losses: 5876, Draws: 1076


Resetting epsilon to 1.0
Episode: 9000, Win Rate: 0.22, Epsilon: 1.000, smartmove0.8
Wins: 1964, Losses: 5937, Draws: 1100
Episode: 9100, Win Rate: 0.22, Epsilon: 0.901, smartmove0.8
Wins: 1968, Losses: 6025, Draws: 1108
Episode: 9200, Win Rate: 0.21, Epsilon: 0.802, smartmove0.8
Wins: 1975, Losses: 6105, Draws: 1121
Episode: 9300, Win Rate: 0.21, Epsilon: 0.703, smartmove0.8
Wins: 1981, Losses: 6183, Draws: 1137
Episode: 9400, Win Rate: 0.21, Epsilon: 0.604, smartmove0.8
Wins: 1991, Losses: 6254, Draws: 1156
Episode: 9500, Win Rate: 0.21, Epsilon: 0.505, smartmove0.8
Wins: 1996, Losses: 6332, Draws: 1173
Episode: 9600, Win Rate: 0.21, Epsilon: 0.406, smartmove0.8
Wins: 2008, Losses: 6403, Draws: 1190
Episode: 9700, Win Rate: 0.21, Epsilon: 0.307, smartmove0.8
Wins: 2021, Losses: 6471, Draws: 1209
Episode: 9800, Win Rate: 0.21, Epsilon: 0.208, smartmove0.8
Wins: 2037, Losses: 6528, Draws: 1236
Episode: 9900, Win Rate: 0.21, Epsilon: 0.109, smartmove0.8
Wins: 2056, Losses: 6589, Draws: 

In [12]:
import numpy as np
import random
from collections import deque
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from TicTacToe import TicTacToe
import os

class SQNAgent:
    def __init__(self, state_size=9, action_size=9, gamma=0.95, model_path=None):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = gamma  
        self.learning_rate = 0.001
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.9995
        self.batch_size = 32
        self.replay_buffer = deque(maxlen=10000)
        
        if model_path and os.path.isfile(model_path):
            print(f"Loading model from {model_path}")
            self.model = load_model(model_path)
        else:
            print("No existing model found; initializing a new model.")
            self.model = self._build_model()

    def _build_model(self):
        model = Sequential([
            Dense(64, input_dim=self.state_size, activation='relu'),
            Dense(64, activation='relu'),
            Dense(self.action_size, activation='linear')
        ])
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def process_state(self, state):
        """
        Converts the state from [0,1,2] format to [-1,0,1] format for better learning.
        """
        processed_state = np.array(state).copy()
        processed_state[processed_state == 0] = -1
        processed_state[processed_state == 1] = 0
        processed_state[processed_state == 2] = 1
        return processed_state

    def select_action(self, state, valid_actions):
        if not valid_actions:
            return None

        if random.random() < self.epsilon:
            return random.choice(valid_actions)
        
        processed_state = self.process_state(state)
        q_values = self.model.predict(processed_state.reshape(1, -1), verbose=0)[0]
        valid_q_values = [(q_values[action], action) for action in valid_actions]
        return max(valid_q_values, key=lambda x: x[0])[1]

    def store_experience(self, state, action, reward, next_state, done):
        self.replay_buffer.append((state, action, reward, next_state, done))

    def train(self):
        if len(self.replay_buffer) < self.batch_size:
            return

        mini_batch = random.sample(self.replay_buffer, self.batch_size)
        states = np.array([self.process_state(exp[0]) for exp in mini_batch])
        next_states = np.array([self.process_state(exp[3]) for exp in mini_batch])
        current_q_values = self.model.predict(states, verbose=0)
        next_q_values = self.model.predict(next_states, verbose=0)

        x = []
        y = []

        for i, (state, action, reward, next_state, done) in enumerate(mini_batch):
            if done:
                target = reward
            else:
                next_valid_actions = [j for j, val in enumerate(next_state) if val == 0]
                if next_valid_actions:
                    max_next_q = max(next_q_values[i][action] for action in next_valid_actions)
                    target = reward + self.gamma * max_next_q
                else:
                    target = reward
            
            current_q = current_q_values[i].copy()
            current_q[action] = target
            x.append(self.process_state(state))
            y.append(current_q)

        self.model.fit(np.array(x), np.array(y), batch_size=32, epochs=1, verbose=0)

def train_agent(episodes=20000):
    agent = SQNAgent()
    history = {'wins': 0, 'losses': 0, 'draws': 0}
    initial_epsilon = 1.0
    min_epsilon = 0.01
    decay_episodes = 1000
    smartness=0
    for episode in range(episodes):
        if episode % 2000 == 0 and episode > 0:
            agent.epsilon = 1.0
            print("Resetting epsilon to 1.0")
        else:
            epsilon_decay = (initial_epsilon - min_epsilon) / decay_episodes
            agent.epsilon = max(min_epsilon, initial_epsilon - (episode % 2000) * epsilon_decay)
        if episode%2000==0 and episode>0:
            smartness = min(0.8, episode / (episodes * 0.9))
            
        game = TicTacToe(smartMovePlayer1=smartness)
        state = np.array(game.board)
        
        game.player1_move()
        state = np.array(game.board)
        
        while True:
            valid_actions = game.empty_positions()
            if not valid_actions:
                history['draws'] += 1
                break
                
            action = agent.select_action(state, valid_actions)
            game.make_move(action, player=2)
            
            reward = 0
            if game.current_winner == 2:
                reward = 1.0+smartness
                history['wins'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            
            game.player1_move()
            if game.current_winner == 1:
                reward = -1.0-smartness
                history['losses'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            elif game.is_full():
                reward = min(-0.1,-0.5*smartness)
                history['draws'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            
            agent.store_experience(state, action, reward, game.board, False)
            state = np.array(game.board)
        
        agent.train()
        
        if episode % 100 == 0:
            win_rate = history['wins'] / (episode + 1)
            print(f"Episode: {episode}, Win Rate: {win_rate:.2f}, Epsilon: {agent.epsilon:.3f}, smartmove{smartness}")
            print(f"Wins: {history['wins']}, Losses: {history['losses']}, Draws: {history['draws']}")
        
        if episode % 1000 == 0:
            agent.model.save(f'model11_retrain_episode_{episode}.h5')
    agent.model.save('model11_retrain.h5')
agent = train_agent()


No existing model found; initializing a new model.
Episode: 0, Win Rate: 0.00, Epsilon: 1.000, smartmove0
Wins: 0, Losses: 1, Draws: 0
Episode: 100, Win Rate: 0.30, Epsilon: 0.901, smartmove0
Wins: 30, Losses: 61, Draws: 10
Episode: 200, Win Rate: 0.27, Epsilon: 0.802, smartmove0
Wins: 54, Losses: 121, Draws: 26
Episode: 300, Win Rate: 0.28, Epsilon: 0.703, smartmove0
Wins: 85, Losses: 180, Draws: 36
Episode: 400, Win Rate: 0.30, Epsilon: 0.604, smartmove0
Wins: 122, Losses: 235, Draws: 44
Episode: 500, Win Rate: 0.32, Epsilon: 0.505, smartmove0
Wins: 159, Losses: 284, Draws: 58
Episode: 600, Win Rate: 0.31, Epsilon: 0.406, smartmove0
Wins: 185, Losses: 347, Draws: 69
Episode: 700, Win Rate: 0.32, Epsilon: 0.307, smartmove0
Wins: 223, Losses: 402, Draws: 76
Episode: 800, Win Rate: 0.32, Epsilon: 0.208, smartmove0
Wins: 258, Losses: 459, Draws: 84
Episode: 900, Win Rate: 0.33, Epsilon: 0.109, smartmove0
Wins: 299, Losses: 508, Draws: 94


Episode: 1000, Win Rate: 0.34, Epsilon: 0.010, smartmove0
Wins: 339, Losses: 561, Draws: 101
Episode: 1100, Win Rate: 0.34, Epsilon: 0.010, smartmove0
Wins: 377, Losses: 614, Draws: 110
Episode: 1200, Win Rate: 0.35, Epsilon: 0.010, smartmove0
Wins: 419, Losses: 666, Draws: 116
Episode: 1300, Win Rate: 0.36, Epsilon: 0.010, smartmove0
Wins: 463, Losses: 713, Draws: 125
Episode: 1400, Win Rate: 0.36, Epsilon: 0.010, smartmove0
Wins: 503, Losses: 765, Draws: 133
Episode: 1500, Win Rate: 0.37, Epsilon: 0.010, smartmove0
Wins: 549, Losses: 811, Draws: 141
Episode: 1600, Win Rate: 0.37, Epsilon: 0.010, smartmove0
Wins: 592, Losses: 858, Draws: 151
Episode: 1700, Win Rate: 0.37, Epsilon: 0.010, smartmove0
Wins: 621, Losses: 918, Draws: 162
Episode: 1800, Win Rate: 0.36, Epsilon: 0.010, smartmove0
Wins: 652, Losses: 981, Draws: 168
Episode: 1900, Win Rate: 0.36, Epsilon: 0.010, smartmove0
Wins: 690, Losses: 1031, Draws: 180


Resetting epsilon to 1.0
Episode: 2000, Win Rate: 0.36, Epsilon: 1.000, smartmove0.1111111111111111
Wins: 727, Losses: 1081, Draws: 193
Episode: 2100, Win Rate: 0.35, Epsilon: 0.901, smartmove0.1111111111111111
Wins: 744, Losses: 1146, Draws: 211
Episode: 2200, Win Rate: 0.35, Epsilon: 0.802, smartmove0.1111111111111111
Wins: 773, Losses: 1206, Draws: 222
Episode: 2300, Win Rate: 0.35, Epsilon: 0.703, smartmove0.1111111111111111
Wins: 801, Losses: 1265, Draws: 235
Episode: 2400, Win Rate: 0.35, Epsilon: 0.604, smartmove0.1111111111111111
Wins: 831, Losses: 1326, Draws: 244
Episode: 2500, Win Rate: 0.34, Epsilon: 0.505, smartmove0.1111111111111111
Wins: 862, Losses: 1386, Draws: 253
Episode: 2600, Win Rate: 0.34, Epsilon: 0.406, smartmove0.1111111111111111
Wins: 888, Losses: 1451, Draws: 262
Episode: 2700, Win Rate: 0.34, Epsilon: 0.307, smartmove0.1111111111111111
Wins: 922, Losses: 1500, Draws: 279
Episode: 2800, Win Rate: 0.34, Epsilon: 0.208, smartmove0.1111111111111111
Wins: 954, L

Episode: 3000, Win Rate: 0.34, Epsilon: 0.010, smartmove0.1111111111111111
Wins: 1028, Losses: 1647, Draws: 326
Episode: 3100, Win Rate: 0.35, Epsilon: 0.010, smartmove0.1111111111111111
Wins: 1074, Losses: 1691, Draws: 336
Episode: 3200, Win Rate: 0.35, Epsilon: 0.010, smartmove0.1111111111111111
Wins: 1110, Losses: 1743, Draws: 348
Episode: 3300, Win Rate: 0.35, Epsilon: 0.010, smartmove0.1111111111111111
Wins: 1153, Losses: 1790, Draws: 358
Episode: 3400, Win Rate: 0.35, Epsilon: 0.010, smartmove0.1111111111111111
Wins: 1196, Losses: 1831, Draws: 374
Episode: 3500, Win Rate: 0.35, Epsilon: 0.010, smartmove0.1111111111111111
Wins: 1237, Losses: 1873, Draws: 391
Episode: 3600, Win Rate: 0.36, Epsilon: 0.010, smartmove0.1111111111111111
Wins: 1281, Losses: 1917, Draws: 403
Episode: 3700, Win Rate: 0.35, Epsilon: 0.010, smartmove0.1111111111111111
Wins: 1312, Losses: 1967, Draws: 422
Episode: 3800, Win Rate: 0.36, Epsilon: 0.010, smartmove0.1111111111111111
Wins: 1359, Losses: 2012, Dra

Resetting epsilon to 1.0
Episode: 4000, Win Rate: 0.36, Epsilon: 1.000, smartmove0.2222222222222222
Wins: 1431, Losses: 2111, Draws: 459
Episode: 4100, Win Rate: 0.35, Epsilon: 0.901, smartmove0.2222222222222222
Wins: 1450, Losses: 2181, Draws: 470
Episode: 4200, Win Rate: 0.35, Epsilon: 0.802, smartmove0.2222222222222222
Wins: 1468, Losses: 2247, Draws: 486
Episode: 4300, Win Rate: 0.35, Epsilon: 0.703, smartmove0.2222222222222222
Wins: 1496, Losses: 2308, Draws: 497
Episode: 4400, Win Rate: 0.34, Epsilon: 0.604, smartmove0.2222222222222222
Wins: 1518, Losses: 2379, Draws: 504
Episode: 4500, Win Rate: 0.34, Epsilon: 0.505, smartmove0.2222222222222222
Wins: 1547, Losses: 2439, Draws: 515
Episode: 4600, Win Rate: 0.34, Epsilon: 0.406, smartmove0.2222222222222222
Wins: 1570, Losses: 2498, Draws: 533
Episode: 4700, Win Rate: 0.34, Epsilon: 0.307, smartmove0.2222222222222222
Wins: 1594, Losses: 2565, Draws: 542
Episode: 4800, Win Rate: 0.34, Epsilon: 0.208, smartmove0.2222222222222222
Wins

...

Episode: 15000, Win Rate: 0.27, Epsilon: 0.010, smartmove0.7777777777777778
Wins: 4108, Losses: 8362, Draws: 2531
Episode: 15100, Win Rate: 0.27, Epsilon: 0.010, smartmove0.7777777777777778
Wins: 4117, Losses: 8421, Draws: 2563
Episode: 15200, Win Rate: 0.27, Epsilon: 0.010, smartmove0.7777777777777778
Wins: 4127, Losses: 8475, Draws: 2599
Episode: 15300, Win Rate: 0.27, Epsilon: 0.010, smartmove0.7777777777777778
Wins: 4141, Losses: 8527, Draws: 2633
Episode: 15400, Win Rate: 0.27, Epsilon: 0.010, smartmove0.7777777777777778
Wins: 4158, Losses: 8587, Draws: 2656
Episode: 15500, Win Rate: 0.27, Epsilon: 0.010, smartmove0.7777777777777778
Wins: 4173, Losses: 8648, Draws: 2680
Episode: 15600, Win Rate: 0.27, Epsilon: 0.010, smartmove0.7777777777777778
Wins: 4196, Losses: 8706, Draws: 2699
Episode: 15700, Win Rate: 0.27, Epsilon: 0.010, smartmove0.7777777777777778
Wins: 4216, Losses: 8762, Draws: 2723
Episode: 15800, Win Rate: 0.27, Epsilon: 0.010, smartmove0.7777777777777778
Wins: 4242, 

Resetting epsilon to 1.0
Episode: 16000, Win Rate: 0.27, Epsilon: 1.000, smartmove0.8
Wins: 4287, Losses: 8921, Draws: 2793
Episode: 16100, Win Rate: 0.27, Epsilon: 0.901, smartmove0.8
Wins: 4291, Losses: 9008, Draws: 2802
Episode: 16200, Win Rate: 0.27, Epsilon: 0.802, smartmove0.8
Wins: 4295, Losses: 9094, Draws: 2812
Episode: 16300, Win Rate: 0.26, Epsilon: 0.703, smartmove0.8
Wins: 4305, Losses: 9173, Draws: 2823
Episode: 16400, Win Rate: 0.26, Epsilon: 0.604, smartmove0.8
Wins: 4320, Losses: 9248, Draws: 2833
Episode: 16500, Win Rate: 0.26, Epsilon: 0.505, smartmove0.8
Wins: 4327, Losses: 9321, Draws: 2853
Episode: 16600, Win Rate: 0.26, Epsilon: 0.406, smartmove0.8
Wins: 4334, Losses: 9392, Draws: 2875
Episode: 16700, Win Rate: 0.26, Epsilon: 0.307, smartmove0.8
Wins: 4343, Losses: 9453, Draws: 2905
Episode: 16800, Win Rate: 0.26, Epsilon: 0.208, smartmove0.8
Wins: 4355, Losses: 9516, Draws: 2930
Episode: 16900, Win Rate: 0.26, Epsilon: 0.109, smartmove0.8
Wins: 4368, Losses: 957

Episode: 17000, Win Rate: 0.26, Epsilon: 0.010, smartmove0.8
Wins: 4384, Losses: 9621, Draws: 2996
Episode: 17100, Win Rate: 0.26, Epsilon: 0.010, smartmove0.8
Wins: 4406, Losses: 9663, Draws: 3032
Episode: 17200, Win Rate: 0.26, Epsilon: 0.010, smartmove0.8
Wins: 4426, Losses: 9708, Draws: 3067
Episode: 17300, Win Rate: 0.26, Epsilon: 0.010, smartmove0.8
Wins: 4442, Losses: 9749, Draws: 3110
Episode: 17400, Win Rate: 0.26, Epsilon: 0.010, smartmove0.8
Wins: 4460, Losses: 9803, Draws: 3138
Episode: 17500, Win Rate: 0.26, Epsilon: 0.010, smartmove0.8
Wins: 4481, Losses: 9844, Draws: 3176
Episode: 17600, Win Rate: 0.26, Epsilon: 0.010, smartmove0.8
Wins: 4505, Losses: 9883, Draws: 3213
Episode: 17700, Win Rate: 0.26, Epsilon: 0.010, smartmove0.8
Wins: 4539, Losses: 9916, Draws: 3246
Episode: 17800, Win Rate: 0.26, Epsilon: 0.010, smartmove0.8
Wins: 4570, Losses: 9954, Draws: 3277
Episode: 17900, Win Rate: 0.26, Epsilon: 0.010, smartmove0.8
Wins: 4593, Losses: 9993, Draws: 3315


Resetting epsilon to 1.0
Episode: 18000, Win Rate: 0.26, Epsilon: 1.000, smartmove0.8
Wins: 4610, Losses: 10032, Draws: 3359
Episode: 18100, Win Rate: 0.26, Epsilon: 0.901, smartmove0.8
Wins: 4618, Losses: 10114, Draws: 3369
Episode: 18200, Win Rate: 0.25, Epsilon: 0.802, smartmove0.8
Wins: 4624, Losses: 10190, Draws: 3387
Episode: 18300, Win Rate: 0.25, Epsilon: 0.703, smartmove0.8
Wins: 4633, Losses: 10265, Draws: 3403
Episode: 18400, Win Rate: 0.25, Epsilon: 0.604, smartmove0.8
Wins: 4641, Losses: 10344, Draws: 3416
Episode: 18500, Win Rate: 0.25, Epsilon: 0.505, smartmove0.8
Wins: 4654, Losses: 10415, Draws: 3432
Episode: 18600, Win Rate: 0.25, Epsilon: 0.406, smartmove0.8
Wins: 4664, Losses: 10486, Draws: 3451
Episode: 18700, Win Rate: 0.25, Epsilon: 0.307, smartmove0.8
Wins: 4676, Losses: 10549, Draws: 3476
Episode: 18800, Win Rate: 0.25, Epsilon: 0.208, smartmove0.8
Wins: 4688, Losses: 10611, Draws: 3502
Episode: 18900, Win Rate: 0.25, Epsilon: 0.109, smartmove0.8
Wins: 4705, Lo

Episode: 19000, Win Rate: 0.25, Epsilon: 0.010, smartmove0.8
Wins: 4724, Losses: 10695, Draws: 3582
Episode: 19100, Win Rate: 0.25, Epsilon: 0.010, smartmove0.8
Wins: 4744, Losses: 10739, Draws: 3618
Episode: 19200, Win Rate: 0.25, Epsilon: 0.010, smartmove0.8
Wins: 4762, Losses: 10777, Draws: 3662
Episode: 19300, Win Rate: 0.25, Epsilon: 0.010, smartmove0.8
Wins: 4785, Losses: 10813, Draws: 3703
Episode: 19400, Win Rate: 0.25, Epsilon: 0.010, smartmove0.8
Wins: 4807, Losses: 10844, Draws: 3750
Episode: 19500, Win Rate: 0.25, Epsilon: 0.010, smartmove0.8
Wins: 4827, Losses: 10879, Draws: 3795
Episode: 19600, Win Rate: 0.25, Epsilon: 0.010, smartmove0.8
Wins: 4844, Losses: 10925, Draws: 3832
Episode: 19700, Win Rate: 0.25, Epsilon: 0.010, smartmove0.8
Wins: 4869, Losses: 10961, Draws: 3871
Episode: 19800, Win Rate: 0.25, Epsilon: 0.010, smartmove0.8
Wins: 4889, Losses: 10997, Draws: 3915
Episode: 19900, Win Rate: 0.25, Epsilon: 0.010, smartmove0.8
Wins: 4919, Losses: 11026, Draws: 3956


In [ ]:
import numpy as np
import random
import os
from collections import deque
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from TicTacToe import TicTacToe

class SQNAgent:
    def __init__(self, state_size=9, action_size=9, gamma=0.95, model_path='model6.h5'):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = gamma  
        self.learning_rate = 0.001
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.9995
        self.batch_size = 32
        self.replay_buffer = deque(maxlen=10000)
        self.model_path = model_path

        # Try to load the model if it exists, otherwise create a new one
        if os.path.exists(model_path):
            try:
                print(f'{model_path} exists. Loading the model.')
                self.model = load_model(model_path, compile=False)
                # Recompile the model with explicit loss function
                self.model.compile(
                    loss=MeanSquaredError(),
                    optimizer=Adam(learning_rate=self.learning_rate)
                )
            except Exception as e:
                print(f"Error loading model: {e}")
                print("Creating new model instead.")
                self.model = self._build_model()
        else:
            print(f'{model_path} does not exist. Starting new training.')
            self.model = self._build_model()

    def _build_model(self):
        model = Sequential([
            Dense(64, input_dim=self.state_size, activation='relu'),
            Dense(64, activation='relu'),
            Dense(self.action_size, activation='linear')
        ])
        model.compile(
            loss=MeanSquaredError(),
            optimizer=Adam(learning_rate=self.learning_rate)
        )
        return model

    def process_state(self, state):
        """
        Converts the state from [0,1,2] format to [-1,0,1] format for better learning.
        """
        processed_state = np.array(state).copy()
        processed_state[processed_state == 0] = -1
        processed_state[processed_state == 1] = 0
        processed_state[processed_state == 2] = 1
        return processed_state

    def select_action(self, state, valid_actions):
        if not valid_actions:
            return None

        if random.random() < self.epsilon:
            return random.choice(valid_actions)
        
        processed_state = self.process_state(state)
        q_values = self.model.predict(processed_state.reshape(1, -1), verbose=0)[0]
        valid_q_values = [(q_values[action], action) for action in valid_actions]
        return max(valid_q_values, key=lambda x: x[0])[1]

    def store_experience(self, state, action, reward, next_state, done):
        self.replay_buffer.append((state, action, reward, next_state, done))

    def train(self):
        if len(self.replay_buffer) < self.batch_size:
            return

        mini_batch = random.sample(self.replay_buffer, self.batch_size)
        states = np.array([self.process_state(exp[0]) for exp in mini_batch])
        next_states = np.array([self.process_state(exp[3]) for exp in mini_batch])
        
        # Use batch prediction for efficiency
        current_q_values = self.model.predict(states, verbose=0)
        next_q_values = self.model.predict(next_states, verbose=0)

        x = []
        y = []

        for i, (state, action, reward, next_state, done) in enumerate(mini_batch):
            if done:
                target = reward
            else:
                next_valid_actions = [j for j, val in enumerate(next_state) if val == 0]
                if next_valid_actions:
                    max_next_q = max(next_q_values[i][action] for action in next_valid_actions)
                    target = reward + self.gamma * max_next_q
                else:
                    target = reward
            
            current_q = current_q_values[i].copy()
            current_q[action] = target
            x.append(self.process_state(state))
            y.append(current_q)

        self.model.fit(np.array(x), np.array(y), batch_size=self.batch_size, epochs=1, verbose=0)

def train_agent(episodes=20000, model_path='model6.h5'):
    agent = SQNAgent(model_path=model_path)
    history = {'wins': 0, 'losses': 0, 'draws': 0}
    initial_epsilon = 1.0
    min_epsilon = 0.01
    decay_episodes = 1000
    
    for episode in range(episodes):
        if episode % 1000 == 0 and episode > 0:
            agent.epsilon = 1.0
            print("Resetting epsilon to 1.0")
        else:
            epsilon_decay = (initial_epsilon - min_epsilon) / decay_episodes
            agent.epsilon = max(min_epsilon, initial_epsilon - (episode % 1000) * epsilon_decay)
        
        smartness = min(1, episode / (episodes * 0.6))
        game = TicTacToe(smartMovePlayer1=smartness)
        state = np.array(game.board)
        
        game.player1_move()
        state = np.array(game.board)
        
        while True:
            valid_actions = game.empty_positions()
            if not valid_actions:
                history['draws'] += 1
                break
                
            action = agent.select_action(state, valid_actions)
            game.make_move(action, player=2)
            
            reward = 0
            if game.current_winner == 2:
                reward = 1.0
                history['wins'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            
            game.player1_move()
            if game.current_winner == 1:
                reward = -1.0
                history['losses'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            elif game.is_full():
                reward = min(-0.1, -0.5 * smartness)
                history['draws'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            
            agent.store_experience(state, action, reward, game.board, False)
            state = np.array(game.board)
        
        agent.train()
        
        if episode % 100 == 0:
            win_rate = history['wins'] / (episode + 1)
            print(f"Episode: {episode}, Win Rate: {win_rate:.2f}, Epsilon: {agent.epsilon:.3f}, smartmove: {smartness:.2f}")
            print(f"Wins: {history['wins']}, Losses: {history['losses']}, Draws: {history['draws']}")
        
        if episode % 1000 == 0:
            try:
                agent.model.save(f'model6_retrain_episode_{episode}.h5')
                print(f"Checkpoint saved at episode {episode}")
            except Exception as e:
                print(f"Error saving checkpoint: {e}")

    # Save the final model
    try:
        agent.model.save("model6_retrain.h5")
        print(f'Final model saved as model6_retrain.h5')
    except Exception as e:
        print(f"Error saving final model: {e}")


train_agent()

model6.h5 exists. Loading the model.
Episode: 0, Win Rate: 0.00, Epsilon: 1.000, smartmove: 0.00
Wins: 0, Losses: 1, Draws: 0
Checkpoint saved at episode 0
Episode: 100, Win Rate: 0.22, Epsilon: 0.901, smartmove: 0.01
Wins: 22, Losses: 61, Draws: 18
Episode: 200, Win Rate: 0.26, Epsilon: 0.802, smartmove: 0.02
Wins: 53, Losses: 122, Draws: 26
Episode: 300, Win Rate: 0.29, Epsilon: 0.703, smartmove: 0.03
Wins: 88, Losses: 178, Draws: 35
Episode: 400, Win Rate: 0.32, Epsilon: 0.604, smartmove: 0.03
Wins: 129, Losses: 224, Draws: 48
Episode: 500, Win Rate: 0.33, Epsilon: 0.505, smartmove: 0.04
Wins: 164, Losses: 274, Draws: 63
Episode: 600, Win Rate: 0.34, Epsilon: 0.406, smartmove: 0.05
Wins: 205, Losses: 319, Draws: 77
Episode: 700, Win Rate: 0.35, Epsilon: 0.307, smartmove: 0.06
Wins: 243, Losses: 360, Draws: 98
Episode: 800, Win Rate: 0.37, Epsilon: 0.208, smartmove: 0.07
Wins: 293, Losses: 394, Draws: 114
Episode: 900, Win Rate: 0.37, Epsilon: 0.109, smartmove: 0.07
Wins: 337, Losses

Resetting epsilon to 1.0
Episode: 1000, Win Rate: 0.38, Epsilon: 1.000, smartmove: 0.08
Wins: 378, Losses: 480, Draws: 143
Checkpoint saved at episode 1000
Episode: 1100, Win Rate: 0.36, Epsilon: 0.901, smartmove: 0.09
Wins: 401, Losses: 545, Draws: 155
Episode: 1200, Win Rate: 0.35, Epsilon: 0.802, smartmove: 0.10
Wins: 424, Losses: 608, Draws: 169
Episode: 1300, Win Rate: 0.35, Epsilon: 0.703, smartmove: 0.11
Wins: 452, Losses: 669, Draws: 180
Episode: 1400, Win Rate: 0.34, Epsilon: 0.604, smartmove: 0.12
Wins: 478, Losses: 728, Draws: 195
Episode: 1500, Win Rate: 0.34, Epsilon: 0.505, smartmove: 0.12
Wins: 505, Losses: 783, Draws: 213
Episode: 1600, Win Rate: 0.34, Epsilon: 0.406, smartmove: 0.13
Wins: 544, Losses: 835, Draws: 222
Episode: 1700, Win Rate: 0.34, Epsilon: 0.307, smartmove: 0.14
Wins: 586, Losses: 880, Draws: 235
Episode: 1800, Win Rate: 0.35, Epsilon: 0.208, smartmove: 0.15
Wins: 633, Losses: 915, Draws: 253
Episode: 1900, Win Rate: 0.35, Epsilon: 0.109, smartmove: 0.

Resetting epsilon to 1.0
Episode: 2000, Win Rate: 0.36, Epsilon: 1.000, smartmove: 0.17
Wins: 712, Losses: 1013, Draws: 276
Checkpoint saved at episode 2000
Episode: 2100, Win Rate: 0.35, Epsilon: 0.901, smartmove: 0.17
Wins: 733, Losses: 1081, Draws: 287
Episode: 2200, Win Rate: 0.34, Epsilon: 0.802, smartmove: 0.18
Wins: 759, Losses: 1141, Draws: 301
Episode: 2300, Win Rate: 0.34, Epsilon: 0.703, smartmove: 0.19
Wins: 780, Losses: 1196, Draws: 325
Episode: 2400, Win Rate: 0.33, Epsilon: 0.604, smartmove: 0.20
Wins: 800, Losses: 1260, Draws: 341
Episode: 2500, Win Rate: 0.33, Epsilon: 0.505, smartmove: 0.21
Wins: 827, Losses: 1310, Draws: 364
Episode: 2600, Win Rate: 0.33, Epsilon: 0.406, smartmove: 0.22
Wins: 860, Losses: 1358, Draws: 383
Episode: 2700, Win Rate: 0.33, Epsilon: 0.307, smartmove: 0.23
Wins: 898, Losses: 1400, Draws: 403
Episode: 2800, Win Rate: 0.33, Epsilon: 0.208, smartmove: 0.23
Wins: 926, Losses: 1455, Draws: 420
Episode: 2900, Win Rate: 0.33, Epsilon: 0.109, smar

Resetting epsilon to 1.0
Episode: 3000, Win Rate: 0.33, Epsilon: 1.000, smartmove: 0.25
Wins: 994, Losses: 1539, Draws: 468
Checkpoint saved at episode 3000
Episode: 3100, Win Rate: 0.33, Epsilon: 0.901, smartmove: 0.26
Wins: 1016, Losses: 1609, Draws: 476
Episode: 3200, Win Rate: 0.32, Epsilon: 0.802, smartmove: 0.27
Wins: 1038, Losses: 1679, Draws: 484
Episode: 3300, Win Rate: 0.32, Epsilon: 0.703, smartmove: 0.28
Wins: 1055, Losses: 1751, Draws: 495
Episode: 3400, Win Rate: 0.32, Epsilon: 0.604, smartmove: 0.28
Wins: 1091, Losses: 1802, Draws: 508
Episode: 3500, Win Rate: 0.32, Epsilon: 0.505, smartmove: 0.29
Wins: 1127, Losses: 1855, Draws: 519
Episode: 3600, Win Rate: 0.32, Epsilon: 0.406, smartmove: 0.30
Wins: 1155, Losses: 1910, Draws: 536
Episode: 3700, Win Rate: 0.32, Epsilon: 0.307, smartmove: 0.31
Wins: 1189, Losses: 1957, Draws: 555
Episode: 3800, Win Rate: 0.32, Epsilon: 0.208, smartmove: 0.32
Wins: 1218, Losses: 2012, Draws: 571
Episode: 3900, Win Rate: 0.32, Epsilon: 0.1

Resetting epsilon to 1.0
Episode: 4000, Win Rate: 0.32, Epsilon: 1.000, smartmove: 0.33
Wins: 1276, Losses: 2125, Draws: 600
Checkpoint saved at episode 4000
Episode: 4100, Win Rate: 0.31, Epsilon: 0.901, smartmove: 0.34
Wins: 1291, Losses: 2202, Draws: 608
Episode: 4200, Win Rate: 0.31, Epsilon: 0.802, smartmove: 0.35
Wins: 1315, Losses: 2265, Draws: 621
Episode: 4300, Win Rate: 0.31, Epsilon: 0.703, smartmove: 0.36
Wins: 1335, Losses: 2330, Draws: 636
Episode: 4400, Win Rate: 0.31, Epsilon: 0.604, smartmove: 0.37
Wins: 1351, Losses: 2397, Draws: 653
Episode: 4500, Win Rate: 0.30, Epsilon: 0.505, smartmove: 0.38
Wins: 1367, Losses: 2463, Draws: 671
Episode: 4600, Win Rate: 0.30, Epsilon: 0.406, smartmove: 0.38
Wins: 1386, Losses: 2528, Draws: 687
Episode: 4700, Win Rate: 0.30, Epsilon: 0.307, smartmove: 0.39
Wins: 1419, Losses: 2584, Draws: 698
Episode: 4800, Win Rate: 0.30, Epsilon: 0.208, smartmove: 0.40
Wins: 1439, Losses: 2641, Draws: 721
Episode: 4900, Win Rate: 0.30, Epsilon: 0.

...

Resetting epsilon to 1.0
Episode: 16000, Win Rate: 0.15, Epsilon: 1.000, smartmove: 1.00
Wins: 2459, Losses: 10035, Draws: 3507
Checkpoint saved at episode 16000
Episode: 16100, Win Rate: 0.15, Epsilon: 0.901, smartmove: 1.00
Wins: 2460, Losses: 10123, Draws: 3518
Episode: 16200, Win Rate: 0.15, Epsilon: 0.802, smartmove: 1.00
Wins: 2462, Losses: 10203, Draws: 3536
Episode: 16300, Win Rate: 0.15, Epsilon: 0.703, smartmove: 1.00
Wins: 2464, Losses: 10289, Draws: 3548
Episode: 16400, Win Rate: 0.15, Epsilon: 0.604, smartmove: 1.00
Wins: 2468, Losses: 10371, Draws: 3562
Episode: 16500, Win Rate: 0.15, Epsilon: 0.505, smartmove: 1.00
Wins: 2468, Losses: 10440, Draws: 3593
Episode: 16600, Win Rate: 0.15, Epsilon: 0.406, smartmove: 1.00
Wins: 2477, Losses: 10502, Draws: 3622
Episode: 16700, Win Rate: 0.15, Epsilon: 0.307, smartmove: 1.00
Wins: 2480, Losses: 10564, Draws: 3657
Episode: 16800, Win Rate: 0.15, Epsilon: 0.208, smartmove: 1.00
Wins: 2485, Losses: 10611, Draws: 3705
Episode: 16900

Resetting epsilon to 1.0
Episode: 17000, Win Rate: 0.15, Epsilon: 1.000, smartmove: 1.00
Wins: 2495, Losses: 10687, Draws: 3819
Checkpoint saved at episode 17000
Episode: 17100, Win Rate: 0.15, Epsilon: 0.901, smartmove: 1.00
Wins: 2495, Losses: 10779, Draws: 3827
Episode: 17200, Win Rate: 0.15, Epsilon: 0.802, smartmove: 1.00
Wins: 2496, Losses: 10862, Draws: 3843
Episode: 17300, Win Rate: 0.14, Epsilon: 0.703, smartmove: 1.00
Wins: 2497, Losses: 10949, Draws: 3855
Episode: 17400, Win Rate: 0.14, Epsilon: 0.604, smartmove: 1.00
Wins: 2498, Losses: 11022, Draws: 3881
Episode: 17500, Win Rate: 0.14, Epsilon: 0.505, smartmove: 1.00
Wins: 2500, Losses: 11099, Draws: 3902
Episode: 17600, Win Rate: 0.14, Epsilon: 0.406, smartmove: 1.00
Wins: 2504, Losses: 11162, Draws: 3935
Episode: 17700, Win Rate: 0.14, Epsilon: 0.307, smartmove: 1.00
Wins: 2505, Losses: 11224, Draws: 3972
Episode: 17800, Win Rate: 0.14, Epsilon: 0.208, smartmove: 1.00
Wins: 2508, Losses: 11286, Draws: 4007
Episode: 17900

Resetting epsilon to 1.0
Episode: 18000, Win Rate: 0.14, Epsilon: 1.000, smartmove: 1.00
Wins: 2518, Losses: 11375, Draws: 4108
Checkpoint saved at episode 18000
Episode: 18100, Win Rate: 0.14, Epsilon: 0.901, smartmove: 1.00
Wins: 2519, Losses: 11463, Draws: 4119
Episode: 18200, Win Rate: 0.14, Epsilon: 0.802, smartmove: 1.00
Wins: 2522, Losses: 11550, Draws: 4129
Episode: 18300, Win Rate: 0.14, Epsilon: 0.703, smartmove: 1.00
Wins: 2525, Losses: 11623, Draws: 4153
Episode: 18400, Win Rate: 0.14, Epsilon: 0.604, smartmove: 1.00
Wins: 2527, Losses: 11700, Draws: 4174
Episode: 18500, Win Rate: 0.14, Epsilon: 0.505, smartmove: 1.00
Wins: 2528, Losses: 11772, Draws: 4201
Episode: 18600, Win Rate: 0.14, Epsilon: 0.406, smartmove: 1.00
Wins: 2534, Losses: 11845, Draws: 4222
Episode: 18700, Win Rate: 0.14, Epsilon: 0.307, smartmove: 1.00
Wins: 2537, Losses: 11899, Draws: 4265
Episode: 18800, Win Rate: 0.14, Epsilon: 0.208, smartmove: 1.00
Wins: 2544, Losses: 11948, Draws: 4309
Episode: 18900

Resetting epsilon to 1.0
Episode: 19000, Win Rate: 0.13, Epsilon: 1.000, smartmove: 1.00
Wins: 2552, Losses: 12037, Draws: 4412
Checkpoint saved at episode 19000
Episode: 19100, Win Rate: 0.13, Epsilon: 0.901, smartmove: 1.00
Wins: 2553, Losses: 12128, Draws: 4420
Episode: 19200, Win Rate: 0.13, Epsilon: 0.802, smartmove: 1.00
Wins: 2554, Losses: 12214, Draws: 4433
Episode: 19300, Win Rate: 0.13, Epsilon: 0.703, smartmove: 1.00
Wins: 2554, Losses: 12302, Draws: 4445
Episode: 19400, Win Rate: 0.13, Epsilon: 0.604, smartmove: 1.00
Wins: 2558, Losses: 12379, Draws: 4464
Episode: 19500, Win Rate: 0.13, Epsilon: 0.505, smartmove: 1.00
Wins: 2559, Losses: 12456, Draws: 4486
Episode: 19600, Win Rate: 0.13, Epsilon: 0.406, smartmove: 1.00
Wins: 2563, Losses: 12526, Draws: 4512
Episode: 19700, Win Rate: 0.13, Epsilon: 0.307, smartmove: 1.00
Wins: 2567, Losses: 12583, Draws: 4551
Episode: 19800, Win Rate: 0.13, Epsilon: 0.208, smartmove: 1.00
Wins: 2575, Losses: 12628, Draws: 4598
Episode: 19900

Final model saved as model6_retrain.h5


In [33]:
import numpy as np
import random
import os
from collections import deque
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from TicTacToe import TicTacToe

class SQNAgent:
    def __init__(self, state_size=9, action_size=9, gamma=0.95, model_path='model6.h5'):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = gamma  
        self.learning_rate = 0.001
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.9995
        self.batch_size = 32
        self.replay_buffer = deque(maxlen=10000)
        self.model_path = model_path

        # Try to load the model if it exists, otherwise create a new one
        if os.path.exists(model_path):
            try:
                print(f'{model_path} exists. Loading the model.')
                self.model = load_model(model_path, compile=False)
                # Recompile the model with explicit loss function
                self.model.compile(
                    loss=MeanSquaredError(),
                    optimizer=Adam(learning_rate=self.learning_rate)
                )
            except Exception as e:
                print(f"Error loading model: {e}")
                print("Creating new model instead.")
                self.model = self._build_model()
        else:
            print(f'{model_path} does not exist. Starting new training.')
            self.model = self._build_model()

    def _build_model(self):
        model = Sequential([
            Dense(64, input_dim=self.state_size, activation='relu'),
            Dense(64, activation='relu'),
            Dense(self.action_size, activation='linear')
        ])
        model.compile(
            loss=MeanSquaredError(),
            optimizer=Adam(learning_rate=self.learning_rate)
        )
        return model

    def process_state(self, state):
        """
        Converts the state from [0,1,2] format to [-1,0,1] format for better learning.
        """
        processed_state = np.array(state).copy()
        processed_state[processed_state == 0] = -1
        processed_state[processed_state == 1] = 0
        processed_state[processed_state == 2] = 1
        return processed_state

    def select_action(self, state, valid_actions):
        if not valid_actions:
            return None

        if random.random() < self.epsilon:
            return random.choice(valid_actions)
        
        processed_state = self.process_state(state)
        q_values = self.model.predict(processed_state.reshape(1, -1), verbose=0)[0]
        valid_q_values = [(q_values[action], action) for action in valid_actions]
        return max(valid_q_values, key=lambda x: x[0])[1]

    def store_experience(self, state, action, reward, next_state, done):
        self.replay_buffer.append((state, action, reward, next_state, done))

    def train(self):
        if len(self.replay_buffer) < self.batch_size:
            return

        mini_batch = random.sample(self.replay_buffer, self.batch_size)
        states = np.array([self.process_state(exp[0]) for exp in mini_batch])
        next_states = np.array([self.process_state(exp[3]) for exp in mini_batch])
        
        # Use batch prediction for efficiency
        current_q_values = self.model.predict(states, verbose=0)
        next_q_values = self.model.predict(next_states, verbose=0)

        x = []
        y = []

        for i, (state, action, reward, next_state, done) in enumerate(mini_batch):
            if done:
                target = reward
            else:
                next_valid_actions = [j for j, val in enumerate(next_state) if val == 0]
                if next_valid_actions:
                    max_next_q = max(next_q_values[i][action] for action in next_valid_actions)
                    target = reward + self.gamma * max_next_q
                else:
                    target = reward
            
            current_q = current_q_values[i].copy()
            current_q[action] = target
            x.append(self.process_state(state))
            y.append(current_q)

        self.model.fit(np.array(x), np.array(y), batch_size=self.batch_size, epochs=1, verbose=0)

def train_agent(episodes=40000, model_path='model6_retrain_retrain_retrain.h5'):
    agent = SQNAgent(model_path=model_path)
    history = {'wins': 0, 'losses': 0, 'draws': 0}
    initial_epsilon = 1.0
    min_epsilon = 0.01
    decay_episodes = 1000
    smartness=0
    for episode in range(episodes):
        if episode % 1000 == 0 and episode > 0:
            
            print("Resetting epsilon to 1.0")
        else:
            epsilon_decay = (initial_epsilon - min_epsilon) / decay_episodes
            agent.epsilon = max(min_epsilon, initial_epsilon - (episode % 1000) * epsilon_decay)
        
        smartness = min(1, episode / (episodes * 0.9))
        game = TicTacToe(smartMovePlayer1=smartness)
        state = np.array(game.board)
        
        game.player1_move()
        state = np.array(game.board)
        
        while True:
            valid_actions = game.empty_positions()
            if not valid_actions:
                history['draws'] += 1
                break
                
            action = agent.select_action(state, valid_actions)
            game.make_move(action, player=2)
            
            reward = 0
            if game.current_winner == 2:
                reward = 1.0
                history['wins'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            
            game.player1_move()
            if game.current_winner == 1:
                reward = -1.0
                history['losses'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            elif game.is_full():
                reward = min(-0.1, -0.5 * smartness)
                history['draws'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            
            agent.store_experience(state, action, reward, game.board, False)
            state = np.array(game.board)
        
        agent.train()
        
        if episode % 100 == 0:
            win_rate = history['wins'] / (episode + 1)
            print(f"Episode: {episode}, Win Rate: {win_rate:.2f}, Epsilon: {agent.epsilon:.3f}, smartmove: {smartness:.2f}")
            print(f"Wins: {history['wins']}, Losses: {history['losses']}, Draws: {history['draws']}")
        
        if episode % 1000 == 0:
            try:
                agent.model.save(f'model6_retrain_retrain_episode_{episode}.h5')
                print(f"Checkpoint saved at episode {episode}")
            except Exception as e:
                print(f"Error saving checkpoint: {e}")

    # Save the final model
    try:
        agent.model.save("model6_retrain_retrain_retrain_retrain.h5")
        print(f'Final model saved as model6_retrain_retrain.h5')
    except Exception as e:
        print(f"Error saving final model: {e}")


train_agent()

model6_retrain_retrain_retrain.h5 exists. Loading the model.
Episode: 0, Win Rate: 0.00, Epsilon: 1.000, smartmove: 0.00
Wins: 0, Losses: 1, Draws: 0
Checkpoint saved at episode 0
Episode: 100, Win Rate: 0.33, Epsilon: 0.901, smartmove: 0.00
Wins: 33, Losses: 53, Draws: 15
Episode: 200, Win Rate: 0.32, Epsilon: 0.802, smartmove: 0.01
Wins: 64, Losses: 113, Draws: 24
Episode: 300, Win Rate: 0.34, Epsilon: 0.703, smartmove: 0.01
Wins: 103, Losses: 161, Draws: 37
Episode: 400, Win Rate: 0.37, Epsilon: 0.604, smartmove: 0.01
Wins: 147, Losses: 197, Draws: 57
Episode: 500, Win Rate: 0.38, Epsilon: 0.505, smartmove: 0.01
Wins: 189, Losses: 233, Draws: 79
Episode: 600, Win Rate: 0.40, Epsilon: 0.406, smartmove: 0.02
Wins: 238, Losses: 265, Draws: 98
Episode: 700, Win Rate: 0.41, Epsilon: 0.307, smartmove: 0.02
Wins: 284, Losses: 306, Draws: 111
Episode: 800, Win Rate: 0.43, Epsilon: 0.208, smartmove: 0.02
Wins: 343, Losses: 331, Draws: 127
Episode: 900, Win Rate: 0.45, Epsilon: 0.109, smartmo

Resetting epsilon to 1.0
Episode: 1000, Win Rate: 0.47, Epsilon: 0.011, smartmove: 0.03
Wins: 471, Losses: 370, Draws: 160
Checkpoint saved at episode 1000
Episode: 1100, Win Rate: 0.45, Epsilon: 0.901, smartmove: 0.03
Wins: 495, Losses: 434, Draws: 172
Episode: 1200, Win Rate: 0.44, Epsilon: 0.802, smartmove: 0.03
Wins: 530, Losses: 482, Draws: 189
Episode: 1300, Win Rate: 0.43, Epsilon: 0.703, smartmove: 0.04
Wins: 555, Losses: 540, Draws: 206
Episode: 1400, Win Rate: 0.42, Epsilon: 0.604, smartmove: 0.04
Wins: 595, Losses: 583, Draws: 223
Episode: 1500, Win Rate: 0.42, Epsilon: 0.505, smartmove: 0.04
Wins: 624, Losses: 623, Draws: 254
Episode: 1600, Win Rate: 0.42, Epsilon: 0.406, smartmove: 0.04
Wins: 667, Losses: 671, Draws: 263
Episode: 1700, Win Rate: 0.41, Epsilon: 0.307, smartmove: 0.05
Wins: 703, Losses: 709, Draws: 289
Episode: 1800, Win Rate: 0.41, Epsilon: 0.208, smartmove: 0.05
Wins: 747, Losses: 737, Draws: 317
Episode: 1900, Win Rate: 0.42, Epsilon: 0.109, smartmove: 0.

Resetting epsilon to 1.0
Episode: 2000, Win Rate: 0.43, Epsilon: 0.011, smartmove: 0.06
Wins: 855, Losses: 782, Draws: 364
Checkpoint saved at episode 2000
Episode: 2100, Win Rate: 0.42, Epsilon: 0.901, smartmove: 0.06
Wins: 890, Losses: 839, Draws: 372
Episode: 2200, Win Rate: 0.42, Epsilon: 0.802, smartmove: 0.06
Wins: 921, Losses: 897, Draws: 383
Episode: 2300, Win Rate: 0.41, Epsilon: 0.703, smartmove: 0.06
Wins: 950, Losses: 952, Draws: 399
Episode: 2400, Win Rate: 0.41, Epsilon: 0.604, smartmove: 0.07
Wins: 984, Losses: 1005, Draws: 412
Episode: 2500, Win Rate: 0.41, Epsilon: 0.505, smartmove: 0.07
Wins: 1023, Losses: 1049, Draws: 429
Episode: 2600, Win Rate: 0.41, Epsilon: 0.406, smartmove: 0.07
Wins: 1060, Losses: 1093, Draws: 448
Episode: 2700, Win Rate: 0.41, Epsilon: 0.307, smartmove: 0.07
Wins: 1109, Losses: 1129, Draws: 463
Episode: 2800, Win Rate: 0.41, Epsilon: 0.208, smartmove: 0.08
Wins: 1160, Losses: 1153, Draws: 488
Episode: 2900, Win Rate: 0.42, Epsilon: 0.109, smar

Resetting epsilon to 1.0
Episode: 3000, Win Rate: 0.42, Epsilon: 0.011, smartmove: 0.08
Wins: 1250, Losses: 1208, Draws: 543
Checkpoint saved at episode 3000
Episode: 3100, Win Rate: 0.41, Epsilon: 0.901, smartmove: 0.09
Wins: 1276, Losses: 1265, Draws: 560
Episode: 3200, Win Rate: 0.41, Epsilon: 0.802, smartmove: 0.09
Wins: 1302, Losses: 1325, Draws: 574
Episode: 3300, Win Rate: 0.40, Epsilon: 0.703, smartmove: 0.09
Wins: 1334, Losses: 1379, Draws: 588
Episode: 3400, Win Rate: 0.40, Epsilon: 0.604, smartmove: 0.09
Wins: 1362, Losses: 1435, Draws: 604
Episode: 3500, Win Rate: 0.40, Epsilon: 0.505, smartmove: 0.10
Wins: 1397, Losses: 1484, Draws: 620
Episode: 3600, Win Rate: 0.40, Epsilon: 0.406, smartmove: 0.10
Wins: 1437, Losses: 1516, Draws: 648
Episode: 3700, Win Rate: 0.40, Epsilon: 0.307, smartmove: 0.10
Wins: 1489, Losses: 1552, Draws: 660
Episode: 3800, Win Rate: 0.40, Epsilon: 0.208, smartmove: 0.11
Wins: 1535, Losses: 1590, Draws: 676
Episode: 3900, Win Rate: 0.41, Epsilon: 0.

Resetting epsilon to 1.0
Episode: 4000, Win Rate: 0.40, Epsilon: 0.011, smartmove: 0.11
Wins: 1619, Losses: 1643, Draws: 739
Checkpoint saved at episode 4000
Episode: 4100, Win Rate: 0.40, Epsilon: 0.901, smartmove: 0.11
Wins: 1646, Losses: 1704, Draws: 751
Episode: 4200, Win Rate: 0.40, Epsilon: 0.802, smartmove: 0.12
Wins: 1673, Losses: 1761, Draws: 767
Episode: 4300, Win Rate: 0.40, Epsilon: 0.703, smartmove: 0.12
Wins: 1702, Losses: 1818, Draws: 781
Episode: 4400, Win Rate: 0.39, Epsilon: 0.604, smartmove: 0.12
Wins: 1734, Losses: 1862, Draws: 805
Episode: 4500, Win Rate: 0.39, Epsilon: 0.505, smartmove: 0.12
Wins: 1766, Losses: 1910, Draws: 825
Episode: 4600, Win Rate: 0.39, Epsilon: 0.406, smartmove: 0.13
Wins: 1803, Losses: 1955, Draws: 843
Episode: 4700, Win Rate: 0.39, Epsilon: 0.307, smartmove: 0.13
Wins: 1841, Losses: 2000, Draws: 860
Episode: 4800, Win Rate: 0.39, Epsilon: 0.208, smartmove: 0.13
Wins: 1870, Losses: 2045, Draws: 886
Episode: 4900, Win Rate: 0.39, Epsilon: 0.

...

Resetting epsilon to 1.0
Episode: 36000, Win Rate: 0.24, Epsilon: 0.011, smartmove: 1.00
Wins: 8473, Losses: 18884, Draws: 8644
Checkpoint saved at episode 36000
Episode: 36100, Win Rate: 0.23, Epsilon: 0.901, smartmove: 1.00
Wins: 8475, Losses: 18973, Draws: 8653
Episode: 36200, Win Rate: 0.23, Epsilon: 0.802, smartmove: 1.00
Wins: 8476, Losses: 19060, Draws: 8665
Episode: 36300, Win Rate: 0.23, Epsilon: 0.703, smartmove: 1.00
Wins: 8480, Losses: 19144, Draws: 8677
Episode: 36400, Win Rate: 0.23, Epsilon: 0.604, smartmove: 1.00
Wins: 8484, Losses: 19220, Draws: 8697
Episode: 36500, Win Rate: 0.23, Epsilon: 0.505, smartmove: 1.00
Wins: 8486, Losses: 19295, Draws: 8720
Episode: 36600, Win Rate: 0.23, Epsilon: 0.406, smartmove: 1.00
Wins: 8490, Losses: 19355, Draws: 8756
Episode: 36700, Win Rate: 0.23, Epsilon: 0.307, smartmove: 1.00
Wins: 8496, Losses: 19401, Draws: 8804
Episode: 36800, Win Rate: 0.23, Epsilon: 0.208, smartmove: 1.00
Wins: 8500, Losses: 19456, Draws: 8845
Episode: 36900

Resetting epsilon to 1.0
Episode: 37000, Win Rate: 0.23, Epsilon: 0.011, smartmove: 1.00
Wins: 8503, Losses: 19529, Draws: 8969
Checkpoint saved at episode 37000
Episode: 37100, Win Rate: 0.23, Epsilon: 0.901, smartmove: 1.00
Wins: 8503, Losses: 19619, Draws: 8979
Episode: 37200, Win Rate: 0.23, Epsilon: 0.802, smartmove: 1.00
Wins: 8506, Losses: 19704, Draws: 8991
Episode: 37300, Win Rate: 0.23, Epsilon: 0.703, smartmove: 1.00
Wins: 8509, Losses: 19785, Draws: 9007
Episode: 37400, Win Rate: 0.23, Epsilon: 0.604, smartmove: 1.00
Wins: 8514, Losses: 19854, Draws: 9033
Episode: 37500, Win Rate: 0.23, Epsilon: 0.505, smartmove: 1.00
Wins: 8518, Losses: 19931, Draws: 9052
Episode: 37600, Win Rate: 0.23, Epsilon: 0.406, smartmove: 1.00
Wins: 8523, Losses: 19983, Draws: 9095
Episode: 37700, Win Rate: 0.23, Epsilon: 0.307, smartmove: 1.00
Wins: 8526, Losses: 20040, Draws: 9135
Episode: 37800, Win Rate: 0.23, Epsilon: 0.208, smartmove: 1.00
Wins: 8533, Losses: 20093, Draws: 9175
Episode: 37900

Resetting epsilon to 1.0
Episode: 38000, Win Rate: 0.22, Epsilon: 0.011, smartmove: 1.00
Wins: 8549, Losses: 20175, Draws: 9277
Checkpoint saved at episode 38000
Episode: 38100, Win Rate: 0.22, Epsilon: 0.901, smartmove: 1.00
Wins: 8551, Losses: 20264, Draws: 9286
Episode: 38200, Win Rate: 0.22, Epsilon: 0.802, smartmove: 1.00
Wins: 8551, Losses: 20343, Draws: 9307
Episode: 38300, Win Rate: 0.22, Epsilon: 0.703, smartmove: 1.00
Wins: 8553, Losses: 20428, Draws: 9320
Episode: 38400, Win Rate: 0.22, Epsilon: 0.604, smartmove: 1.00
Wins: 8557, Losses: 20500, Draws: 9344
Episode: 38500, Win Rate: 0.22, Epsilon: 0.505, smartmove: 1.00
Wins: 8562, Losses: 20574, Draws: 9365
Episode: 38600, Win Rate: 0.22, Epsilon: 0.406, smartmove: 1.00
Wins: 8568, Losses: 20639, Draws: 9394
Episode: 38700, Win Rate: 0.22, Epsilon: 0.307, smartmove: 1.00
Wins: 8574, Losses: 20698, Draws: 9429
Episode: 38800, Win Rate: 0.22, Epsilon: 0.208, smartmove: 1.00
Wins: 8579, Losses: 20744, Draws: 9478
Episode: 38900

Resetting epsilon to 1.0
Episode: 39000, Win Rate: 0.22, Epsilon: 0.011, smartmove: 1.00
Wins: 8588, Losses: 20820, Draws: 9593
Checkpoint saved at episode 39000
Episode: 39100, Win Rate: 0.22, Epsilon: 0.901, smartmove: 1.00
Wins: 8589, Losses: 20910, Draws: 9602
Episode: 39200, Win Rate: 0.22, Epsilon: 0.802, smartmove: 1.00
Wins: 8593, Losses: 20996, Draws: 9612
Episode: 39300, Win Rate: 0.22, Epsilon: 0.703, smartmove: 1.00
Wins: 8594, Losses: 21080, Draws: 9627
Episode: 39400, Win Rate: 0.22, Epsilon: 0.604, smartmove: 1.00
Wins: 8597, Losses: 21148, Draws: 9656
Episode: 39500, Win Rate: 0.22, Epsilon: 0.505, smartmove: 1.00
Wins: 8599, Losses: 21222, Draws: 9680
Episode: 39600, Win Rate: 0.22, Epsilon: 0.406, smartmove: 1.00
Wins: 8604, Losses: 21287, Draws: 9710
Episode: 39700, Win Rate: 0.22, Epsilon: 0.307, smartmove: 1.00
Wins: 8615, Losses: 21342, Draws: 9744
Episode: 39800, Win Rate: 0.22, Epsilon: 0.208, smartmove: 1.00
Wins: 8622, Losses: 21398, Draws: 9781
Episode: 39900

Final model saved as model6_retrain_retrain.h5


In [34]:
import numpy as np
import random
import os
from collections import deque
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from TicTacToe import TicTacToe

class SQNAgent:
    def __init__(self, state_size=9, action_size=9, gamma=0.95, model_path='model6.h5'):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = gamma  
        self.learning_rate = 0.001
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.9995
        self.batch_size = 32
        self.replay_buffer = deque(maxlen=10000)
        self.model_path = model_path

        # Try to load the model if it exists, otherwise create a new one
        if os.path.exists(model_path):
            try:
                print(f'{model_path} exists. Loading the model.')
                self.model = load_model(model_path, compile=False)
                # Recompile the model with explicit loss function
                self.model.compile(
                    loss=MeanSquaredError(),
                    optimizer=Adam(learning_rate=self.learning_rate)
                )
            except Exception as e:
                print(f"Error loading model: {e}")
                print("Creating new model instead.")
                self.model = self._build_model()
        else:
            print(f'{model_path} does not exist. Starting new training.')
            self.model = self._build_model()

    def _build_model(self):
        model = Sequential([
            Dense(64, input_dim=self.state_size, activation='relu'),
            Dense(64, activation='relu'),
            Dense(self.action_size, activation='linear')
        ])
        model.compile(
            loss=MeanSquaredError(),
            optimizer=Adam(learning_rate=self.learning_rate)
        )
        return model

    def process_state(self, state):
        """
        Converts the state from [0,1,2] format to [-1,0,1] format for better learning.
        """
        processed_state = np.array(state).copy()
        processed_state[processed_state == 0] = -1
        processed_state[processed_state == 1] = 0
        processed_state[processed_state == 2] = 1
        return processed_state

    def select_action(self, state, valid_actions):
        if not valid_actions:
            return None

        if random.random() < self.epsilon:
            return random.choice(valid_actions)
        
        processed_state = self.process_state(state)
        q_values = self.model.predict(processed_state.reshape(1, -1), verbose=0)[0]
        valid_q_values = [(q_values[action], action) for action in valid_actions]
        return max(valid_q_values, key=lambda x: x[0])[1]

    def store_experience(self, state, action, reward, next_state, done):
        self.replay_buffer.append((state, action, reward, next_state, done))

    def train(self):
        if len(self.replay_buffer) < self.batch_size:
            return

        mini_batch = random.sample(self.replay_buffer, self.batch_size)
        states = np.array([self.process_state(exp[0]) for exp in mini_batch])
        next_states = np.array([self.process_state(exp[3]) for exp in mini_batch])
        
        # Use batch prediction for efficiency
        current_q_values = self.model.predict(states, verbose=0)
        next_q_values = self.model.predict(next_states, verbose=0)

        x = []
        y = []

        for i, (state, action, reward, next_state, done) in enumerate(mini_batch):
            if done:
                target = reward
            else:
                next_valid_actions = [j for j, val in enumerate(next_state) if val == 0]
                if next_valid_actions:
                    max_next_q = max(next_q_values[i][action] for action in next_valid_actions)
                    target = reward + self.gamma * max_next_q
                else:
                    target = reward
            
            current_q = current_q_values[i].copy()
            current_q[action] = target
            x.append(self.process_state(state))
            y.append(current_q)

        self.model.fit(np.array(x), np.array(y), batch_size=self.batch_size, epochs=1, verbose=0)

def train_agent(episodes=40000, model_path='model6_retrain_retrain_retrain_retrain.h5'):
    agent = SQNAgent(model_path=model_path)
    history = {'wins': 0, 'losses': 0, 'draws': 0}
    initial_epsilon = 1.0
    min_epsilon = 0.01
    decay_episodes = 1000
    smartness=0
    for episode in range(episodes):
        if episode % 1000 == 0 and episode > 0:
            
            print("Resetting epsilon to 1.0")
        else:
            epsilon_decay = (initial_epsilon - min_epsilon) / decay_episodes
            agent.epsilon = max(min_epsilon, initial_epsilon - (episode % 1000) * epsilon_decay)
        
        smartness = min(1, episode / (episodes * 0.9))
        game = TicTacToe(smartMovePlayer1=smartness)
        state = np.array(game.board)
        
        game.player1_move()
        state = np.array(game.board)
        
        while True:
            valid_actions = game.empty_positions()
            if not valid_actions:
                history['draws'] += 1
                break
                
            action = agent.select_action(state, valid_actions)
            game.make_move(action, player=2)
            
            reward = 0
            if game.current_winner == 2:
                reward = 1.0
                history['wins'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            
            game.player1_move()
            if game.current_winner == 1:
                reward = -1.0
                history['losses'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            elif game.is_full():
                reward = min(-0.1, -0.5 * smartness)
                history['draws'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            
            agent.store_experience(state, action, reward, game.board, False)
            state = np.array(game.board)
        
        agent.train()
        
        if episode % 100 == 0:
            win_rate = history['wins'] / (episode + 1)
            print(f"Episode: {episode}, Win Rate: {win_rate:.2f}, Epsilon: {agent.epsilon:.3f}, smartmove: {smartness:.2f}")
            print(f"Wins: {history['wins']}, Losses: {history['losses']}, Draws: {history['draws']}")
        
        if episode % 1000 == 0:
            try:
                agent.model.save(f'model6_retrain_retrain_retrain_episode_{episode}.h5')
                print(f"Checkpoint saved at episode {episode}")
            except Exception as e:
                print(f"Error saving checkpoint: {e}")

    # Save the final model
    try:
        agent.model.save("model6_retrain_retrain_retrain_retrain_retrain.h5")
        print(f'Final model saved as model6_retrain_retrain.h5')
    except Exception as e:
        print(f"Error saving final model: {e}")


train_agent()

model6_retrain_retrain_retrain_retrain.h5 exists. Loading the model.
Episode: 0, Win Rate: 0.00, Epsilon: 1.000, smartmove: 0.00
Wins: 0, Losses: 0, Draws: 1
Checkpoint saved at episode 0
Episode: 100, Win Rate: 0.28, Epsilon: 0.901, smartmove: 0.00
Wins: 28, Losses: 61, Draws: 12
Episode: 200, Win Rate: 0.32, Epsilon: 0.802, smartmove: 0.01
Wins: 64, Losses: 114, Draws: 23
Episode: 300, Win Rate: 0.32, Epsilon: 0.703, smartmove: 0.01
Wins: 96, Losses: 170, Draws: 35
Episode: 400, Win Rate: 0.35, Epsilon: 0.604, smartmove: 0.01
Wins: 142, Losses: 207, Draws: 52
Episode: 500, Win Rate: 0.36, Epsilon: 0.505, smartmove: 0.01
Wins: 181, Losses: 251, Draws: 69
Episode: 600, Win Rate: 0.37, Epsilon: 0.406, smartmove: 0.02
Wins: 224, Losses: 296, Draws: 81
Episode: 700, Win Rate: 0.39, Epsilon: 0.307, smartmove: 0.02
Wins: 272, Losses: 330, Draws: 99
Episode: 800, Win Rate: 0.40, Epsilon: 0.208, smartmove: 0.02
Wins: 317, Losses: 369, Draws: 115
Episode: 900, Win Rate: 0.40, Epsilon: 0.109, s

Resetting epsilon to 1.0
Episode: 1000, Win Rate: 0.40, Epsilon: 0.011, smartmove: 0.03
Wins: 396, Losses: 447, Draws: 158
Checkpoint saved at episode 1000
Episode: 1100, Win Rate: 0.38, Epsilon: 0.901, smartmove: 0.03
Wins: 418, Losses: 510, Draws: 173
Episode: 1200, Win Rate: 0.38, Epsilon: 0.802, smartmove: 0.03
Wins: 451, Losses: 563, Draws: 187
Episode: 1300, Win Rate: 0.37, Epsilon: 0.703, smartmove: 0.04
Wins: 484, Losses: 612, Draws: 205
Episode: 1400, Win Rate: 0.37, Epsilon: 0.604, smartmove: 0.04
Wins: 520, Losses: 656, Draws: 225
Episode: 1500, Win Rate: 0.38, Epsilon: 0.505, smartmove: 0.04
Wins: 563, Losses: 695, Draws: 243
Episode: 1600, Win Rate: 0.38, Epsilon: 0.406, smartmove: 0.04
Wins: 609, Losses: 732, Draws: 260
Episode: 1700, Win Rate: 0.38, Epsilon: 0.307, smartmove: 0.05
Wins: 654, Losses: 772, Draws: 275
Episode: 1800, Win Rate: 0.39, Epsilon: 0.208, smartmove: 0.05
Wins: 694, Losses: 813, Draws: 294
Episode: 1900, Win Rate: 0.39, Epsilon: 0.109, smartmove: 0.

Resetting epsilon to 1.0
Episode: 2000, Win Rate: 0.39, Epsilon: 0.011, smartmove: 0.06
Wins: 788, Losses: 861, Draws: 352
Checkpoint saved at episode 2000
Episode: 2100, Win Rate: 0.39, Epsilon: 0.901, smartmove: 0.06
Wins: 811, Losses: 927, Draws: 363
Episode: 2200, Win Rate: 0.38, Epsilon: 0.802, smartmove: 0.06
Wins: 839, Losses: 983, Draws: 379
Episode: 2300, Win Rate: 0.38, Epsilon: 0.703, smartmove: 0.06
Wins: 877, Losses: 1031, Draws: 393
Episode: 2400, Win Rate: 0.38, Epsilon: 0.604, smartmove: 0.07
Wins: 914, Losses: 1080, Draws: 407
Episode: 2500, Win Rate: 0.38, Epsilon: 0.505, smartmove: 0.07
Wins: 947, Losses: 1126, Draws: 428
Episode: 2600, Win Rate: 0.38, Epsilon: 0.406, smartmove: 0.07
Wins: 985, Losses: 1171, Draws: 445
Episode: 2700, Win Rate: 0.38, Epsilon: 0.307, smartmove: 0.07
Wins: 1031, Losses: 1206, Draws: 464
Episode: 2800, Win Rate: 0.38, Epsilon: 0.208, smartmove: 0.08
Wins: 1061, Losses: 1251, Draws: 489
Episode: 2900, Win Rate: 0.38, Epsilon: 0.109, smart

Resetting epsilon to 1.0
Episode: 3000, Win Rate: 0.38, Epsilon: 0.011, smartmove: 0.08
Wins: 1143, Losses: 1307, Draws: 551
Checkpoint saved at episode 3000
Episode: 3100, Win Rate: 0.38, Epsilon: 0.901, smartmove: 0.09
Wins: 1170, Losses: 1370, Draws: 561
Episode: 3200, Win Rate: 0.37, Epsilon: 0.802, smartmove: 0.09
Wins: 1197, Losses: 1427, Draws: 577
Episode: 3300, Win Rate: 0.37, Epsilon: 0.703, smartmove: 0.09
Wins: 1232, Losses: 1480, Draws: 589
Episode: 3400, Win Rate: 0.37, Epsilon: 0.604, smartmove: 0.09
Wins: 1270, Losses: 1520, Draws: 611
Episode: 3500, Win Rate: 0.37, Epsilon: 0.505, smartmove: 0.10
Wins: 1310, Losses: 1559, Draws: 632
Episode: 3600, Win Rate: 0.37, Epsilon: 0.406, smartmove: 0.10
Wins: 1350, Losses: 1601, Draws: 650
Episode: 3700, Win Rate: 0.38, Epsilon: 0.307, smartmove: 0.10
Wins: 1408, Losses: 1623, Draws: 670
Episode: 3800, Win Rate: 0.38, Epsilon: 0.208, smartmove: 0.11
Wins: 1451, Losses: 1658, Draws: 692
Episode: 3900, Win Rate: 0.39, Epsilon: 0.

Resetting epsilon to 1.0
Episode: 4000, Win Rate: 0.39, Epsilon: 0.011, smartmove: 0.11
Wins: 1551, Losses: 1713, Draws: 737
Checkpoint saved at episode 4000
Episode: 4100, Win Rate: 0.38, Epsilon: 0.901, smartmove: 0.11
Wins: 1576, Losses: 1776, Draws: 749
Episode: 4200, Win Rate: 0.38, Epsilon: 0.802, smartmove: 0.12
Wins: 1603, Losses: 1834, Draws: 764
Episode: 4300, Win Rate: 0.38, Epsilon: 0.703, smartmove: 0.12
Wins: 1629, Losses: 1891, Draws: 781
Episode: 4400, Win Rate: 0.38, Epsilon: 0.604, smartmove: 0.12
Wins: 1670, Losses: 1935, Draws: 796
Episode: 4500, Win Rate: 0.38, Epsilon: 0.505, smartmove: 0.12
Wins: 1703, Losses: 1987, Draws: 811
Episode: 4600, Win Rate: 0.38, Epsilon: 0.406, smartmove: 0.13
Wins: 1742, Losses: 2029, Draws: 830
Episode: 4700, Win Rate: 0.38, Epsilon: 0.307, smartmove: 0.13
Wins: 1791, Losses: 2063, Draws: 847
Episode: 4800, Win Rate: 0.38, Epsilon: 0.208, smartmove: 0.13
Wins: 1835, Losses: 2097, Draws: 869
Episode: 4900, Win Rate: 0.38, Epsilon: 0.

...

Resetting epsilon to 1.0
Episode: 36000, Win Rate: 0.25, Epsilon: 0.011, smartmove: 1.00
Wins: 8822, Losses: 18419, Draws: 8760
Checkpoint saved at episode 36000
Episode: 36100, Win Rate: 0.24, Epsilon: 0.901, smartmove: 1.00
Wins: 8826, Losses: 18506, Draws: 8769
Episode: 36200, Win Rate: 0.24, Epsilon: 0.802, smartmove: 1.00
Wins: 8829, Losses: 18590, Draws: 8782
Episode: 36300, Win Rate: 0.24, Epsilon: 0.703, smartmove: 1.00
Wins: 8835, Losses: 18667, Draws: 8799
Episode: 36400, Win Rate: 0.24, Epsilon: 0.604, smartmove: 1.00
Wins: 8836, Losses: 18745, Draws: 8820
Episode: 36500, Win Rate: 0.24, Epsilon: 0.505, smartmove: 1.00
Wins: 8839, Losses: 18809, Draws: 8853
Episode: 36600, Win Rate: 0.24, Epsilon: 0.406, smartmove: 1.00
Wins: 8840, Losses: 18879, Draws: 8882
Episode: 36700, Win Rate: 0.24, Epsilon: 0.307, smartmove: 1.00
Wins: 8845, Losses: 18929, Draws: 8927
Episode: 36800, Win Rate: 0.24, Epsilon: 0.208, smartmove: 1.00
Wins: 8848, Losses: 18980, Draws: 8973
Episode: 36900

Resetting epsilon to 1.0
Episode: 37000, Win Rate: 0.24, Epsilon: 0.011, smartmove: 1.00
Wins: 8864, Losses: 19069, Draws: 9068
Checkpoint saved at episode 37000
Episode: 37100, Win Rate: 0.24, Epsilon: 0.901, smartmove: 1.00
Wins: 8864, Losses: 19154, Draws: 9083
Episode: 37200, Win Rate: 0.24, Epsilon: 0.802, smartmove: 1.00
Wins: 8864, Losses: 19240, Draws: 9097
Episode: 37300, Win Rate: 0.24, Epsilon: 0.703, smartmove: 1.00
Wins: 8864, Losses: 19323, Draws: 9114
Episode: 37400, Win Rate: 0.24, Epsilon: 0.604, smartmove: 1.00
Wins: 8865, Losses: 19400, Draws: 9136
Episode: 37500, Win Rate: 0.24, Epsilon: 0.505, smartmove: 1.00
Wins: 8866, Losses: 19471, Draws: 9164
Episode: 37600, Win Rate: 0.24, Epsilon: 0.406, smartmove: 1.00
Wins: 8868, Losses: 19530, Draws: 9203
Episode: 37700, Win Rate: 0.24, Epsilon: 0.307, smartmove: 1.00
Wins: 8874, Losses: 19590, Draws: 9237
Episode: 37800, Win Rate: 0.23, Epsilon: 0.208, smartmove: 1.00
Wins: 8882, Losses: 19638, Draws: 9281
Episode: 37900

Resetting epsilon to 1.0
Episode: 38000, Win Rate: 0.23, Epsilon: 0.011, smartmove: 1.00
Wins: 8894, Losses: 19726, Draws: 9381
Checkpoint saved at episode 38000
Episode: 38100, Win Rate: 0.23, Epsilon: 0.901, smartmove: 1.00
Wins: 8896, Losses: 19818, Draws: 9387
Episode: 38200, Win Rate: 0.23, Epsilon: 0.802, smartmove: 1.00
Wins: 8899, Losses: 19906, Draws: 9396
Episode: 38300, Win Rate: 0.23, Epsilon: 0.703, smartmove: 1.00
Wins: 8903, Losses: 19983, Draws: 9415
Episode: 38400, Win Rate: 0.23, Epsilon: 0.604, smartmove: 1.00
Wins: 8908, Losses: 20057, Draws: 9436
Episode: 38500, Win Rate: 0.23, Epsilon: 0.505, smartmove: 1.00
Wins: 8911, Losses: 20131, Draws: 9459
Episode: 38600, Win Rate: 0.23, Epsilon: 0.406, smartmove: 1.00
Wins: 8914, Losses: 20200, Draws: 9487
Episode: 38700, Win Rate: 0.23, Epsilon: 0.307, smartmove: 1.00
Wins: 8920, Losses: 20256, Draws: 9525
Episode: 38800, Win Rate: 0.23, Epsilon: 0.208, smartmove: 1.00
Wins: 8924, Losses: 20301, Draws: 9576
Episode: 38900

Resetting epsilon to 1.0
Episode: 39000, Win Rate: 0.23, Epsilon: 0.011, smartmove: 1.00
Wins: 8939, Losses: 20366, Draws: 9696
Checkpoint saved at episode 39000
Episode: 39100, Win Rate: 0.23, Epsilon: 0.901, smartmove: 1.00
Wins: 8941, Losses: 20455, Draws: 9705
Episode: 39200, Win Rate: 0.23, Epsilon: 0.802, smartmove: 1.00
Wins: 8944, Losses: 20542, Draws: 9715
Episode: 39300, Win Rate: 0.23, Epsilon: 0.703, smartmove: 1.00
Wins: 8946, Losses: 20624, Draws: 9731
Episode: 39400, Win Rate: 0.23, Epsilon: 0.604, smartmove: 1.00
Wins: 8950, Losses: 20702, Draws: 9749
Episode: 39500, Win Rate: 0.23, Epsilon: 0.505, smartmove: 1.00
Wins: 8952, Losses: 20769, Draws: 9780
Episode: 39600, Win Rate: 0.23, Epsilon: 0.406, smartmove: 1.00
Wins: 8955, Losses: 20833, Draws: 9813
Episode: 39700, Win Rate: 0.23, Epsilon: 0.307, smartmove: 1.00
Wins: 8958, Losses: 20894, Draws: 9849
Episode: 39800, Win Rate: 0.23, Epsilon: 0.208, smartmove: 1.00
Wins: 8963, Losses: 20945, Draws: 9893
Episode: 39900

Final model saved as model6_retrain_retrain.h5


In [24]:
import numpy as np
import random
import os
from collections import deque
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from TicTacToe import TicTacToe

class SQNAgent:
    def __init__(self, state_size=9, action_size=9, gamma=0.95, model_path='model6.h5'):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = gamma  
        self.learning_rate = 0.001
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.9995
        self.batch_size = 32
        self.replay_buffer = deque(maxlen=10000)
        self.model_path = model_path

        # Try to load the model if it exists, otherwise create a new one
        if os.path.exists(model_path):
            try:
                print(f'{model_path} exists. Loading the model.')
                self.model = load_model(model_path, compile=False)
                # Recompile the model with explicit loss function
                self.model.compile(
                    loss=MeanSquaredError(),
                    optimizer=Adam(learning_rate=self.learning_rate)
                )
            except Exception as e:
                print(f"Error loading model: {e}")
                print("Creating new model instead.")
                self.model = self._build_model()
        else:
            print(f'{model_path} does not exist. Starting new training.')
            self.model = self._build_model()

    def _build_model(self):
        model = Sequential([
            Dense(64, input_dim=self.state_size, activation='relu'),
            Dense(64, activation='relu'),
            Dense(self.action_size, activation='linear')
        ])
        model.compile(
            loss=MeanSquaredError(),
            optimizer=Adam(learning_rate=self.learning_rate)
        )
        return model

    def process_state(self, state):
        """
        Converts the state from [0,1,2] format to [-1,0,1] format for better learning.
        """
        processed_state = np.array(state).copy()
        processed_state[processed_state == 0] = -1
        processed_state[processed_state == 1] = 0
        processed_state[processed_state == 2] = 1
        return processed_state

    def select_action(self, state, valid_actions):
        if not valid_actions:
            return None

        if random.random() < self.epsilon:
            return random.choice(valid_actions)
        
        processed_state = self.process_state(state)
        q_values = self.model.predict(processed_state.reshape(1, -1), verbose=0)[0]
        valid_q_values = [(q_values[action], action) for action in valid_actions]
        return max(valid_q_values, key=lambda x: x[0])[1]

    def store_experience(self, state, action, reward, next_state, done):
        self.replay_buffer.append((state, action, reward, next_state, done))

    def train(self):
        if len(self.replay_buffer) < self.batch_size:
            return

        mini_batch = random.sample(self.replay_buffer, self.batch_size)
        states = np.array([self.process_state(exp[0]) for exp in mini_batch])
        next_states = np.array([self.process_state(exp[3]) for exp in mini_batch])
        
        # Use batch prediction for efficiency
        current_q_values = self.model.predict(states, verbose=0)
        next_q_values = self.model.predict(next_states, verbose=0)

        x = []
        y = []

        for i, (state, action, reward, next_state, done) in enumerate(mini_batch):
            if done:
                target = reward
            else:
                next_valid_actions = [j for j, val in enumerate(next_state) if val == 0]
                if next_valid_actions:
                    max_next_q = max(next_q_values[i][action] for action in next_valid_actions)
                    target = reward + self.gamma * max_next_q
                else:
                    target = reward
            
            current_q = current_q_values[i].copy()
            current_q[action] = target
            x.append(self.process_state(state))
            y.append(current_q)

        self.model.fit(np.array(x), np.array(y), batch_size=self.batch_size, epochs=1, verbose=0)

def train_agent(episodes=20000, model_path='model6_retrain_retrain.h5'):
    agent = SQNAgent(model_path=model_path)
    history = {'wins': 0, 'losses': 0, 'draws': 0}
    initial_epsilon = 1.0
    min_epsilon = 0.01
    decay_episodes = 1000
    smartness=0
    for episode in range(episodes):
        if episode % 2000 == 0 and episode > 0:
            if episode%2000==0:
                agent.epsilon = 1.0
            smartness = min(0.8, episode / (episodes * 0.9))
            print("Resetting epsilon to 1.0")
        else:
            epsilon_decay = (initial_epsilon - min_epsilon) / decay_episodes
            agent.epsilon = max(min_epsilon, initial_epsilon - (episode % 2000) * epsilon_decay)
        
        
        game = TicTacToe(smartMovePlayer1=smartness)
        state = np.array(game.board)
        
        game.player1_move()
        state = np.array(game.board)
        
        while True:
            valid_actions = game.empty_positions()
            if not valid_actions:
                history['draws'] += 1
                break
                
            action = agent.select_action(state, valid_actions)
            game.make_move(action, player=2)
            
            reward = 0
            if game.current_winner == 2:
                reward = 1.0
                history['wins'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            
            game.player1_move()
            if game.current_winner == 1:
                reward = -1.0
                history['losses'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            elif game.is_full():
                reward = min(-0.1, -0.5 * smartness)
                history['draws'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            
            agent.store_experience(state, action, reward, game.board, False)
            state = np.array(game.board)
        
        agent.train()
        
        if episode % 100 == 0:
            win_rate = history['wins'] / (episode + 1)
            print(f"Episode: {episode}, Win Rate: {win_rate:.2f}, Epsilon: {agent.epsilon:.3f}, smartmove: {smartness:.2f}")
            print(f"Wins: {history['wins']}, Losses: {history['losses']}, Draws: {history['draws']}")
        
        if episode % 1000 == 0:
            try:
                agent.model.save(f'model6_retrain_special_retrain_retrain_episode_{episode}.h5')
                print(f"Checkpoint saved at episode {episode}")
            except Exception as e:
                print(f"Error saving checkpoint: {e}")

    # Save the final model
    try:
        agent.model.save("model6_retrain_special_retrain.h5")
        print(f'Final model saved as model6_retrain_special_retrain.h5')
    except Exception as e:
        print(f"Error saving final model: {e}")


train_agent()

model6_retrain_retrain.h5 exists. Loading the model.
Episode: 0, Win Rate: 0.00, Epsilon: 1.000, smartmove: 0.00
Wins: 0, Losses: 0, Draws: 1
Checkpoint saved at episode 0
Episode: 100, Win Rate: 0.34, Epsilon: 0.901, smartmove: 0.00
Wins: 34, Losses: 56, Draws: 11
Episode: 200, Win Rate: 0.33, Epsilon: 0.802, smartmove: 0.00
Wins: 67, Losses: 108, Draws: 26
Episode: 300, Win Rate: 0.38, Epsilon: 0.703, smartmove: 0.00
Wins: 114, Losses: 144, Draws: 43
Episode: 400, Win Rate: 0.39, Epsilon: 0.604, smartmove: 0.00
Wins: 155, Losses: 192, Draws: 54
Episode: 500, Win Rate: 0.38, Epsilon: 0.505, smartmove: 0.00
Wins: 188, Losses: 245, Draws: 68
Episode: 600, Win Rate: 0.40, Epsilon: 0.406, smartmove: 0.00
Wins: 242, Losses: 270, Draws: 89
Episode: 700, Win Rate: 0.42, Epsilon: 0.307, smartmove: 0.00
Wins: 295, Losses: 295, Draws: 111
Episode: 800, Win Rate: 0.43, Epsilon: 0.208, smartmove: 0.00
Wins: 343, Losses: 325, Draws: 133
Episode: 900, Win Rate: 0.43, Epsilon: 0.109, smartmove: 0.00

Episode: 1000, Win Rate: 0.44, Epsilon: 0.010, smartmove: 0.00
Wins: 437, Losses: 374, Draws: 190
Checkpoint saved at episode 1000
Episode: 1100, Win Rate: 0.46, Epsilon: 0.010, smartmove: 0.00
Wins: 501, Losses: 389, Draws: 211
Episode: 1200, Win Rate: 0.47, Epsilon: 0.010, smartmove: 0.00
Wins: 565, Losses: 409, Draws: 227
Episode: 1300, Win Rate: 0.48, Epsilon: 0.010, smartmove: 0.00
Wins: 627, Losses: 428, Draws: 246
Episode: 1400, Win Rate: 0.50, Epsilon: 0.010, smartmove: 0.00
Wins: 702, Losses: 441, Draws: 258
Episode: 1500, Win Rate: 0.51, Epsilon: 0.010, smartmove: 0.00
Wins: 764, Losses: 460, Draws: 277
Episode: 1600, Win Rate: 0.52, Epsilon: 0.010, smartmove: 0.00
Wins: 830, Losses: 476, Draws: 295
Episode: 1700, Win Rate: 0.52, Epsilon: 0.010, smartmove: 0.00
Wins: 893, Losses: 493, Draws: 315
Episode: 1800, Win Rate: 0.52, Epsilon: 0.010, smartmove: 0.00
Wins: 942, Losses: 523, Draws: 336
Episode: 1900, Win Rate: 0.52, Epsilon: 0.010, smartmove: 0.00
Wins: 984, Losses: 547

Resetting epsilon to 1.0
Episode: 2000, Win Rate: 0.51, Epsilon: 1.000, smartmove: 0.11
Wins: 1030, Losses: 579, Draws: 392
Checkpoint saved at episode 2000
Episode: 2100, Win Rate: 0.50, Epsilon: 0.901, smartmove: 0.11
Wins: 1053, Losses: 644, Draws: 404
Episode: 2200, Win Rate: 0.49, Epsilon: 0.802, smartmove: 0.11
Wins: 1081, Losses: 699, Draws: 421
Episode: 2300, Win Rate: 0.49, Epsilon: 0.703, smartmove: 0.11
Wins: 1124, Losses: 743, Draws: 434
Episode: 2400, Win Rate: 0.48, Epsilon: 0.604, smartmove: 0.11
Wins: 1157, Losses: 790, Draws: 454
Episode: 2500, Win Rate: 0.48, Epsilon: 0.505, smartmove: 0.11
Wins: 1193, Losses: 837, Draws: 471
Episode: 2600, Win Rate: 0.47, Epsilon: 0.406, smartmove: 0.11
Wins: 1220, Losses: 893, Draws: 488
Episode: 2700, Win Rate: 0.46, Epsilon: 0.307, smartmove: 0.11
Wins: 1249, Losses: 939, Draws: 513
Episode: 2800, Win Rate: 0.46, Epsilon: 0.208, smartmove: 0.11
Wins: 1285, Losses: 987, Draws: 529
Episode: 2900, Win Rate: 0.46, Epsilon: 0.109, smar

Episode: 3000, Win Rate: 0.46, Epsilon: 0.010, smartmove: 0.11
Wins: 1372, Losses: 1058, Draws: 571
Checkpoint saved at episode 3000
Episode: 3100, Win Rate: 0.46, Epsilon: 0.010, smartmove: 0.11
Wins: 1431, Losses: 1076, Draws: 594
Episode: 3200, Win Rate: 0.46, Epsilon: 0.010, smartmove: 0.11
Wins: 1476, Losses: 1108, Draws: 617
Episode: 3300, Win Rate: 0.46, Epsilon: 0.010, smartmove: 0.11
Wins: 1516, Losses: 1143, Draws: 642
Episode: 3400, Win Rate: 0.46, Epsilon: 0.010, smartmove: 0.11
Wins: 1558, Losses: 1180, Draws: 663
Episode: 3500, Win Rate: 0.45, Epsilon: 0.010, smartmove: 0.11
Wins: 1592, Losses: 1216, Draws: 693
Episode: 3600, Win Rate: 0.45, Epsilon: 0.010, smartmove: 0.11
Wins: 1628, Losses: 1252, Draws: 721
Episode: 3700, Win Rate: 0.45, Epsilon: 0.010, smartmove: 0.11
Wins: 1676, Losses: 1279, Draws: 746
Episode: 3800, Win Rate: 0.45, Epsilon: 0.010, smartmove: 0.11
Wins: 1719, Losses: 1313, Draws: 769
Episode: 3900, Win Rate: 0.45, Epsilon: 0.010, smartmove: 0.11
Wins

Resetting epsilon to 1.0
Episode: 4000, Win Rate: 0.45, Epsilon: 1.000, smartmove: 0.22
Wins: 1792, Losses: 1388, Draws: 821
Checkpoint saved at episode 4000
Episode: 4100, Win Rate: 0.44, Epsilon: 0.901, smartmove: 0.22
Wins: 1817, Losses: 1452, Draws: 832
Episode: 4200, Win Rate: 0.44, Epsilon: 0.802, smartmove: 0.22
Wins: 1841, Losses: 1515, Draws: 845
Episode: 4300, Win Rate: 0.44, Epsilon: 0.703, smartmove: 0.22
Wins: 1872, Losses: 1574, Draws: 855
Episode: 4400, Win Rate: 0.43, Epsilon: 0.604, smartmove: 0.22
Wins: 1900, Losses: 1632, Draws: 869
Episode: 4500, Win Rate: 0.43, Epsilon: 0.505, smartmove: 0.22
Wins: 1923, Losses: 1695, Draws: 883
Episode: 4600, Win Rate: 0.42, Epsilon: 0.406, smartmove: 0.22
Wins: 1949, Losses: 1753, Draws: 899
Episode: 4700, Win Rate: 0.42, Epsilon: 0.307, smartmove: 0.22
Wins: 1988, Losses: 1792, Draws: 921
Episode: 4800, Win Rate: 0.42, Epsilon: 0.208, smartmove: 0.22
Wins: 2022, Losses: 1836, Draws: 943
Episode: 4900, Win Rate: 0.42, Epsilon: 0.

...

Resetting epsilon to 1.0
Episode: 16000, Win Rate: 0.29, Epsilon: 1.000, smartmove: 0.80
Wins: 4606, Losses: 7037, Draws: 4358
Checkpoint saved at episode 16000
Episode: 16100, Win Rate: 0.29, Epsilon: 0.901, smartmove: 0.80
Wins: 4615, Losses: 7120, Draws: 4366
Episode: 16200, Win Rate: 0.29, Epsilon: 0.802, smartmove: 0.80
Wins: 4618, Losses: 7201, Draws: 4382
Episode: 16300, Win Rate: 0.28, Epsilon: 0.703, smartmove: 0.80
Wins: 4630, Losses: 7270, Draws: 4401
Episode: 16400, Win Rate: 0.28, Epsilon: 0.604, smartmove: 0.80
Wins: 4635, Losses: 7347, Draws: 4419
Episode: 16500, Win Rate: 0.28, Epsilon: 0.505, smartmove: 0.80
Wins: 4642, Losses: 7418, Draws: 4441
Episode: 16600, Win Rate: 0.28, Epsilon: 0.406, smartmove: 0.80
Wins: 4656, Losses: 7483, Draws: 4462
Episode: 16700, Win Rate: 0.28, Epsilon: 0.307, smartmove: 0.80
Wins: 4673, Losses: 7528, Draws: 4500
Episode: 16800, Win Rate: 0.28, Epsilon: 0.208, smartmove: 0.80
Wins: 4684, Losses: 7572, Draws: 4545
Episode: 16900, Win Rat

Episode: 17000, Win Rate: 0.28, Epsilon: 0.010, smartmove: 0.80
Wins: 4737, Losses: 7636, Draws: 4628
Checkpoint saved at episode 17000
Episode: 17100, Win Rate: 0.28, Epsilon: 0.010, smartmove: 0.80
Wins: 4757, Losses: 7664, Draws: 4680
Episode: 17200, Win Rate: 0.28, Epsilon: 0.010, smartmove: 0.80
Wins: 4777, Losses: 7689, Draws: 4735
Episode: 17300, Win Rate: 0.28, Epsilon: 0.010, smartmove: 0.80
Wins: 4796, Losses: 7719, Draws: 4786
Episode: 17400, Win Rate: 0.28, Epsilon: 0.010, smartmove: 0.80
Wins: 4811, Losses: 7751, Draws: 4839
Episode: 17500, Win Rate: 0.28, Epsilon: 0.010, smartmove: 0.80
Wins: 4833, Losses: 7782, Draws: 4886
Episode: 17600, Win Rate: 0.28, Epsilon: 0.010, smartmove: 0.80
Wins: 4856, Losses: 7805, Draws: 4940
Episode: 17700, Win Rate: 0.28, Epsilon: 0.010, smartmove: 0.80
Wins: 4875, Losses: 7831, Draws: 4995
Episode: 17800, Win Rate: 0.28, Epsilon: 0.010, smartmove: 0.80
Wins: 4902, Losses: 7853, Draws: 5046
Episode: 17900, Win Rate: 0.27, Epsilon: 0.010, 

Resetting epsilon to 1.0
Episode: 18000, Win Rate: 0.27, Epsilon: 1.000, smartmove: 0.80
Wins: 4946, Losses: 7922, Draws: 5133
Checkpoint saved at episode 18000
Episode: 18100, Win Rate: 0.27, Epsilon: 0.901, smartmove: 0.80
Wins: 4956, Losses: 7999, Draws: 5146
Episode: 18200, Win Rate: 0.27, Epsilon: 0.802, smartmove: 0.80
Wins: 4965, Losses: 8073, Draws: 5163
Episode: 18300, Win Rate: 0.27, Epsilon: 0.703, smartmove: 0.80
Wins: 4975, Losses: 8145, Draws: 5181
Episode: 18400, Win Rate: 0.27, Epsilon: 0.604, smartmove: 0.80
Wins: 4984, Losses: 8217, Draws: 5200
Episode: 18500, Win Rate: 0.27, Epsilon: 0.505, smartmove: 0.80
Wins: 4989, Losses: 8287, Draws: 5225
Episode: 18600, Win Rate: 0.27, Epsilon: 0.406, smartmove: 0.80
Wins: 4996, Losses: 8354, Draws: 5251
Episode: 18700, Win Rate: 0.27, Epsilon: 0.307, smartmove: 0.80
Wins: 5009, Losses: 8405, Draws: 5287
Episode: 18800, Win Rate: 0.27, Epsilon: 0.208, smartmove: 0.80
Wins: 5028, Losses: 8453, Draws: 5320
Episode: 18900, Win Rat

Episode: 19000, Win Rate: 0.27, Epsilon: 0.010, smartmove: 0.80
Wins: 5063, Losses: 8516, Draws: 5422
Checkpoint saved at episode 19000
Episode: 19100, Win Rate: 0.27, Epsilon: 0.010, smartmove: 0.80
Wins: 5086, Losses: 8540, Draws: 5475
Episode: 19200, Win Rate: 0.27, Epsilon: 0.010, smartmove: 0.80
Wins: 5109, Losses: 8563, Draws: 5529
Episode: 19300, Win Rate: 0.27, Epsilon: 0.010, smartmove: 0.80
Wins: 5129, Losses: 8582, Draws: 5590
Episode: 19400, Win Rate: 0.27, Epsilon: 0.010, smartmove: 0.80
Wins: 5153, Losses: 8605, Draws: 5643
Episode: 19500, Win Rate: 0.26, Epsilon: 0.010, smartmove: 0.80
Wins: 5166, Losses: 8630, Draws: 5705
Episode: 19600, Win Rate: 0.26, Epsilon: 0.010, smartmove: 0.80
Wins: 5193, Losses: 8650, Draws: 5758
Episode: 19700, Win Rate: 0.26, Epsilon: 0.010, smartmove: 0.80
Wins: 5211, Losses: 8677, Draws: 5813
Episode: 19800, Win Rate: 0.26, Epsilon: 0.010, smartmove: 0.80
Wins: 5245, Losses: 8702, Draws: 5854
Episode: 19900, Win Rate: 0.26, Epsilon: 0.010, 

Final model saved as model6_retrain_special_retrain.h5


In [29]:
import numpy as np
import random
import os
from collections import deque
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from TicTacToe import TicTacToe

class SQNAgent:
    def __init__(self, state_size=9, action_size=9, gamma=0.99, model_path='model6_retrain.h5'):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = gamma  
        self.learning_rate = 0.0005
        self.epsilon = 1.0
        self.epsilon_min = 0.01  # Keeping original minimum epsilon
        self.epsilon_decay = 0.9994
        self.batch_size = 64
        self.replay_buffer = deque(maxlen=50000)
        self.model_path = model_path

        if os.path.exists(model_path):
            try:
                print(f'{model_path} exists. Loading the model.')
                self.model = load_model(model_path, compile=False)
                self.model.compile(
                    loss=MeanSquaredError(),
                    optimizer=Adam(learning_rate=self.learning_rate)
                )
            except Exception as e:
                print(f"Error loading model: {e}")
                print("Creating new model instead.")
                self.model = self._build_model()
        else:
            print(f'{model_path} does not exist. Starting new training.')
            self.model = self._build_model()

    def _build_model(self):
        model = Sequential([
            Dense(64, input_dim=self.state_size, activation='relu'),
            Dense(64, activation='relu'),
            Dense(self.action_size, activation='linear')
        ])
        model.compile(
            loss=MeanSquaredError(),
            optimizer=Adam(learning_rate=self.learning_rate)
        )
        return model

    def process_state(self, state):
        processed_state = np.array(state).copy()
        processed_state[processed_state == 0] = -1
        processed_state[processed_state == 1] = 0
        processed_state[processed_state == 2] = 1
        return processed_state

    def select_action(self, state, valid_actions):
        if not valid_actions:
            return None

        if random.random() < self.epsilon:
            return random.choice(valid_actions)
        
        processed_state = self.process_state(state)
        q_values = self.model.predict(processed_state.reshape(1, -1), verbose=0)[0]
        
        # Prioritize center and corner moves when Q-values are close
        if len(valid_actions) > 1:
            preferred_moves = [4] if 4 in valid_actions else []  # Center
            preferred_moves.extend([0, 2, 6, 8] for pos in [0, 2, 6, 8] if pos in valid_actions)  # Corners
            
            max_q = max(q_values[action] for action in valid_actions)
            good_actions = [action for action in valid_actions 
                          if q_values[action] > max_q - 0.1]
            
            for preferred in preferred_moves:
                if preferred in good_actions:
                    return preferred

        valid_q_values = [(q_values[action], action) for action in valid_actions]
        return max(valid_q_values, key=lambda x: x[0])[1]

    def store_experience(self, state, action, reward, next_state, done):
        self.replay_buffer.append((state, action, reward, next_state, done))

    def train(self):
        if len(self.replay_buffer) < self.batch_size:
            return

        mini_batch = random.sample(self.replay_buffer, self.batch_size)
        states = np.array([self.process_state(exp[0]) for exp in mini_batch])
        next_states = np.array([self.process_state(exp[3]) for exp in mini_batch])
        
        current_q_values = self.model.predict(states, verbose=0)
        next_q_values = self.model.predict(next_states, verbose=0)

        x = []
        y = []

        for i, (state, action, reward, next_state, done) in enumerate(mini_batch):
            if done:
                target = reward
            else:
                next_valid_actions = [j for j, val in enumerate(next_state) if val == 0]
                if next_valid_actions:
                    max_next_q = max(next_q_values[i][action] for action in next_valid_actions)
                    target = reward + self.gamma * max_next_q
                else:
                    target = reward
            
            current_q = current_q_values[i].copy()
            current_q[action] = target
            x.append(self.process_state(state))
            y.append(current_q)

        self.model.fit(np.array(x), np.array(y), batch_size=self.batch_size, epochs=1, verbose=0)

def train_agent(episodes=20000, model_path='model6_retrain_special_retrain.h5'):
    agent = SQNAgent(model_path=model_path)
    history = {'wins': 0, 'losses': 0, 'draws': 0}
    initial_epsilon = 1.0
    min_epsilon = 0.01
    decay_episodes = 1000
    smartness = 0
    
    for episode in range(episodes):
        # Original exploration strategy
        if episode % 5000== 0 and episode > 0:
            if episode % 5000== 0:
                agent.epsilon = 1.0
            smartness = min(0.8, episode / (episodes * 0.8))  # Cap at 0.8 smartness
            print("Resetting epsilon to 1.0")
        else:
            agent.epsilon = max(min_epsilon, agent.epsilon*agent.epsilon_decay)
        
        game = TicTacToe(smartMovePlayer1=smartness)
        state = np.array(game.board)
        
        game.player1_move()
        state = np.array(game.board)
        
        while True:
            valid_actions = game.empty_positions()
            if not valid_actions:
                history['draws'] += 1
                break
                
            action = agent.select_action(state, valid_actions)
            game.make_move(action, player=2)
            
            reward = 0
            if game.current_winner == 2:
                reward = 1.0
                history['wins'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            
            game.player1_move()
            if game.current_winner == 1:
                reward = -1.0
                history['losses'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            elif game.is_full():
                reward = min(-0.1, -0.5 * smartness)
                history['draws'] += 1
                agent.store_experience(state, action, reward, game.board, True)
                break
            
            agent.store_experience(state, action, reward, game.board, False)
            state = np.array(game.board)
        
        agent.train()
        
        if episode % 100 == 0:
            recent_games = 1000
            recent_start = max(0, episode - recent_games)
            recent_total = episode - recent_start + 1
            win_rate = (history['wins'] - (history['wins'] // recent_total)) / recent_total
            print(f"Episode: {episode}, Recent Win Rate: {win_rate:.2f}, "
                  f"Epsilon: {agent.epsilon:.3f}, Smartness: {smartness:.2f}")
            print(f"Wins: {history['wins']}, Losses: {history['losses']}, Draws: {history['draws']}")
        
        if episode % 1000 == 0:
            try:
                agent.model.save(f'model6_retrain_optimized_episode_{episode}.h5')
                print(f"Checkpoint saved at episode {episode}")
            except Exception as e:
                print(f"Error saving checkpoint: {e}")

    try:
        agent.model.save("modle16_final_final.h5")
        print(f'Final model saved as modle16_final_final.h5')
    except Exception as e:
        print(f"Error saving final model: {e}")

if __name__ == "__main__":
    train_agent()

model6_retrain_special_retrain.h5 exists. Loading the model.
Episode: 0, Recent Win Rate: 0.00, Epsilon: 0.999, Smartness: 0.00
Wins: 1, Losses: 0, Draws: 0
Checkpoint saved at episode 0
Episode: 100, Recent Win Rate: 0.31, Epsilon: 0.941, Smartness: 0.00
Wins: 31, Losses: 60, Draws: 10
Episode: 200, Recent Win Rate: 0.35, Epsilon: 0.886, Smartness: 0.00
Wins: 71, Losses: 105, Draws: 25
Episode: 300, Recent Win Rate: 0.36, Epsilon: 0.835, Smartness: 0.00
Wins: 108, Losses: 150, Draws: 43
Episode: 400, Recent Win Rate: 0.36, Epsilon: 0.786, Smartness: 0.00
Wins: 146, Losses: 203, Draws: 52
Episode: 500, Recent Win Rate: 0.38, Epsilon: 0.740, Smartness: 0.00
Wins: 188, Losses: 245, Draws: 68
Episode: 600, Recent Win Rate: 0.37, Epsilon: 0.697, Smartness: 0.00
Wins: 225, Losses: 285, Draws: 91
Episode: 700, Recent Win Rate: 0.40, Epsilon: 0.657, Smartness: 0.00
Wins: 278, Losses: 319, Draws: 104
Episode: 800, Recent Win Rate: 0.39, Epsilon: 0.618, Smartness: 0.00
Wins: 316, Losses: 363, D

Episode: 1000, Recent Win Rate: 0.41, Epsilon: 0.548, Smartness: 0.00
Wins: 414, Losses: 436, Draws: 151
Checkpoint saved at episode 1000
Episode: 1100, Recent Win Rate: 0.46, Epsilon: 0.516, Smartness: 0.00
Wins: 464, Losses: 474, Draws: 163
Episode: 1200, Recent Win Rate: 0.52, Epsilon: 0.486, Smartness: 0.00
Wins: 521, Losses: 505, Draws: 175
Episode: 1300, Recent Win Rate: 0.58, Epsilon: 0.458, Smartness: 0.00
Wins: 584, Losses: 524, Draws: 193
Episode: 1400, Recent Win Rate: 0.63, Epsilon: 0.431, Smartness: 0.00
Wins: 633, Losses: 559, Draws: 209
Episode: 1500, Recent Win Rate: 0.69, Epsilon: 0.406, Smartness: 0.00
Wins: 691, Losses: 582, Draws: 228
Episode: 1600, Recent Win Rate: 0.75, Epsilon: 0.383, Smartness: 0.00
Wins: 748, Losses: 609, Draws: 244
Episode: 1700, Recent Win Rate: 0.81, Epsilon: 0.360, Smartness: 0.00
Wins: 810, Losses: 636, Draws: 255
Episode: 1800, Recent Win Rate: 0.87, Epsilon: 0.339, Smartness: 0.00
Wins: 869, Losses: 667, Draws: 265
Episode: 1900, Recent 

Episode: 2000, Recent Win Rate: 0.97, Epsilon: 0.301, Smartness: 0.00
Wins: 971, Losses: 726, Draws: 304
Checkpoint saved at episode 2000
Episode: 2100, Recent Win Rate: 1.03, Epsilon: 0.283, Smartness: 0.00
Wins: 1031, Losses: 753, Draws: 317
Episode: 2200, Recent Win Rate: 1.09, Epsilon: 0.267, Smartness: 0.00
Wins: 1088, Losses: 779, Draws: 334
Episode: 2300, Recent Win Rate: 1.15, Epsilon: 0.251, Smartness: 0.00
Wins: 1149, Losses: 801, Draws: 351
Episode: 2400, Recent Win Rate: 1.22, Epsilon: 0.237, Smartness: 0.00
Wins: 1220, Losses: 820, Draws: 361
Episode: 2500, Recent Win Rate: 1.28, Epsilon: 0.223, Smartness: 0.00
Wins: 1285, Losses: 844, Draws: 372
Episode: 2600, Recent Win Rate: 1.34, Epsilon: 0.210, Smartness: 0.00
Wins: 1345, Losses: 868, Draws: 388
Episode: 2700, Recent Win Rate: 1.41, Epsilon: 0.198, Smartness: 0.00
Wins: 1412, Losses: 886, Draws: 403
Episode: 2800, Recent Win Rate: 1.48, Epsilon: 0.186, Smartness: 0.00
Wins: 1479, Losses: 902, Draws: 420
Episode: 2900,

Episode: 3000, Recent Win Rate: 1.62, Epsilon: 0.165, Smartness: 0.00
Wins: 1624, Losses: 934, Draws: 443
Checkpoint saved at episode 3000
Episode: 3100, Recent Win Rate: 1.68, Epsilon: 0.155, Smartness: 0.00
Wins: 1681, Losses: 957, Draws: 463
Episode: 3200, Recent Win Rate: 1.74, Epsilon: 0.146, Smartness: 0.00
Wins: 1744, Losses: 971, Draws: 486
Episode: 3300, Recent Win Rate: 1.81, Epsilon: 0.138, Smartness: 0.00
Wins: 1808, Losses: 986, Draws: 507
Episode: 3400, Recent Win Rate: 1.87, Epsilon: 0.130, Smartness: 0.00
Wins: 1868, Losses: 1002, Draws: 531
Episode: 3500, Recent Win Rate: 1.93, Epsilon: 0.122, Smartness: 0.00
Wins: 1929, Losses: 1024, Draws: 548
Episode: 3600, Recent Win Rate: 2.00, Epsilon: 0.115, Smartness: 0.00
Wins: 1998, Losses: 1037, Draws: 566
Episode: 3700, Recent Win Rate: 2.06, Epsilon: 0.108, Smartness: 0.00
Wins: 2062, Losses: 1057, Draws: 582
Episode: 3800, Recent Win Rate: 2.12, Epsilon: 0.102, Smartness: 0.00
Wins: 2125, Losses: 1075, Draws: 601
Episode:

Episode: 4000, Recent Win Rate: 2.26, Epsilon: 0.091, Smartness: 0.00
Wins: 2261, Losses: 1106, Draws: 634
Checkpoint saved at episode 4000
Episode: 4100, Recent Win Rate: 2.32, Epsilon: 0.085, Smartness: 0.00
Wins: 2325, Losses: 1124, Draws: 652
Episode: 4200, Recent Win Rate: 2.38, Epsilon: 0.080, Smartness: 0.00
Wins: 2389, Losses: 1139, Draws: 673
Episode: 4300, Recent Win Rate: 2.45, Epsilon: 0.076, Smartness: 0.00
Wins: 2453, Losses: 1152, Draws: 696
Episode: 4400, Recent Win Rate: 2.50, Epsilon: 0.071, Smartness: 0.00
Wins: 2507, Losses: 1176, Draws: 718
Episode: 4500, Recent Win Rate: 2.57, Epsilon: 0.067, Smartness: 0.00
Wins: 2574, Losses: 1188, Draws: 739
Episode: 4600, Recent Win Rate: 2.63, Epsilon: 0.063, Smartness: 0.00
Wins: 2639, Losses: 1205, Draws: 757
Episode: 4700, Recent Win Rate: 2.69, Epsilon: 0.060, Smartness: 0.00
Wins: 2699, Losses: 1222, Draws: 780
Episode: 4800, Recent Win Rate: 2.75, Epsilon: 0.056, Smartness: 0.00
Wins: 2755, Losses: 1242, Draws: 804
Epis

...

Episode: 16000, Recent Win Rate: 5.76, Epsilon: 0.549, Smartness: 0.80
Wins: 5774, Losses: 6815, Draws: 3412
Checkpoint saved at episode 16000
Episode: 16100, Recent Win Rate: 5.77, Epsilon: 0.517, Smartness: 0.80
Wins: 5778, Losses: 6891, Draws: 3432
Episode: 16200, Recent Win Rate: 5.78, Epsilon: 0.487, Smartness: 0.80
Wins: 5791, Losses: 6956, Draws: 3454
Episode: 16300, Recent Win Rate: 5.80, Epsilon: 0.458, Smartness: 0.80
Wins: 5807, Losses: 7017, Draws: 3477
Episode: 16400, Recent Win Rate: 5.81, Epsilon: 0.432, Smartness: 0.80
Wins: 5819, Losses: 7069, Draws: 3513
Episode: 16500, Recent Win Rate: 5.82, Epsilon: 0.406, Smartness: 0.80
Wins: 5831, Losses: 7126, Draws: 3544
Episode: 16600, Recent Win Rate: 5.83, Epsilon: 0.383, Smartness: 0.80
Wins: 5840, Losses: 7183, Draws: 3578
Episode: 16700, Recent Win Rate: 5.85, Epsilon: 0.360, Smartness: 0.80
Wins: 5858, Losses: 7237, Draws: 3606
Episode: 16800, Recent Win Rate: 5.86, Epsilon: 0.339, Smartness: 0.80
Wins: 5875, Losses: 728

Episode: 17000, Recent Win Rate: 5.89, Epsilon: 0.301, Smartness: 0.80
Wins: 5905, Losses: 7401, Draws: 3695
Checkpoint saved at episode 17000
Episode: 17100, Recent Win Rate: 5.91, Epsilon: 0.284, Smartness: 0.80
Wins: 5919, Losses: 7461, Draws: 3721
Episode: 17200, Recent Win Rate: 5.92, Epsilon: 0.267, Smartness: 0.80
Wins: 5934, Losses: 7517, Draws: 3750
Episode: 17300, Recent Win Rate: 5.94, Epsilon: 0.251, Smartness: 0.80
Wins: 5951, Losses: 7568, Draws: 3782
Episode: 17400, Recent Win Rate: 5.96, Epsilon: 0.237, Smartness: 0.80
Wins: 5968, Losses: 7615, Draws: 3818
Episode: 17500, Recent Win Rate: 5.97, Epsilon: 0.223, Smartness: 0.80
Wins: 5985, Losses: 7666, Draws: 3850
Episode: 17600, Recent Win Rate: 5.99, Epsilon: 0.210, Smartness: 0.80
Wins: 6004, Losses: 7712, Draws: 3885
Episode: 17700, Recent Win Rate: 6.00, Epsilon: 0.198, Smartness: 0.80
Wins: 6016, Losses: 7770, Draws: 3915
Episode: 17800, Recent Win Rate: 6.02, Epsilon: 0.186, Smartness: 0.80
Wins: 6034, Losses: 781

Episode: 18000, Recent Win Rate: 6.05, Epsilon: 0.165, Smartness: 0.80
Wins: 6064, Losses: 7910, Draws: 4027
Checkpoint saved at episode 18000
Episode: 18100, Recent Win Rate: 6.06, Epsilon: 0.156, Smartness: 0.80
Wins: 6076, Losses: 7957, Draws: 4068
Episode: 18200, Recent Win Rate: 6.08, Epsilon: 0.147, Smartness: 0.80
Wins: 6093, Losses: 7995, Draws: 4113
Episode: 18300, Recent Win Rate: 6.10, Epsilon: 0.138, Smartness: 0.80
Wins: 6111, Losses: 8041, Draws: 4149
Episode: 18400, Recent Win Rate: 6.12, Epsilon: 0.130, Smartness: 0.80
Wins: 6130, Losses: 8085, Draws: 4186
Episode: 18500, Recent Win Rate: 6.14, Epsilon: 0.122, Smartness: 0.80
Wins: 6152, Losses: 8124, Draws: 4225
Episode: 18600, Recent Win Rate: 6.16, Epsilon: 0.115, Smartness: 0.80
Wins: 6172, Losses: 8168, Draws: 4261
Episode: 18700, Recent Win Rate: 6.17, Epsilon: 0.109, Smartness: 0.80
Wins: 6187, Losses: 8202, Draws: 4312
Episode: 18800, Recent Win Rate: 6.19, Epsilon: 0.102, Smartness: 0.80
Wins: 6205, Losses: 824

Episode: 19000, Recent Win Rate: 6.22, Epsilon: 0.091, Smartness: 0.80
Wins: 6232, Losses: 8321, Draws: 4448
Checkpoint saved at episode 19000
Episode: 19100, Recent Win Rate: 6.23, Epsilon: 0.085, Smartness: 0.80
Wins: 6244, Losses: 8360, Draws: 4497
Episode: 19200, Recent Win Rate: 6.25, Epsilon: 0.080, Smartness: 0.80
Wins: 6261, Losses: 8404, Draws: 4536
Episode: 19300, Recent Win Rate: 6.27, Epsilon: 0.076, Smartness: 0.80
Wins: 6285, Losses: 8439, Draws: 4577
Episode: 19400, Recent Win Rate: 6.29, Epsilon: 0.071, Smartness: 0.80
Wins: 6302, Losses: 8477, Draws: 4622
Episode: 19500, Recent Win Rate: 6.32, Epsilon: 0.067, Smartness: 0.80
Wins: 6328, Losses: 8514, Draws: 4659
Episode: 19600, Recent Win Rate: 6.33, Epsilon: 0.063, Smartness: 0.80
Wins: 6345, Losses: 8551, Draws: 4705
Episode: 19700, Recent Win Rate: 6.35, Epsilon: 0.060, Smartness: 0.80
Wins: 6367, Losses: 8582, Draws: 4752
Episode: 19800, Recent Win Rate: 6.38, Epsilon: 0.056, Smartness: 0.80
Wins: 6388, Losses: 861

Final model saved as modle16_final_final.h5
